<a href="https://colab.research.google.com/github/Palaeoprot/pFind/blob/main/pFind_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**pFind Analysis** Scripts


# Global Varibles

In [1]:
#!pip install Biopython

## Import Packages

In [2]:
# Data manipulation:
import pandas as pd
pd.set_option('display.max_rows', 500)  # Adjust the value as needed
from itertools import islice
#import sketch
from collections import  defaultdict, Counter
from prettytable import PrettyTable

# Debugging:
import traceback

# File I/O and path handling:
import os
import copy, os
import re
import requests
import json
from datetime import datetime
from typing import Dict, Any

# Numerical analysis and statistics:
import numpy as np
from scipy import stats
from statistics import mode, multimode  # Consider removing if unused

# Third-party modules (Commented out modules can be imported as needed):
#from Bio import SeqIO  # Only import if used
# from icecream import ic  # Import on demand if needed

# Google Colab specific for mounting Google Drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Sequence dictionaries

In [4]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/FASTA_dicts/sequences_dict.json'

# Reading the JSON file and converting it back into a dictionary
with open(file_path, 'r') as json_file:
    sequences_dict = json.load(json_file)


# Print only the first few items of the dictionary
for key in list(sequences_dict.keys())[:10]:  # Adjust the number as needed
    print(f"{key}: {sequences_dict[key]}")

# # Or, if you want to check a specific entry
# key_to_check = 'COL1A1-Bos_tau'
# if key_to_check in sequences_dict:
#     print(f"{key_to_check}: {sequences_dict[key_to_check]}")
# else:
#     print(f"Key '{key_to_check}' not found in the dictionary.")

COL1A1-Bos_tau: {'GNC': 'COL1A1', 'species': 'Bos taurus', 'offset': -7, 'sequence': 'STGISVPGPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEGGPQGPRGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGSKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPAGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGAPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGAPGKDGVRGLTGPIGPPGPAGAPGDKGEAGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPAGPPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGSKGPRGETGPAGRPGEVGPPGPPGPAGEKGAPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPMGPPGLAGPPGESGREGAPGAEGSPGRDGSPGAKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPIGPVGARG

In [50]:
# # non_sequences_dict = {
# #     'TRYUP-Sus_sco': {'species': 'Bos taurus','offset': 0, 'sequence': 'IVGGYTCAANSIPYQVSLNSGSHFCGGSLINSQWVVSAAHCYKSRIQVRLGEHNIDVLEGNEQFINAAKIITHPNFNGNTLDNDIMLIKLSSPATLNSRVATVSLPRSCAAAGTECLISGWGNTKSSGSSYPSLLQCLKAPVLSDSSCKSSYPGQITGNMICVGFLEGGKDSCQGDSGGPVVCNGQLQGIVSWGYGCAQKNKPGVYTKVCNYVNWIQQTIAAN'},
# #     'AHSG-Bos_tau' : {'species': 'Bos taurus','offset': 0, 'sequence':'IPLDPVAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLANCSVRQQTQHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNDSRVVHAVEVALATFNAESNGSYLQLVEISRAQFVPLPVSVSVEFAVAATDCIAKEVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVRVTCTLFQTQPVIPQPQPDGAEAEAPSAVPDAAGPTPSAAGPPVASVVVGPSVVAVPLPLHRAHYDLRHTFSGVASVESSSGEAFHVGKTPIVGQPSIPGGPVRLCPGRIRYFKI'},
# #     'AHSG-Bub_bub' : {'species': 'Bos taurus','offset': 0, 'sequence':'IPLDPVAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLANCSVRQQTEHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNDSRVVHAVEVALATFNAQSNGSYLQLVEISRAQFVPLPASVSVEFAVAATDCIAKDVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVAVTCTLFQTQPVILQPQPDGAEAGAPSAVPDAAGPAPSAAGPPVASVVVGPSVVAVPLPLHRAHYDLRHTFSGVASVESASGEAFHVGKTPIVGQPSVPGGPVRLCPGRIRYFKI'},
# #     'AHSG-Equ_cab' : {'species': 'Bos taurus','offset': 1, 'sequence':' LPNGLSPAYRQLNCDDPETEQAALLAVDYINSHIHQGYKHVLNQIDKVQVWAQPTGESFKLEIDTLETTCHALDPTPLANCSVRQLTQHAVEGDCDVRLLKQNGQFSVSFVKCKSSPDSAEDVRKVCLDCPLLAPLNDTRVVHAVEAALAAFNAQNNGSYFQLVEISRAQLVPLPVSVHVEFAVAATDCVAKEVIDPAKCNLLAEKQYGFCKATLTEKVGGEDVAVTCTVFQTQPVVLLPQPDGPDVGVPGPVADAVTPAPSPADLPVASLVVGPVVVAASQLPPPVHRAHYDLRHAFAGVGSGESASGEAFHVEKPPKVAHPNTAAAAGPVVRPCPGRIRYFKII'},
# #     'AHSG-Ovi_are' : {'species': 'Bos taurus','offset': 1, 'sequence':'IPLDPIAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLVNCSVRQQTEHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNNSQVVHAAEVALATFNAQNNGSYFQLVEISRAQFVPLPGSVSVEFAVAATDCIAKEVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVTVTCTLFQTQPVIPQPQPEGAEAGAPSAVPDAAVPDAAVPAPSAAGLPVGSVVAGPSVVAVPLPLHRAHYDLRHTFSGVASVESASGEAFHVGKTPIVGQPSVPGGPVHLCPGRIRYFKI'},
# #     'AHSG-Cap_hir' : {'species': 'Bos taurus','offset': 1, 'sequence':'IPLDPIAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLANCSVRQQTEHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNNSQVVHAAEVALATFNAQNNGSYFQLVEISRAQFVPLPVSVSVEFAVAATDCIAKEVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVAVTCTLFQTQPVIPQPQPEGAEAGAPSAVPDAAVPAPSAAGLPVGSVVAGPSVVAVALPLHRAHYDLRHTFSGVASVESASGEAFHVGKTPIVGQPSVPGGPVHLCPGRIRYFKI'},
# #     'AHSG-Equ_asn' : {'species': 'Bos taurus','offset': 1, 'sequence':' LPNGLSPAYRQLNCDDPETEQAALLAVDYINSHIHQGYKHVLNQIDKVQVWAQPTGESFKLEIDTLETTCHALDPTPLANCSVRQLTQHAVEGDCDVRLLKQNGQFSVSFVKCKSSPDSAEDVRKVCLDCPLLAPLNDTRVVHAVEAALAAFNAQNNGSYFQLVEISRAQLVPLPVSVHVEFAVAATDCVAKEVIDPAKCNLLAEKQYGFCKATLTEKVGGEDVAVTCTVFQTQPVVPLPQPDGPDVGVPGPVADAATPEPSPADLPVASLVVGPVVVAAPQLPPPVHRAHYDLRHAFAGVGSGESASGEAFHVEKPPKVAHPNTAAAAGPVVRPCPGRIRYFKI'},
# #     'TPM1-Cam_dro' : {'species': 'Bos taurus','offset': 0, 'sequence':'MDAIKKKMQMLKLDKENAIDRAEQAEADKKQAEDRCKQLEEEQQALQKKLKGTEDEVEKYSESVKDAQEKLEQAEKKATDAEADVASLNRRIQLVEEELDRAQERLATALQKLEEAEKAADESERGMKVIENRAMKDEEKMELQEMQLKEAKHIAEDSDRKYEEVARKLVILEGELERSEERAEVAESRARQLEEELRTMDQALKSLMASEEEEGLGLPNSICCHSHNFALLFSPPPLPHCAISTVSPTVSSHPPPPHTPCSKCGDLEEELKIVTNNLKSLEAQADKYSTKEDKYEEEIKLLEEKLKEAETRAEFAERSVAKLEKTIDDLEDEVYAQKMKYKAISEELDNALNDITSL'},
# #     'KS1-Hom_sap' : {'species': 'Bos taurus','offset': 0, 'sequence':'SRQFSSRSGYRSGGGFSSGSAGIINYQRRTTSSSTRRSGGGGGRFSSCGGGGGSFGAGGGFGSRSLVNLGGSKSISISVARGGGRGSGFGGGYGGGGFGGGGFGGGGFGGGGIGGGGFGGFGSGGGGFGGGGFGGGGYGGGYGPVCPPGGIQEVTINQSLLQPLNVEIDPEIQKVKSREREQIKSLNNQFASFIDKVRFLEQQNQVLQTKWELLQQVDTSTRTHNLEPYFESFINNLRRRVDQLKSDQSRLDSELKNMQDMVEDYRNKYEDEINKRTNAENEFVTIKKDVDGAYMTKVDLQAKLDNLQQEIDFLTALYQAELSQMQTQISETNVILSMDNNRSLDLDSIIAEVKAQYEDIAQKSKAEAESLYQSKYEELQITAGRHGDSVRNSKIEISELNRVIQRLRSEIDNVKKQISNLQQSISDAEQRGENALKDAKNKLNDLEDALQQAKEDLARLLRDYQELMNTKLALDLEIATYRTLLEGEESRMSGECAPNVSVSVSTSHTTISGGGSRGGGGGGYGSGGSSYGSGGGSYGSGGGGGGGRGSYGSGGSSYGSGGGSYGSGGGGGGHGSYGSGSSSGGYRGGSGGGGGGSSGGRGSGGGSSGGSIGGRGSSSGGVKSSGGSSSVKFVSTTYSGVTR'},
# #     'KS9-Hom_sap' : {'species': 'Bos taurus','offset': 0, 'sequence':'SRQFSSRSGYRSGGGFSSGSAGIINYQRRTTSSSTRRSGGGGGRFSSCGGGGGSFGAGGGFGSRSLVNLGGSKSISISVARGGGRGSGFGGGYGGGGFGGGGFGGGGFGGGGIGGGGFGGFGSGGGGFGGGGFGGGGYGGGYGPVCPPGGIQEVTINQSLLQPLNVEIDPEIQKVKSREREQIKSLNNQFASFIDKVRFLEQQNQVLQTKWELLQQVDTSTRTHNLEPYFESFINNLRRRVDQLKSDQSRLDSELKNMQDMVEDYRNKYEDEINKRTNAENEFVTIKKDVDGAYMTKVDLQAKLDNLQQEIDFLTALYQAELSQMQTQISETNVILSMDNNRSLDLDSIIAEVKAQYEDIAQKSKAEAESLYQSKYEELQITAGRHGDSVRNSKIEISELNRVIQRLRSEIDNVKKQISNLQQSISDAEQRGENALKDAKNKLNDLEDALQQAKEDLARLLRDYQELMNTKLALDLEIATYRTLLEGEESRMSGECAPNVSVSVSTSHTTISGGGSRGGGGGGYGSGGSSYGSGGGSYGSGGGGGGGRGSYGSGGSSYGSGGGSYGSGGGGGGHGSYGSGSSSGGYRGGSGGGGGGSSGGRGSGGGSSGGSIGGRGSSSGGVKSSGGSSSVKFVSTTYSGVTR'},
# #     'KS2-Hom_sap' : {'species': 'Bos taurus','offset': 0, 'sequence':'SRQFSSRSGYRSGGGFSSGSAGIINYQRRTTSSSTRRSGGGGGRFSSCGGGGGSFGAGGGFGSRSLVNLGGSKSISISVARGGGRGSGFGGGYGGGGFGGGGFGGGGFGGGGIGGGGFGGFGSGGGGFGGGGFGGGGYGGGYGPVCPPGGIQEVTINQSLLQPLNVEIDPEIQKVKSREREQIKSLNNQFASFIDKVRFLEQQNQVLQTKWELLQQVDTSTRTHNLEPYFESFINNLRRRVDQLKSDQSRLDSELKNMQDMVEDYRNKYEDEINKRTNAENEFVTIKKDVDGAYMTKVDLQAKLDNLQQEIDFLTALYQAELSQMQTQISETNVILSMDNNRSLDLDSIIAEVKAQYEDIAQKSKAEAESLYQSKYEELQITAGRHGDSVRNSKIEISELNRVIQRLRSEIDNVKKQISNLQQSISDAEQRGENALKDAKNKLNDLEDALQQAKEDLARLLRDYQELMNTKLALDLEIATYRTLLEGEESRMSGECAPNVSVSVSTSHTTISGGGSRGGGGGGYGSGGSSYGSGGGSYGSGGGGGGGRGSYGSGGSSYGSGGGSYGSGGGGGGHGSYGSGSSSGGYRGGSGGGGGGSSGGRGSGGGSSGGSIGGRGSSSGGVKSSGGSSSVKFVSTTYSGVTRR'},
# #     'ALB-Bos_tau' : {'species': 'Bos taurus','offset': 0, 'sequence':'MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGLVLIAFSQYLQQCPFDEHVKLVNELTEFAKTCVADESHAGCEKSLHTLFGDELCKVASLRETYGDMADCCEKQEPERNECFLSHKDDSPDLPKLKPDPNTLCDEFKADEKKFWGKYLYEIARRHPYFYAPELLYYANKYNGVFQECCQAEDKGACLLPKIETMREKVLASSARQRLRCASIQKFGERALKAWSVARLSQKFPKAEFVEVTKLVTDLTKVHKECCHGDLLECADDRADLAKYICDNQDTISSKLKECCDKPLLEKSHCIAEVEKDAIPENLPPLTADFAEDKDVCKNYQEAKDAFLGSFLYEYSRRHPEYAVSVLLRLAKEYEATLEECCAKDDPHACYSTVFDKLKHLVDEPQNLIKQNCDQFEKLGEYGFQNALIVRYTRKVPQVSTPTLVEVSRSLGKVGTRCCTKPESERMPCTEDYLSLILNRLCVLHEKTPVSEKVTKCCTESLVNRRPCFSALTPDETYVPKAFDEKLFTFHADICTLPDTEKQIKKQTALVELLKHKPKATEEQLKTVMENFVAFVDKCCAADDKEACFAVEGPKLVVSTQTALA'},
# #     'ALB-Equ_cab' : {'species': 'Bos taurus','offset': 0, 'sequence':'MKWVTFVSLLFLFSSAYSRGVLRRDTHKSEIAHRFNDLGEKHFKGLVLVAFSQYLQQCPFEDHVKLVNEVTEFAKKCAADESAENCDKSLHTLFGDKLCTVATLRATYGELADCCEKQEPERNECFLTHKDDHPNLPKLKPEPDAQCAAFQEDPDKFLGKYLYEVARRHPYFYGPELLFHAEEYKADFTECCPADDKLACLIPKLDALKERILLSSAKERLKCSSFQNFGERAVKAWSVARLSQKFPKADFAEVSKIVTDLTKVHKECCHGDLLECADDRADLAKYICEHQDSISGKLKACCDKPLLQKSHCIAEVKEDDLPSDLPALAADFAEDKEICKHYKDAKDVFLGTFLYEYSRRHPDYSVSLLLRIAKTYEATLEKCCAEADPPACYRTVFDQFTPLVEEPKSLVKKNCDLFEEVGEYDFQNALIVRYTKKAPQVSTPTLVEIGRTLGKVGSRCCKLPESERLPCSENHLALALNRLCVLHEKTPVSEKITKCCTDSLAERRPCFSALELDEGYVPKEFKAETFTFHADICTLPEDEKQIKKQSALAELVKHKPKATKEQLKTVLGNFSAFVAKCCGREDKEACFAEEGPKLVASSQLALA'},
# #     'ALB-Equ_asn' : {'species': 'Bos taurus','offset': 0, 'sequence':'MKWVTFVSLLFLFSSAYFRGVLRRDTHKSEIAHRFNDLGEKHFKGLVLVAFSQYLQQCPFEDHVKLVNEVTEFAKKCAADESAENCDKSLHTLFGDKLCTVATLRATYGELADCCEKQEPERNECFLTHKDDHPNLPKLKPEPDAQCAAFQEDPDKFLGKYLYEVARRHPYFYGPELLFHAEEYKADFTECCPADDKAGCLIPKLDALKERILLSSAKERLKCSSFQKFGERAFKAWSVARLSQKFPKADFAEVSKIVTDLTKVHKECCHGDLLECADDRADLTKYICEHQDSISGKLKACCDKPLLQKSHCIAEVKEDDLPSDLPALAADFAEDKEICKHYKDAKDVFLGTFLYEYSRRHPDYSVSLLLRIAKTYEATLEKCCAEADPPACYATVFDQFTPLVEEPKSLVKKNCDLFEEVGEYDFQNALIVRYTKKAPQVSTPTLVEIGRTLGKVGSRCCKLPESERLPCSENHLALALNRLCVLHEKTPVSEKITKCCTDSLAERRPCFSALELDEGYIPKEFKAETFTFHADICTLPEDEKQIKKQSALAELVKHKPKATKEQLKTVLGNFSAFVAKCCGAEDKEACFAEEGPKLVASSQLALA'},
# #     'ALB-Sus_scr' : {'species': 'Bos taurus','offset': 0, 'sequence':'MKWVTFISLLFLFSSAYSRGVFRRDTYKSEIAHRFKDLGEQYFKGLVLIAFSQHLQQCPYEEHVKLVREVTEFAKTCVADESAENCDKSIHTLFGDKLCAIPSLREHYGDLADCCEKEEPERNECFLQHKNDNPDIPKLKPDPVALCADFQEDEQKFWGKYLYEIARRHPYFYAPELLYYAIIYKDVFSECCQAADKAACLLPKIEHLREKVLTSAAKQRLKCASIQKFGERAFKAWSLARLSQRFPKADFTEISKIVTDLAKVHKECCHGDLLECADDRADLAKYICENQDTISTKLKECCDKPLLEKSHCIAEAKRDELPADLNPLEHDFVEDKEVCKNYKEAKHVFLGTFLYEYSRRHPDYSVSLLLRIAKIYEATLEDCCAKEDPPACYATVFDKFQPLVDEPKNLIKQNCELFEKLGEYGFQNALIVRYTKKVPQVSTPTLVEVARKLGLVGSRCCKRPEEERLSCAEDYLSLVLNRLCVLHEKTPVSEKVTKCCTESLVNRRPCFSALTPDETYKPKEFVEGTFTFHADLCTLPEDEKQIKKQTALVELLKHKPHATEEQLRTVLGNFAAFVQKCCAAPDHEACFAVEGPKFVIEIRGILA'},

# #     }

# sequences_dict = {
#     'COL1A1-Bos_tau': {'species': 'Bos taurus','offset': -7, 'sequence': 'STGISVPGPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEGGPQGPRGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGSKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPAGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGAPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGAPGKDGVRGLTGPIGPPGPAGAPGDKGEAGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPAGPPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGSKGPRGETGPAGRPGEVGPPGPPGPAGEKGAPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPMGPPGLAGPPGESGREGAPGAEGSPGRDGSPGAKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPIGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGSPGKDGLNGLPGPIGPPGPRGRTGDAGPAGPPGPPGPPGPPGPPSGGYDLSFLPQPPQEK'},
#     'COL1A1-Bub_bub': {'species': 'Bubalus bubalis','offset': -7, 'sequence': 'STGISVPGPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEGGPQGPRGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGSKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPAGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGAPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGAPGKDGVRGLTGPIGPPGPAGAPGDKGEAGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDSGPPGPAGPAGPPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGSKGPRGETGPAGRPGEVGPPGPPGPSGEKGAPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPMGPPGLAGPPGESGREGAPGAEGSPGRDGSPGAKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPIGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGSPGKDGLNGLPGPIGPPGPRGRTGDAGPAGPPGPPGPPGPPGPPSGGYDLSFLPQPPQEKAHDGGRYYRA'},
#     'COL1A1-Equ_asi': {'species': 'Equus asinus','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEAGPQGARGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGNKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPTGLPGPPGERGGPGARGFPGADGVAGPKGPAGERGAPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGESGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGSNGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGSPGKDGVRGLTGPIGPPGPAGAPGDKGETGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPAGPPGPIGSVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPVGKEGGKGPRGETGPAGRPGEAGPPGPPGPAGEKGSPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPVGPPGLAGPPGESGREGSPGAEGSPGRDGSPGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGEAGPAGPAGPIGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAPGKDGLNGLPGPIGPPGPRGRTGDAGPVGPPGPPGPPGPPGPPSAGFDFSFLPQPPQEKSHDGGR'},
#     'COL1A1-Equ_cab': {'species': 'Equus caballus','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEAGPQGARGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGNKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPTGLPGPPGERGGPGARGFPGADGVAGPKGPAGERGAPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGESGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGSNGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGSPGKDGVRGLTGPIGPPGPAGAPGDKGETGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPAGPPGPIGSVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPVGKEGGKGPRGETGPAGRPGEAGPPGPPGPAGEKGSPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPVGPPGLAGPPGESGREGSPGAEGSPGRDGSPGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGEAGPAGPAGPIGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAPGKDGLNGLPGPIGPPGPRGRTGDAGPVGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEKSHDGGR'},




#     'COL1A1-Hip_amp': {'species': 'Hippopotamus amphibius','offset': 0, 'sequence': 'GPMGPSGPRGIPGPPGAPGPQGF------PGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPSGPQGARGIPGTAGIPGMKGHRGFSGIDGAKGDAGPAGPK---------GAPGQMGPRGIPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEAGPQGARGSEGPQGVRGEPGPPGPAGAA---------------GANGAPGIAGAPGFPGARGPSGPQGPSGPSGPKGNSGEPGAPGSKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPAGIPGPPGERGGPGSRGFPGADGVAGPKGPAGERGSPGPAGPKGSPGEAGRPGEAGIPGAKGITGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGIPGPAGPPGEAGKPGEQGVPGDIGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGIQGMPGERGAAGIPGPKGDRGDAGPKGADGSPGKDGVRGITGPIGPPGPAGAPGDKGEAGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAK----------GTGPPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPVGK------RGETGPAGRPGEVGPPGPPGPAGEKGAPGADGPAGAPGTPGPQGIAGQRGVVGIPGQRGERGFPGIPGPSGEPGKQGPSGTSGERGPPGPMGPPGIAGPPGESGREGAPGAEGSPGRDGSPGAKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPIGPVGARGAAGPQGPRGDKGETGEQGDRGIKGHRGFSGIQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGSPGKDGINGIPGPIGPPGPR-----------PGPPGPPGPPGPP'},
#     'COL1A1-Ovi_Cap': {'species': 'Ovis Capra','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEAGPQGPRGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGSKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPAGLPGPPGERGGPGSRGFPGSDGVAGPKGPAGERGAPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGAPGKDGVRGLTGPIGPPGPAGAPGDKGETGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPAGPPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGSKGPRGETGPAGRAGEVGPPGPPGPAGEKGAPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPMGPPGLAGPPGESGREGAPGAEGSPGRDGAPGAKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPIGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGTPGKDGLNGLPGPIGPPGPRGRTGDAGPAGPPGPPGPPGPPGPPSGGYDLSFLPQPPQEKAHDGGR'},
#     'COL1A1-Pog_vit': {'species': 'Pogona vitticeps','offset': -0, 'sequence': 'GPMGPSGPRGPPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPAGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDPGPAGPKGEPGSPGENGAPGQVGPRGLPGERGRPGAPGPAGARGNDGSPGAAGPPGPTGPAGPPGFPGAVGPKGETGAQGSRGSEGPQGARGEPGPPGPAGAAGPSGNPGTDGQPGAKGSPGAPGIAGAPGFPGARGPAGPQGPAGAPGPKGNSGEPGAPGNKGDTGAKGETGPAGVQGPPGPPGEEGKRGSRGEPGPAGLPGPAGERGAPGSRGFPGADGIAGPKGPPGERGSPGPAGPKGSTGEAGRPGEPGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPAGPPGARGQAGVMGFPGPKGAAGEPGKPGERGAPGAVGAVGAPGKDGEVGAQGPPGPTGPAGERGEQGPSGAPGFQGLPGPAGAPGEAGKPGEQGVPGDVGAPGPAGARGERGFPGERGVQGAAGPPGPRGANGSPGNDGAKGDAGAPGAPGNQGPPGLQGMPGERGAAGLPGAKGDRGDTGPKGLDGAPGKDGLRGLTGPIGPPGPAGAPGDKGEAGPQGPAGPTGARGAPGDRGEPGPAGPAGFAGPPGTDGQPGAKGEPGDAGAKGDAGPPGPAGATGPAGPAGPIGAPGPKGARGSPGPPGATGFPGAAGRVGPPGPSGNIGLPGPPGPVGKEGAKGPRGETGPAGRPGEAGPAGPPGPPGEKGSPGSDGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPMGPPGLAGPPGEAGREGSPGAEGAPGRDGPAGPKGDRGETGPAGPPGAPGAPGAPGPMGPAGKNGDRGETGPAGPAGPAGPAGARGAAGPQGPRGDKGETGEQGDRGMKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAPGKDGLNGLPGPIGPPGPRGRTGDVGPAGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEKAHDGR'},
#     'COL1A1-Alg_sin': {'species': 'Alligator sinensis','offset': -0, 'sequence': 'GPMGPAGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPAGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGEPGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGPSGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAAGAKGETGPQGARGSEGPQGARGEPGPPGPAGAAGPAGNPGADGQPGAKGATGAPGIAGAPGFPGARGPSGPQGPSGAPGPKGNSGEPGAPGNKGDTGAKGEPGPAGVQGPPGPAGEEGKRGARGEPGPAGLPGPAGERGAPGSRGFPGADGIAGPKGPPGERGSPGAVGPKGSPGEAGRPGEPGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKPGERGAPGPPGAVGAAGKDGEAGAQGPPGPTGPAGERGEQGPAGAPGFQGLPGPAGPPGEAGKPGEQGVPGDAGAPGPAGARGERGFPGERGVQGPPGPQGPRGANGAPGNDGAKGDAGAPGAPGNQGPPGLQGMPGERGAAGLPGAKGDRGDPGPKGADGAPGKDGLRGLTGPIGPPGPAGAPGDKGEAGPPGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGETGDAGAKGDAGPPGPAGPTGAPGPAGAVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNIGLPGPPGPSGKEGGKGPRGETGPAGRPGEPGPAGPPGPPGEKGSPGADGPIGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGSPGERGPPGPMGPPGLAGPPGEAGREGAPGAEGAPGRDGAAGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKNGDRGETGPAGPAGPPGPAGARGPAGPQGPRGDKGETGEQGDRGMKGHRGFSGLQGPPGPPGQPGEQGPSGASGPAGPRGPPGSAGAAGKDGLNGLPGPIGPPGPRGRTGDVGPVGPPGPPGPPGPPGPPSGGFDFSFLPQ'},
#     'COL1A1-Lox_afr': {'species': 'Loxodonta africana','offset': -8, 'sequence': 'SAGGISVPGPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPVSPTGPAGPPGFPGAVGAKGEAGPQGARGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPAGPQGPSGAPGPKGNSGEPGAPGSKGDAGAKGEPGPVGIQGPPGPAGEEGKRGARGEPGPTGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGSPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGAAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGSNGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGSPGKDGPRGLTGPIGPPGPAGAPGDKGEAGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPTGAPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGGKGPRGETGPAGRPGEVGPPGPPGPAGEKGSPGADGPAGAPGTPGPQGIGGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGSSGERGPPGPAGPPGLAGPPGESGREGAPGAEGSPGRDGSPGPKGDRGETGPSGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPAGPAGVRGPAGPQGPRGDKGETGEQGDRGLKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAPGKDGLNGLPGPIGPPGPRGRTGDAGPVGPPGPPGPPGPPGPPSGAFDFSFLPQPPQEKAHDGGRYYRA'},
#     'COL1A1-Sus_Scr': {'species': 'Sus scrofa','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGPPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEAGPQGARGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGGKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGSKGDTGAKGEPGPTGVQGPPGPAGEEGKRGARGEPGPAGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGSPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGAPGKDGVRGLTGPIGPPGPAGAPGDKGETGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPTGPPGPIGSVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGSKGPRGETGPAGRPGEAGPPGPPGPAGEKGSPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGPSGERGPPGPMGPPGLAGPPGESGREGAPGAEGSPGRDGAPGPKGDRGESGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPVGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAPGKDGLNGLPGPIGPPGPRGRTGDAGPVGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEKAHDGGR'},
#     'COL1A1-Mam_mam': {'species': 'Mammuthus primigenius','offset': -8, 'sequence': 'SAGGISVPGPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEAGPQGARGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPAGPQGPSGAPGPKGNSGEPGAPGSKGDAGAKGEPGPIGIQGPPGPAGEEGKRGARGEPGPTGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGSPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGAAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGSNGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGSPGKDGPRGLTGPIGPPGPAGAPGDKGEAGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPTGAPGPIGNVGAPGAKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGGKGPRGETGPAGRPGEVGPPGPPGPAGEKGSPGADGPAGAPGTPGPQGIGGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGSSGERGPPGPAGPPGLAGPPGESGREGAPGAEGSPGRDGSPGPKGDRGETGPSGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPAGPAGVRGPAGPQGPRGDKGETGEQGDRGLKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAPGKDGLNGLPGPPGPPGPRGRTGDAGPVGPPGPPGPPGPPGPPSGAFDFSFLPQPPQEK'},
#     'COL1A1-Tur_tur': {'species': 'Tursiops truncatus','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGTPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGSTGPAGPPGFPGAVGAKGEAGPQGSRGSEGPQGVRGEPGPPGPAGATGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGNKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPAGLPGPPGERGGPGSRGFPGADGLAGPKGPAGERGAPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGSRGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAAGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPSGPPGEAGKPGEQGAPGDIGAPGPSGARGERGFPGERGVQGPSGPAGPRGSNGPPGNDGAKGDAGAPGAPGNQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGAPGKDGVRGLTGPIGPPGPAGAPGDKGETGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGAPGSAGPTGPPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPSGPAGKEGSKGPRGETGPAGRAGEVGPPGPPGPAGEKGAPGADGPAGSPGSPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPMGPPGLAGPPGESGREGAPGAEGSPGRDGSPGPKGDRGETGPAGPPGAPGSPGAPGPVGPAGKSGDRGETGPAGPAGPIGPAGARGPTGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGTPGKDGLNGLPGPIGPPGPRGRTGDAGPAGPPGPPGPPGPPGPPSGGYDFSFLPQPPQEKAQDGGR'},


#     'COL1A2-Lox_afr': {'species': 'Loxodonta africana','offset': -5, 'sequence': 'KGIGLGPGPMGLMGPRGPPGATGPPGSPGFQGPPGEPGEPGQTGPAGSRGPAGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQIGARGLPGERGRVGGPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGEIGPVGNPGPSGPAGPRGEAGLPGVSGPVGPPGNPGANGLAGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGSKGEPGSAGPQGPPGPSGEEGKRGSSGEAGSAGPAGPPGLRGGPGSRGLPGADGRAGVMGPPGSRGASGPAGVRGPSGDSGRPGEPGVMGPRGLPGSPGNVGPAGKEGPAGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPAGDPGKNGDKGHAGLAGPRGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPSGTAGEAGKPGERGIPGEFGLPGPAGPRGERGPPGQSGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPGGLPGERGAAGIPGGKGEKGETGLRGDTGNTGRDGARGAPGAVGAPGPAGATGDRGEAGPAGSAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQAGAKGERGTKGPKGENGPVGPTGPVGAAGPAGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPAGITGPPGPPGAAGKEGLRGPRGDQGPVGRTGETGASGPPGFAGEKGSSGEPGTAGPPGTPGPQGILGPPGILGLPGSRGERGLPGVAGAVGEPGPLGIAGPPGARGPPGAVGSPGVNGAPGEAGRDGNPGSDGPPGRDGLPGHKGERGYPGNAGPVGTAGAPGPQGPLGPAGKHGNRGEPGPAGSVGPVGAVGPRGPSGPQGARGDKGEAGDKGPRGLPGFKGHNGLQGLPGLAGQHGDQGSPGSVGPAGPRGPAGPSGPVGKDGRPGHAGAVGPAGVRGSQGSQGPSGPPGPPGPPGPPGPSGGGYDFGYDGDFYRA'},
#     'COL1A2-Bos_mut': {'species': 'Bubalus bubalis','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGASGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAIGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGISGPPGPPGPAGKEGLRGPRGDQGPVGRSGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGAPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPVGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYEFGFDGDFYRA'},
#     'COL1A2-Bos_tau': {'species': 'Bos taurus','offset': -4, 'sequence': 'GGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAIGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGISGPPGPPGPAGKEGLRGPRGDQGPVGRSGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGAPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPVGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYEFGFDGDFYR'},
#     'COL1A2-Equ_asi': {'species': 'Equus asinus','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHKGLDGLKGQPGAPGVKGEPGAPGENGTPGQAGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPAGAAGATGARGLVGEPGPAGSKGESGNKGEPGAAGPQGPPGPSGEEGKRGPNGEPGSTGPAGPPGLRGSPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGEPGKPGDKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGARGERGPPGESGAAGPAGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGEIGNPGRDGARGAPGAVGAPGPAGANGDRGEAGAAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGVTGFPGAAGRTGPPGPSGISGPPGPPGAAGKEGLRGPRGDQGPVGRAGETGASGPPGFAGEKGPSGEPGTAGPPGTPGPQGLLGAPGILGLPGSRGERGLPGVAGSLGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGSDGPPGRDGQPGHKGERGYPGNAGPVGAVGAPGPHGPVGPTGKHGNRGEPGPVGSVGPVGAVGPRGPSGPQGVRGDKGEPGDKGPRGLPGIKGHNGLQGLPGLAGQHGDQGAPGSVGPAGPRGPAGPTGPVGKDGRSGQPGTVGPAGVRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYDGDFYRA'},
#     'COL1A2-EquxPrz': {'species': 'EquusxPrezwal','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQAGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPAGAAGATGARGLVGEPGPAGSKGESGNKGEPGAAGPQGPPGPSGEEGKRGPNGEPGSTGPAGPPGLRGSPGSRGLPGADGRAGVMGPAGSRGASGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGEPGKPGDKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGARGERGPPGESGAAGPAGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGEIGNPGRDGARGAPGAVGAPGPAGANGDRGEAGAAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGVTGFPGAAGRTGPPGPSGISGPPGPPGAAGKEGLRGPRGDQGPVGRAGETGASGPPGFAGEKGPSGEPGTAGPPGTPGPQGLLGAPGILGLPGSRGERGLPGVAGSLGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGSDGPPGRDGQPGHKGERGYPGNAGPVGAVGAPGPHGPVGPTGKHGHRGEPGPVGSVGPVGAVGPRGPSGPQGVRGDKGEPGDKGPRGLPGIKGHNGLQGLPGLAGQHGDQGAPGSVGPAGPRGPAGPTGPVGKDGRSGQPGTVGPAGVRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYDGDFYRA'},
#     'COL1A2-Equ_cab': {'species': 'Equus caballus','offset': 0, 'sequence': '---------GPMGIMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGAR--------AGEDGHPGKPGRPGERGVVGPQGARGFPGTPGIPGFK --GHNGIDGIKGQPGAPGVKGEPGAPGENGTPGQAGARGIPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGEIGPVGNPGPAGPAGPRGEVGIPGISGPVGPPGNPGANGITGAKGAAGIPGVAGAPGIPGPRGIPGPAGAAGATGARGIVGEPGPAGSKGESGNKGEPGAAGPQGPPGPSGEEGKRGPNGEPGSTGPAGPPGIR------GIPGADGRAGVMGPAGSRGASGPAGVRGPNGDSGRPGEPGIMGPRGFPGSPGNIGPAGKEGPVGIPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGEPGKPGDKGDAGIAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGIPGPAGTAGEVGKPGERGIPGEFGIPGPAGAR---GPPGESGAAGPAGPIGSRGPSGPPGPDGNKGEPGVIGAPGTAGPSGPSGIPGERGAAGIPGGK---------GEIGNPGRDGARGAPGAVGAPGPAGANGDRGEAGAAGPAGPAGPR------GEVGPAGPNGFAGPAGAAGQPGAKGER---GPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGVTGFPGAAGRTGPPGPSGISGPPGPPGAAGK-------GDQGPVGRAGETGASGPPGFAGEKGPSGEPGTAGPPGTPGPQGIIGAPGIIGIPGSR---GIPGVAGSIGEPGPIGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGSDGPPGR----------GYPGNAGPVGAVGAPGPHGPVGPTGK---RGEPGPVGSVGPVGAVGPRGPSGPQGVR------------------GHNGIQGIPGIAGQHGDQGAPGSVGPAGPRGPAGPTGPVGKD--SGQPGTVGPAGVRGSQ'},
#     'COL1A2-Cap_hir': {'species': 'Capra hircus','offset': -7, 'sequence': 'QFDGKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPAGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKAGEKGHAGLAGPRGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDVGSPGRDGARGAPGAVGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPAGISGPPGPPGPAGKEGLRGPRGDQGPVGRTGEPGAAGPPGFVGEKGPSGEPGTAGPPGTPGPQGFLGPPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPTGKHGSRGEPGPVGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPTGPAGKDGRTGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGFDGDFYRA'},
#     'COL1A2-Ovi_are': {'species': 'Ovis aries','offset': -7, 'sequence': 'QFDGKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPAGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKAGEKGHAGLAGPRGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDVGSPGRDGARGAPGAVGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPAGISGPPGPPGPAGKEGLRGPRGDQGPVGRTGEPGAAGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGAPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPTGKHGSRGEPGPVGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPTGPAGKDGRTGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGFDGDFYRA'},
#     'COL1A2-Bub_bub': {'species': 'Bubalus bubalis','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGPAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPSGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGVRGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAVGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPAGISGPPGPPGPAGKEGLRGPRGDQGPVGRTGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGLPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPVGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGSVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGFDGDFYRA'},

#     'COL1A2-Pog_vit': {'species': 'Pogona vitticeps','offset': -12, 'sequence':'QYDISKSADYGPGPMGIMGPRGPPGPNGPPGSPGYQGPPGEPGEPGQTGPQGARGPMGPPGKAGEDGHPGKPGRPGERGPSGPQGARGFPGTPGLPGFKGIRGHNGLDGRKGEPGTAGNKGEPGAAGENGTPGQPGARGAPGERGRVGANGPVGARGADGSNGPTGPAGPIGAAGPPGFPGGPGAKGEIGAVGNPGPSGPAGPRGEAGLPGASGPVGPAGNPGTNGINGAKGAAGLPGAAGPPGLPGPRGILGAPGPSGPAGARGLTGEPGPAGSKGETGSKGEPGSAGPAGPQGPSGEEGKRGSTGEPGSSGLPGPAGLRGVPGSRGLPGSDGRAGVIGPAGNRGATGLPGAKGPNGDTGRPGEPGLMGARGLPGQSGSPGPAGKEGAVGLPGADGRAGPIGPAGPRGEPGNIGFPGPKGPNGEPGKPGDRGNVGVTGSRGAPGPDGNNGAQGAPGVIGNPGPKGDAGPAGPPGFQGLPGPSGPVGEAGKPGERGIHGEFGLPGPAGSRGERGPPGESGATGPVGPIGSRGPSGAPGPDGNKGEPGNVGAAGAPGPAGSNGIPGERGAAGMPGGKGEKGPTGFRGEPGATGRDGSRGIPGAVGAPGPAGGAGERGEAGPAGQAGPPGSRGIPGERGEVGPAGPQGFAGPPGAAGQPGAKGERGQRGPKGDVGPQGAIGAVGNAGPAGPLGAAGPAGTRGDSGPPGMTGAPGSAGRPGPPGPAGISGPPGPPGPAGKEGIRGPRGDTGPTGRSGETGISGPPGFQGEKGPSGESGAAGPPGTPGPQGILGAPGILGLPGSRGERGLPGIAGAVGEPGPLGIAGPPGARGAPGPIGSPGVNGAPGEAGRDGNPGNDGAPGRNGAPGYKGERGAPGSPGPVGAAGAPGAHGIVGPVGKTGNRGEPGPVGPAGSTGPAGPRGLPGPQGPRGEKGVVGDKGPRGLPGLKGHNGLQGLPGIAGHHGEQGPSGIPGPAGPRGPAGPSGPAGKDGRNGLPGPVGPAGVRGSQGSQGPSGPPGPPGLPGPAGPNGGGYEIGYDMEYYRA'},
#     'COL1A2-Cro_por': {'species': 'Crocodylus porosus','offset': -12, 'sequence':'QYDGAKAGDYGQGPMGLMGPRGPPGTSGPPGPPGFQGPHGEPGEPGQTGAQGPRGPPGPPGKAGEDGHPGKSGRSGERGVSGPQGARGFPGTPGLPGFKGIRGHNGLDGQKGQAGTPGIKGESGAPGENGTPGQSGARGLPGERGRIGAPGPAGARGSDGSGGPTGPAGPIGAAGPPGFPGAPGAKGEIGAAGNVGPSGPAGPRGEAGLPGSSGPAGPPGNPGANGLAGAKGATGLPGVAGAPGLPGPRGIPGPPGPAGAAGTRGLVGEPGPAGAKGESGNKGEPGAAGPPGPAGPSGEEGKKGTTGEPGSSGPPGPAGLRGVPGSRGLPGADGRAGVMGPAGSRGATGPAGAKGPTGDNGRPGEPGLMGPRGLPGQPGSSGPAGKEGPVGFPGADGRVGPIGPAGARGEPGNIGFPGPKGPTGDPGKPGDRGHAGLAGARGAPGPEGNNGAQGPPGVAGNPGAKGEQGPAGPPGFQGLPGPSGPAGEAGKPGERGMAGEFGAPGPAGSRGERGPPGESGAVGPVGSIGSRGPSGPPGTDGNKGEPGNVGNAGTAGPSGASGAPGERGIAGIPGPKGEKGATGLRGDTGATGRDGARGAPGAIGAPGPAGGAGERGEGGPAGPAGPSGARGIPGERGEPGPAGPTGFAGPAGAAGQPGAKGERGPKGPKGESGPQGSVGPVGASGPSGPVGAAGPAGPRGDGGPPGVTGFPGAAGRTGPPGPSGITGPPGPSGPAGKDGVRGPRGDTGPVGRTGEQGIVGPPGFSGEKGPSGESGAAGPPGSPGPQGILGAPGVLGLPGSRGERGLPGISGATGEPGPLGIAGPSGARGPPGPVGSAGLNGAPGEAGRDGNPGHDGAPGRDGAPGFKGERGAPGNNGPAGAVGAPGAHGQVGPVGKPGNRGDPGPVGPSGPAGAFGARGLAGPQGARGEKGETGDKGHRGMPGFKGHNGLQGLPGLAGQHGDQGPPGSTGPAGPRGPSGPSGPAGKDGRNGLPGPIGPAGVRGSQGSQGPAGPPGPPGLPGPPGANGGGYEVGYDLEYYRA'},
#     'COL1A2-Hip_amp': {'species': 'Hippopotamus amphibius','offset': 0, 'sequence': 'GPMGIMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRSGERGVVGPQGARGFPGTPGIPGFKGIRGHNGIDGIKGQPGAPGVKGEPGAP---------GARGIPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGEIGPVGSPGASGPAGPRGEVGIPGVSGPVGPPGN----------GAAGIPGVAGAPGIPGPRGIPGPVGAAGATGARGIVGEPGPAGSK------------GPQGIPGPSGEEGKRGSTGEIGPAGPPGPPGIRGSPGSRGIPGADGRAGVMGIPGSRGATGPAGVRG------------------FPGSPGNIGPAGKEGPVGIPGIDGRPGPTGPAGAR----NIGFPGPKGPTGD---NGDKGHAGIAGARGAPGPDGNNGAQGPP--Q---GGKGEQGPAGPPGFQGIPGPAGTAGEAGKPGERGIPGEFGIPGPAGPRGERGPPGESGAAGPTGPIGNRGPSGPAGPDGNKGEPGVVGAPGTAGPSGPSGIPGERGAAGIPGPKGEKGEPGIR-------RDGARGAPGAVGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGER---GTRGDGGPPGATGFPGAAGR------------------------------TGPPGPSGISGPPGPPGPAGKEGIRGPRGDQGPVGRSGETGASGPPGFAGEK-------------TPGPQGIIGAPGFIGIPGSRGERGIPGVAGSVGEPGPIGIAGPPGARGPPGAVGNPGVNGAPGEAGRDGNPGSDGPPGR----GHKGERGYP-------GAGAPGPQGPVGPTGKHGNRGEPGPAGVVGPTGAVGPRGPSGPQGIRGDKGEPGDKGPRGIPGIKGHNGIQGIPGIAGHHGDQGAPGSVGPAGPRGPAGPSGPVGKDGRTGHPGAVGPAGIRGSQGPAGPPGPPGPPGPPGPS'},
#     'COL1A2-Mel_mel': {'species': 'Meles meles','offset': -7, 'sequence': 'QYDPGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGVSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGSAGPQGPPGPSGEEGKRGPNGEAGSTGPSGPPGLRGSPGSRGLPGADGRAGVMGPPGPRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGAPGNIGPAGKEGPMGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKPGEKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGPRGERGPPGESGAAGPSGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPGGLPGERGAAGVPGGKGEKGETGLRGEVGNPGRDGARGAPGAVGAPGPAGATGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGSAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGAAGKEGLRGPRGDQGPVGRTGETGAHGPPGFAGEKGPSGEAGTAGPPGTPGPQGLLGAPGILGLPGSRGERGLPGVSGSVGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNIGPVGAVGAPGPHGPVGPTGKHGNRGEPGPAGSVGPVGAAGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGAPGSVGPAGPRGPAGPSGPAGKDGRTGHPGTVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENIPVKNWYRNSKVKKHTWLGETINDGTQFEYNTEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVILQGSNDVELVAEGNSRFTYTVLADGCSKKTNEWRKTIIEYKTNKPSRLPILDIAPLDIGGADQEFRVDVGPVCFK'},
#     'COL1A2_Alu_mel': {'species': 'Ailuropoda melanoleuca','offset': -7, 'sequence': 'QYDPGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGVSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGSVGPQGPPGPSGEEGKRGPNGEAGSAGPSGPPGLRGSPGSRGLPGADGRAGVMGPPGPRGSTGPAGVRGPNGDSGRPGEPGLMGPRGFPGAPGNVGPAGKEGPMGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGEPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGPRGERGPPGESGAAGPSGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPGGLPGERGAAGVPGGKGEKGETGLRGEVGNPGRDGARGAPGAVGAPGPAGATGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGSAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGAAGKEGLRGPRGDQGPVGRTGETGAHGPPGFAGEKGPSGEPGTAGPPGTAGPQGLLGAPGILGLPGSRGERGLPGVSGSVGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNIGPVGTVGAPGPHGPVGPTGKHGNRGEPGPAGAVGPVGAVGPRGPSGPQGVRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGAPGSVGPAGPRGPAGPSGPAGKDGRTGHPGTVGPAGVRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENIPAKNWYRNSKVKKHIWLGETINGGTQFEYNVEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVMLQGSNDVELVAEGNSRFTYTVLVDGCSKKTNEWGKTIIEYKTNKPSRLPILDIAPLDIGGADQEFRVDVGPVCFK'},
#     'COL1A2-Enh_lut': {'species': 'Enhydra lutris kenyoni','offset': -7, 'sequence': 'QYDPGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGVSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGSAGPQGPPGPSGEEGKRGPNGEAGSTGPSGPPGLRGSPGSRGLPGADGRAGVMGPPGPRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGAPGNIGPAGKEGPMGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKPGEKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGPRGERGPPGESGAAGPSGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPGGLPGERGSAGVPGGKGEKGETGLRGEVGNPGRDGARGAPGAVGAPGPAGATGDRGEAGPAGPSGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGSAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGAAGKEGLRGPRGDQGPVGRTGETGAHGPPGFAGEKGPSGEPGTAGPPGTSGPQGLLGAPGILGLPGSRGERGLPGVSGSVGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNIGSVGAVGAPGPHGPVGPTGKHGNRGEPGPAGSVGPVGAAGPRGPSGPQGVRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGAPGSLGPAGPRGPAGPSGPAGKDGRTGHPGTVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENIPVKNWYRNSKVKKHTWLGETINGGTQFEYNTEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVILQGSNDVELVAEGNSRFTYTVLADGCSKKTNEWRKTIIEYKTNKPSRLPILDIAPLDIGGADQEFRVDVGPVCFK'},
#     'COL1A2-Cro_cro': {'species': 'Crocuta crocuta','offset': -7, 'sequence': '---------------------------------------------------------------------------------------------------------------------------------------------------- -------------------------------------------------GELGPVGNPGPAGPAGPRGEMGLPGVSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGSAGPQGPPGPSGEEGKRGPNGEAGSTGPSGPPGLRGSPGSRGLPGADGRAGVMGPPGPRGATGPAGVRGPNGDAGRPGEPGLMGPRGFPGAPGNVGPAGKEGPMGVPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGEPGKGGEKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGLPGEFGLPGPAGPRGERGPPGESGAAGPSGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGEPGLRGEIGSPGRDGARGAPGAVGAPGPAGATGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGSAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGAAGKEGVRGPRGDQGPVGRTGETGASGPPGFAGEKGPSGEPGTAGPPGTPGPQGLLGAPGILGLPGSRGERGLPGVSGSVGEPGPLGLAGPPGARGGPGAVGAPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNIGPVGAAGAPGPHGPVGPAGKHGN----GPAGVVGPVGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGSPGSVGPAGPRGPAGPSGPVGKDGRTGHPGSVGPAGVRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENILAKNWYRNSKVKKHVWLGETINGGTQFEYNVEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVILQGSNDVELVAEGNSRFTYNVLADGCSKKTNEWRKTIIEYKTNKPSRLPILDIAPSDIGGADQEIRVDVGPVCFK'},
#     'COL1A2-Fel_sil': {'species': 'Felis silvestris catus','offset': -7, 'sequence': 'QYDPGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEMGLPGVSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGSAGPQGPPGPSGEEGKRGPNGEAGSAGPSGPPGLRGSPGSRGLPGADGRAGVMGPPGPRGATGPAGVRGPNGDAGRPGEPGLMGPRGFPGAPGNVGPAGKEGPMGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKNGDKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGPRGERGPPGESGAAGPSGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGEIGNPGRDGARGAPGAVGAPGPAGATGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGSAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGAAGKEGLRGPRGDQGPVGRTGETGASGPPGFAGEKGPSGEPGTAGPPGTPGPQGLLGAPGILGLPGSRGERGLPGVSGSVGEPGPLGISGPPGARGPSGAVGAPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNIGPVGAVGAPGPHGPVGPTGKHGNRGEPGPAGVVGPVGAVGPRGPTGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGAPGSVGPAGPRGPAGPSGPMGKDGRTGHPGSVGPAGVRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENILAKNWYRNSKVKKHVWLGETINGGTQFEYNVEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVILQGSNDVELVAEGNSRFTYTVLVDGCSKKTNEWKKTIIEYKTNKPSRLPILDIAPLDIGGADQEIRVDVGPVCFK'},
#     'COL1A2-Enh_lut': {'species': 'Enhydra lutris kenyoni','offset': -7, 'sequence': 'QYDPGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGVSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGSAGPQGPPGPSGEEGKRGPNGEAGSTGPSGPPGLRGSPGSRGLPGADGRAGVMGPPGPRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGAPGNIGPAGKEGPMGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKPGEKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGPRGERGPPGESGAAGPSGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPGGLPGERGSAGVPGGKGEKGETGLRGEVGNPGRDGARGAPGAVGAPGPAGATGDRGEAGPAGPSGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGSAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGAAGKEGLRGPRGDQGPVGRTGETGAHGPPGFAGEKGPSGEPGTAGPPGTSGPQGLLGAPGILGLPGSRGERGLPGVSGSVGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNIGSVGAVGAPGPHGPVGPTGKHGNRGEPGPAGSVGPVGAAGPRGPSGPQGVRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGAPGSLGPAGPRGPAGPSGPAGKDGRTGHPGTVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENIPVKNWYRNSKVKKHTWLGETINGGTQFEYNTEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVILQGSNDVELVAEGNSRFTYTVLADGCSKKTNEWRKTIIEYKTNKPSRLPILDIAPLDIGGADQEFRVDVGPVCFK'},
#     'COL1A2-Ail_mel': {'species': 'Ailuropoda melanoleuca','offset': -7, 'sequence': 'QYDPGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGVSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGSVGPQGPPGPSGEEGKRGPNGEAGSAGPSGPPGLRGSPGSRGLPGADGRAGVMGPPGPRGSTGPAGVRGPNGDSGRPGEPGLMGPRGFPGAPGNVGPAGKEGPMGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGEPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGPRGERGPPGESGAAGPSGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPGGLPGERGAAGVPGGKGEKGETGLRGEVGNPGRDGARGAPGAVGAPGPAGATGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGSAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGAAGKEGLRGPRGDQGPVGRTGETGAHGPPGFAGEKGPSGEPGTAGPPGTAGPQGLLGAPGILGLPGSRGERGLPGVSGSVGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNIGPVGTVGAPGPHGPVGPTGKHGNRGEPGPAGAVGPVGAVGPRGPSGPQGVRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGAPGSVGPAGPRGPAGPSGPAGKDGRTGHPGTVGPAGVRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENIPAKNWYRNSKVKKHIWLGETINGGTQFEYNVEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVMLQGSNDVELVAEGNSRFTYTVLVDGCSKKTNEWGKTIIEYKTNKPSRLPILDIAPLDIGGADQEFRVDVGPVCFK'},
#     'COL1A2-Del_leu': {'species': 'Delphinapterus leucas','offset': -6, 'sequence': 'QYDGKGVGIGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGTPGVKGEPGAPGENGIPGQIGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPLGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGSRGEVGLPGVSGPVGPPGNPGANGLHGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGSKGEPGAAGPTGPPGPSGEEGKRGTTGEIGSAGPPGPPGLRGNPGSRGLPGADGRAGVMGPHGSRGGTGPAGVRGPSGDSGRPGEPGLMGPRGFPGSPGNVGPAGKEGPMGLPGIDGRPGPIGPAGTRGEPGNIGFPGPKGPTGDPGKNGEKGHAGLAGPRGAPGPDGNNGAQGPPGPQGVSGGKGEQGPAGPPGFQGLPGPAGTAGEAGKAGERGIPGEFGLPGPAGSRGERGPPGESGAAGPAGPIGSRGPSGPAGPDGNKGEPGVVGAPGTAGPSGPNGLPGERGAAGIPGGKGEKGETGLRGDVGSHGRDGARGAPGAVGAPGPAGANGDRGEAGPAGPAGPVGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPTGPTGPVGAVGPAGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGPAGKEGLRGPRGDQGPVGRTGETGASGPPGFVGEKGPSGEPGTAGSPGTPGPQGLLGVPGFLGLPGSRGERGLPGVAGSMGEPGPLGIAGPAGARGPPGAVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQAGHKGDRGYPGNAGPTGTAGAPGPQGPVGPTGKHGNRGEPGPSGPAGLAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGTVGPAGPRGPSGPSGPSGKDGRTGHPGAVGPAGIRGSQGSQGPSGPPGPPGPPGPPGPSGGGYDFGFEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWVDPNQGCTMDAIKVYCDFSTGETCIRAQPENIPVKNWYRSSKAKKHVWVGETINGGTQFEYNVEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVILQGSNDVELVAEGNSRFTYTVLVDGCSKKINEWRKTIIEYKTNKPSRLPILDIAPLDIGGADQEIRLNIGPVCFK'},
#     'COL1A2-Can_lup': {'species': 'Canus Lupu','offset': -6, 'sequence': 'QYDGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGEIGPVGNPGPAGPAGPRGEVGLPGVSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGIVGEPGPAGSKGESGNKGEPGSAGAQGPPGPSGEEGKRGPNGEAGSAGPSGPPGLRGSPGSRGLPGADGPAGVMGPPGPRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGAPGNVGPAGKEGPMGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKNGDKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGPRGERGPPGESGAAGPSGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGASGPGGLPGERGAAGIPGGKGEKGETGLRGEIGNPGRDGARGAPGAMGAPGPAGATGDRGEAGPAGPAGPAGPRGTPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPIGSAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGAAGKEGLRGPRGDQGPVGRTGETGASGPPGFTGEKGPSGEPGTAGPPGTPGPQGLLGAPGILGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGNDGPPGRDGQAGHKGERGYPGNIGPVGAVGAPGPHGPVGPTGKHGNRGEPGPAGSVGPVGAVGPRGPSGPQGIRGDKGEPGEKGPRGLPGLKGHNGLQGLPGLAGQHGDQGAPGSVGPAGPRGPAGPSGPAGKDGRTGQPGTVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYEGDFYRADQPRSPPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENIPAKNWYRNSKVKKHIWLGETINGGTQFEYNVEGVTTKEMATQLAFMRLLANHASQNITYHCKNSIAYMDEETGNLKKAVILQGSNDVELVAEGNSRFTYTVLVDGCSKKTNEWRKTIIEYKTNKPSRLPILDIAPLDIGDADQEFRVDVGPVCFK'},

#     'COL3A1-Equ_cab': {'species': 'Equus caballus','offset': -17, 'sequence': 'QYESYDVKAGVAGGGLAGYPGPVGPPGPPGPPGASGHPGSPGSPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGLPGPPGIKGPAGIPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGLPGENGAPGPMGPRGAPGERGRPGLPGAAGSRGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGSSGAPGQRGEPGPQGNAGAPGPPGPPGASGSPGGKGEMGPAGIPGAPGLIGARGPPGPSGANGAPGQRGPAGEPGKNGAKGEPGARGERGEAGSPGIPGAKGEDGKDGSPGEPGANGLPGAAGERGAPGFRGPAGPNGLPGEKGAPGDRGSPGPAGPRGAAGEPGRDGVPGGPGIRGVPGSPGGPGSDGKPGPPGSQGESGRPGPPGPSGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPPGPAGKNGETGPQGPPGPTGPSGDKGDAGLPGPQGLQGLPGTSGPPGENGKPGEPGPKGEAGSPGIPGGKGDSGAPGERGPPGAAGPPGLRGGAGPPGPEGGKGPAGAPGPPGAAGTPGLQGMPGERGGPGGPGVKGDKGEPGSAGADGAPGKDGPRGPTGPIGPPGPAGQPGDKGEGGAPGLPGIAGPRGGPGERGEVGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGAPGIAGPPGGSGPAGPPGPQGVKGERGSPGGPGAAGFPGARGLPGPPGSNGNPGPPGPSGAPGKDGPAGPPGSSGSPGSPGVSGPKGDAGQPGEKGSPGPQGPPGVPGPSGLIGITGARGLAGPPGMPGARGSPGPQGVKGESGKPGAPGHNGERGPPGPQGLPGLAGTAGEPGRDGNPGSDGLPGRDGAPGGKGDRGENGSPGAPGAPGHPGPPGPVGPAGKNGDRGESGPAGPAGAPGPAGARGAPGPQGPRGDKGETGERGSTGIKGHRGFPGSPGSPGSPGPAGHQGAVGSPGPAGPRGPAGPSGPPGKDGTLGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGAGVSAISGVIGPEKSGGYAPYYG'},
#     'COL3A1-Equ_asi': {'species': 'Equus asinus','offset': -17, 'sequence': 'QYESYDVKAGVAGGGLAGYPGPVGPPGPPGPPGASGHPGSPGSPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGLPGPPGIKGPAGIPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGLPGENGAPGPMGPRGAPGERGRPGLPGAAGSRGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGSSGAPGQRGEPGPQGNAGAPGPPGPPGASGSPGGKGEMGPAGIPGAPGLIGARGPPGPSGANGAPGQRGPAGEPGKNGAKGEPGARGERGEAGSPGIPGAKGEDGKDGSPGEPGANGLPGAAGERGAPGFRGPAGPNGLPGEKGAPGDRGSPGPAGPRGAAGEPGRDGVPGGPGIRGVPGSPGGPGSDGKPGPPGSQGESGRPGPPGPSGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPPGPAGKNGETGPQGPPGPTGPSGDKGDAGLPGPQGLQGLPGTSGPPGENGKPGEPGPKGEAGSPGIPGGKGDSGAPGERGPPGAAGPPGLRGGAGPPGPEGGKGPAGAPGPPGAAGTPGLQGMPGERGGPGGPGVKGDKGEPGSAGADGAPGKDGPRGPTGPIGPPGPAGQPGDKGEGGAPGLPGIAGPRGGPGERGEVGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGAPGIAGPPGGSGPAGPPGPQGVKGERGSPGGPGAAGFPGARGLPGPPGSNGNPGPPGPSGAPGKDGPAGPPGSSGSPGSPGVSGPKGDAGQPGEKGSPGPQGPPGVPGPSGLIGITGARGLAGPPGMPGARGSPGPQGVKGESGKPGAPGHNGERGPPGPQGLPGLAGTAGEPGRDGNPGSDGLPGRDGAPGGKGDRGENGSPGAPGAPGHPGPPGPVGPAGKNGDRGESGPAGPAGAPGPAGARGAPGPQGPRGDKGETGERGSTGIKGHRGFPGSPGSPGSPGPAGHQGAVGSPGPAGPRGPAGPSGPPGKDGTLGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGAGVSAISGVVGPEKSGGYAPYYG'},
#     'COL3A1-Bos_tau': {'species': 'Bos taurus','offset': -9, 'sequence': ' SGVAGGGIAGYPGPAGPPGPPGPPGTSGHPGAPGAPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGFPGPPGMKGPAGMPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGVPGENGAPGPMGPRGAPGERGRPGLPGAAGARGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGSSGAPGQRGEPGPQGHAGAPGPPGPPGSNGSPGGKGEMGPAGIPGAPGLIGARGPPGPPGTNGVPGQRGAAGEPGKNGAKGDPGPRGERGEAGSPGIAGPKGEDGKDGSPGEPGANGLPGAAGERGVPGFRGPAGANGLPGEKGPPGDRGGPGPAGPRGVAGEPGRDGLPGGPGLRGIPGSPGGPGSDGKPGPPGSQGETGRPGPPGSPGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPQGPAGKNGETGPQGPPGPTGPSGDKGDTGPPGPQGLQGLPGTSGPPGENGKPGEPGPKGEAGAPGIPGGKGDSGAPGERGPPGAGGPPGPRGGAGPPGPEGGKGAAGPPGPPGSAGTPGLQGMPGERGGPGGPGPKGDKGEPGSSGVDGAPGKDGPRGPTGPIGPPGPAGQPGDKGESGAPGVPGIAGPRGGPGERGEQGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGPPGAAGPAGGSGPAGPPGPQGVKGERGSPGGPGAAGFPGGRGPPGPPGSNGNPGPPGSSGAPGKDGPPGPPGSNGAPGSPGISGPKGDSGPPGERGAPGPQGPPGAPGPLGIAGLTGARGLAGPPGMPGARGSPGPQGIKGENGKPGPSGQNGERGPPGPQGLPGLAGTAGEPGRDGNPGSDGLPGRDGAPGAKGDRGENGSPGAPGAPGHPGPPGPVGPAGKSGDRGETGPAGPSGAPGPAGSRGPPGPQGPRGDKGETGERGAMGIKGHRGFPGNPGAPGSPGPAGHQGAVGSPGPAGPRGPVGPSGPPGKDGASGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGAGGVAAIAGVGAEK'},
#     'COL3A1-Bos_ind': {'species': 'Bos indicus','offset': -17, 'sequence': 'QYEAYDVKSGXAGGGIAGYPGPAGPPGPPGPPGTSGHPGAPGAPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGFPGPPGMKGPAGMPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGVPGENGAPGPMGPRGAPGERGRPGLPGAAGARGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGSSGAPGQRGEPGPQGHAGAPGPPGPPGSNGSPGGKGEMGPAGIPGAPGLIGARGPPGPPGTNGVPGQRGAAGEPGKNGAKGDPGPRGERGEAGSPGIAGPKGEDGKDGSPGEPGANGLPGAAGERGVPGFRGPAGANGLPGEKGPPGDRGGPGPAGPRGVAGEPGRDGLPGGPGLRGIPGSPGGPGSDGKPGPPGSQGETGRPGPPGSPGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPQGPAGKNGETGPQGPPGPTGPSGDKGDTGPPGPQGLQGLPGTSGPPGENGKPGEPGPKGEAGAPGIPGGKGDSGAPGERGPPGAGGPPGPRGGAGPPGPEGGKGAAGPPGPPGSAGTPGLQGMPGERGGPGGPGPKGDKGEPGSSGVDGAPGKDGPRGPTGPIGPPGPAGQPGDKGESGAPGVPGIAGPRGGPGERGEQGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGPPGAAGPAGGSGPAGPPGPQGVKGERGSPGGPGAAGFPGGRGPPGPPGSNGNPGPPGSSGAPGKDGPPGPPGSNGAPGSPGISGPKGDSGPPGERGAPGPQGPPGAPGPLGIAGLTGARGLAGPPGMPGARGSPGPQGIKGENGKPGPSGQNGERGPPGPQGLPGLAGTAGEPGRDGNPGSDGLPGRDGAPGAKGDRGENGSPGAPGAPGHPGPPGPVGPAGKSGDRGETGPAGPSGAPGPAGSRGPPGPQGPRGDKGETGERGAMGIKGHRGFPGNPGAPGSPGPAGHQGAVGSPGPAGPRGPVGPSGPPGKDGASGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGAGGVAAIAGVGAEKAGGFAPYYG'},
#     'COL3A1-Bos_mut' : {'species': 'Bos mutus','offset': -17, 'sequence': 'QYEAYDVKSGIAGGGIAGYPGPAGPPGPPGPPGTSGHPGAPGAPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGFPGPPGMKGPAGMPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGVPGENGAPGPMGPRGAPGERGRPGLPGAAGARGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGSSGAPGQRGEPGPQGHAGAPGPPGPPGSNGSPGGKGEMGPAGIPGAPGLIGARGPPGPPGTNGVPGQRGAAGEPGKNGAKGDPGPRGERGEAGSPGIAGPKGEDGKDGSPGEPGANGLPGAAGERGVPGFRGPAGANGLPGEKGPPGDRGGPGPAGPRGVAGEPGRDGLPGGPGLRGIPGSPGGPGSDGKPGPPGSQGETGRPGPPGSPGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPQGPAGKNGETGPQGPPGPTGPSGDKGDTGPPGPQGLQGLPGTSGPPGENGKPGEPGPKGEAGAPGIPGGKGDSGAPGERGPPGAGGPPGPRGGAGPPGPEGGKGAAGPPGPPGSAGTPGLQGMPGERGGPGGPGPKGDKGEPGSSGVDGAPGKDGPRGPTGPIGPPGPAGQPGDKGESGAPGVPGIAGPRGGPGERGEQGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEG------------------------------------------------------------------------------------------------------GPPGAAG---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------PPGKDGASGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGAGGVAAIAGVGAEKAGGFAPYYG'},
#     'COL3A1-Bub_bub' : {'species': 'Bubalus bubalis','offset': -17, 'sequence': 'QYEAYDVKSGIAGGGIAGYPGPAGPPGPPGPPGTSGHPGAPGAPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGFPGPPGMKGPAGMPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGIPGENGAPGPMGPRGAPGERGRPGLPGAAGARGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGSSGAPGQRGEPGPQGHAGAPGPPGPPGSNGSPGGKGEMGPAGIPGAPGLIGARGPPGPSGTNGVPGQRGAAGEPGKNGAKGDPGPRGERGEAGSPGIPGPKGEDGKDGSPGEPGANGLPGAAGERGVPGFRGPAGANGLPGEKGPPGERGGPGPAGPRGVAGEPGRDGLPGGPGLRGIPGSPGGPGSDGKPGPPGSQGETGRPGPPGSPGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPQGPAGKNGETGPQGPPGPTGPSGDKGDTGPPGPQGLQGLPGTSGPPGENGKPGEPGPKGESGAPGTPGGKGDSGAPGERGPPGAGGPPGPRGGPGPPGPEGGKGAAGPPGPPGSAGTPGLQGMPGERGGPGGPGPKGDKGEPGSSGVDGAPGKDGPRGPTGPIGPPGPAGQPGDKGESGAPGVPGIAGPRGGPGERGEQGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGPPGAAGPAGGSGPAGPPGPQGVKGERGSPGGPGAAGFPGGRGPPGPPGSNGNPGPPGSSGAPGKDGPPGPPGSNGAPGSPGISGPKGDAGPPGERGAPGTQGPPGAPGPLGIAGVAGARGLAGPPGMPGARGSPGPQGIKGENGKPGPSGQNGERGPPGPQGLPGLAGAAGEPGRDGNPGSDGLPGRDGAPGTKGDRGENGSPGAPGAPGHPGPAGPVGPPGKSGDRGETGPAGPSGAPGPAGSRGPPGPQGPRGDKGETGERGATGIKGHRGFPGNPGAPGSPGPAGHQGAVGSPGPAGPRGPVGPSGPPGKDGTSGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGAGGVAAIAGVGAEKAGGFAPYYG'},
#     'COL3A1-Lox_afr' : {'species': 'Loxodonta africana','offset': -17, 'sequence': 'QYDSYDVKAGGAGLVGGYPGPAGPPGPPGPPGTSGHPGSPGAPGYQGPPGEPGQAGPAGPPGPPGAIGPAGPAGKDGESGRPGRTGERGLTGPPGLKGPAGMPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGLPGENGSPGPMGPRGAPGERGRPGLPGAAGARGNDGARGSDGLPGPPGPPGTSGFPGSPGAKGEVGPAGSPGSNGSPGQRGEPGPQGHAGAPGPPGPPGSNGSPGGKGEMGPAGIPGAPGLLGARGPPGPPGTNGIPGLRGSAGEPGKNGAKGDTGPRGERGEAGSPGIPGPKGEDGKDGSPGEPGANGSPGTAGERGAPGFRGPAGANGLPGEKGPSGERGGPGPAGPRGVPGEPGRDGLPGGPGMRGVPGSPGGPGSDGKPGPPGSQGESGRPGPPGPSGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPQGPAGKNGETGPQGPPGPTGASGDKGESGPPGPPGIQGLPGTGGPPGENGKPGEPGPKGDAGAPGTPGGKGDSGAPGERGPPGQAGPPGVRGGAGPPGPEGGKGAPGPPGPPGTSGSPGLQGMPGERGGPGGPGPKGDKGEPGSSGVDGAPGKDGPRGPVGPIGPPGPPGQNGDKGESGAPGLPGPPGARGGPGERGEHGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGPPGAAGPSGGSGPAGPPGPQGVKGERGSPGGPGAAGFPGARGLPGPPGSSGNPGPPGSSGAPGKDGPPGPPGSNGSPGTPGVSGPKGDSGQPGEKGPPGPQGPPGAPGPLGLAGTTGSRGLAGLPGMPGARGSPGPQGPKGENGKPGPSGQNGERGPPGAQGLPGLAGAAGEPGRDGNPGSDGLPGRDGSPGTKGDRGENGSPGAPGAPGHPGPAGPVGPAGKSGDRGETGPAGPAGAPGPAGARGPAGAPGPRGDKGETGERGATGIKGHRGFPGNPGSPGSPGPAGHQGAIGSPGPAGPRGPVGPSGPPGKDGTSGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGGVGAVGLGAVGEKSGGYSPYYG'},
#     'COL3A1-Sus_scr': {'species': 'Sus scrofa','offset': -17, 'sequence': 'QYESYDVKAGVAGGGIGGYPGPAGPPGPPGPPGVSGHPGAPGSPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGLPGPPGLKGPAGMPGFPGMKGHRGFDGRNGEKGDTGAPGLKGENGLPGENGAPGPMGPRGAPGERGRPGLPGAAGARGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGPSGSPGQRGEPGPQGHAGAAGPPGPPGSNGSPGGKGEMGPAGIPGAPGLMGARGPPGPPGTNGAPGQRGAAGEPGKNGAKGEPGPRGERGEAGSPGIPGPKGEDGKDGSPGEPGANGLPGAAGERGMPGFRGAPGANGLPGEKGPAGERGGPGPAGPRGVAGEPGRDGVPGGPGLRGMPGSPGGPGSDGKPGPPGSQGESGRPGPPGSPGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGLPGPPGKNGETGPQGPPGPTGPGGDKGDTGPPGQQGLQGLPGTSGPPGENGKPGEPGPKGEAGAPGIPGGKGDSGAPGERGPPGAVGPSGPRGGAGPPGPEGGKGPAGPPGPPGAAGTPGLQGMPGERGGSGGPGPKGDKGDPGGSGADGAPGKDGPRGPTGPIGPPGPAGQPGDKGESGAPGLPGIAGPRGGPGERGEHGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGPPGIAGQPGGTGPPGPPGPQGVKGERGSPGGPGAAGFPGGRGLPGPPGSNGNPGPPGSSGPPGKDGPPGPPGSSGAPGSPGVSGPKGDAGQPGEKGSPGPQGPPGAPGPGGISGITGARGLAGPPGMPGARGSPGPQGVKGENGKPGPSGLNGERGPPGPQGLPGLAGAAGEPGRDGNPGSDGLPGRDGAPGSKGDRGENGSPGAPGAPGHPGPPGPVGPAGKNGDRGETGPAGPAGAPGPAGSRGAPGPQGPRGDKGETGERGANGIKGHRGFPGNPGAPGSPGPAGHQGAVGSPGPAGPRGPVGPSGPPGKDGASGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGGGAAAIAGVGGEKAGGFAPYYG'},
#     'COL3A1-Ovi_are': {'species': 'Ovis aries','offset': -17, 'sequence': 'QYEAYDVKSGVAGGVGLAGYPGPAGPPGPPGPPGTSGHPGAPGAPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGLPGPPGMKGPAGMPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGIPGENGAPGPMGPRGAPGERGRPGLPGAAGARGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGSSGAPGQRGEPGPQGHAGAPGPPGPPGSNGSPGGKGEMGPAGIPGAPGLLGARGPPGPPGTNGAPGQRGAAGEPGKNGAKGDPGPRGERGEAGSPGIAGPKGEDGKDGSPGEPGANGLPGAAGERGVPGFRGPAGANGLPGEKGPPGERGGPGPAGPRGVAGEPGRDGLPGGPGLRGIPGSPGGPGSDGKPGPPGSQGETGRPGPPGSPGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPQGPAGKNGETGPQGPPGPTGPSGDKGDTGPPGPQGLQGLPGTSGPPGENGKPGEPGPKGEAGAPGIPGGKGDSGAPGERGPPGAGGPPGPRGGAGPPGPEGGKGAAGPPGPPGSAGTPGLQGMPGERGGPGGPGPKGDKGEPGTSGVDGAPGKDGPRGPTGPIGPPGPAGQPGDKGEGGAPGVPGIAGPRGGPGERGEPGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGPPGAAGPPGGSGPAGPPGPQGVKGERGSPGGPGAAGFPGGRGPPGPPGSNGNPGPPGSSGAPGKDGPPGPPGSNGAPGSPGISGPKGDAGQPGERGAPGPQGPPGAPGPLGIAGLTGARGLAGPPGMPGARGSPGPQGIKGENGKPGPSGQNGERGPPGPQGLPGLAGAAGEPGRDGNPGSDGLPGRDGAPGTKGDRGENGSPGAPGAPGHPGPPGPVGPAGKNGDRGETGPAGPSGAPGPAGSRGAPGPQGPRGDKGETGERGANGIKGHRGFPGNPGPPGSPGPAGHQGAVGSPGPAGPRGPVGPSGPPGKDGTSGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGAGGAAAIAGIGGEKAGGFAPYYG'},
#     'COL5A1-Bos_tau': {'species': 'Bos taurus','offset': -9, 'sequence': ' AQPADLLKVLDFHNLPDGITKTTGFCAARRSSKGPDVAYRVTKDAQLSAPTKQLYPASAFPEDFSILTTVKAKKGSQAFLVSIYNEQGIQQIGLEMGRSPVFLYEDHTGKPGPEDYPLFRGINLSDGKWHRIALSVHKKNVTLILDCKKKTTKFLDRSEHPVIDVNGIVVFGTRILDEEVFEGDIQQLLLVSDHRAAYDYCEHYSPDCDTAVPDKPQSQDPNPDEYYPDGDNEGDTYYYEYPYYEDTDDTGKDPITTKTPVEAARETTEVAEELTQPPTEAPSAPSTTEGPGKEEDPGIGDYDYVPSEDYYTPPPYEDFGYGEGIENPDENPDQLPDQGARAEVPTSTVLTSNDSNPAPPPEEGIDDLEEEFTEETIKNLDESYYDPYYDPTISPSEIGPGMPANQDTIYEGMGGPRGEKGQKGEPAIIEPGMLLEGPPGPEGPAGLPGPPGTMGPTGQVGDPGERGPPGRPGLPGADGLPGPPGTMLMLPFRFGGGGDAGSKGPMVSAQESQAQAILQQARLALRGPAGPMGLTGRPGPMGPPGSGGLKGEPGDMGPQGPRGVQGPPGPAGKPGRRGRAGSDGARGMPGQTGPKGDRGFDGLAGLPGEKGHRGDPGPSGPPGPPGEDGERGDDGEVGPRGLPGEPGPRGLLGPKGPPGPPGPPGVTGMDGQPGPKGNVGPQGEPGPPGQQGNPGAQGLPGPQGAIGPPGEKGPLGKPGLPGMPGADGPPGHPGKEGPPGEKGGQGPPGPQGPIGYPGPRGVKGADGIRGLKGTKGEKGEDGFPGFKGDMGIKGDRGEIGPPGPRGEDGPEGPKGRGGPNGDPGPLGPPGEKGKLGVPGLPGYPGRQGPKGSIGFPGFPGANGEKGGRGTPGKPGPRGQRGPTGPRGERGPRGITGKPGPKGNSGGDGPAGPPGERGPNGPQGPTGFPGPKGPPGPPGKDGLPGHPGQRGETGFQGKTGPPGPPGVVGPQGPTGETGPMGERGHPGPPGPPGEQGLPGLAGKEGTKGDPGPAGLPGKDGPPGLRGFPGDRGLPGPVGALGLKGSEGPPGPPGPAGSPGERGPAGAAGPIGIPGRPGPQGPPGPAGEKGAPGEKGPQGPAGRDGLQGPVGLPGPAGPVGPPGEDGDKGEIGEPGQKGSKGDKGEQGPPGPTGPQGPIGQPGPSGADGEPGPRGQQGLFGQKGDEGPRGFPGPPGPVGLQGLPGPPGEKGETGDVGQMGPPGPPGPRGPSGAPGADGPQGPPGGIGNPGAVGEKGEPGEAGEPGLPGEGGPPGPKGERGEKGESGPSGAAGPPGPKGPPGDDGPKGSPGPVGFPGDPGPPGEPGPAGQDGPPGDKGDDGEPGQTGSPGPTGEPGPSGPPGKRGPPGPAGPEGRQGEKGAKGEAGLEGPPGKTGPIGPQGAPGKPGPDGLRGIPGPVGEQGLPGAPGPDGPPGPMGPPGLPGLKGDSGPKGEKGHPGLIGLIGPPGEQGEKGDRGVPGPQGSSGPKGEQGITGPSGPIGPPGPPGLPGPPGPKGAKGSSGPTGPKGEAGQPGPPGPPGPPGEVIQPLPIQASRTRRNIDASQLMDDAEGEQYMDYADGMEEIFGSLNSLKLEIEQMKRPLGTPQNPARTCKDLQLCHPDFPDGEYWVDPNQGCSRDSFKVYCNFTAGGSTCVFPDKKSEGARITSWPKENPGSWFSEFKRGKLLSYVDAEGNPVGVVQMTFLRLLSASAHQNITYNCYQSVAWQDAATGSYDKAMRFLGSNDEEMSYDNSPYIRALVDGCATKKGYQKTVLEIDTPKVEQVPIVDIMFNDFGEASQKFGFEVGPACFLG'},
#     'COL6A1-Bos_tau': {'species': 'Bos taurus','offset': -9, 'sequence': ' QVCCSFECQPARGPPGPRGDPGYEGERGKPGLPGEKGEAGDPGRPGDLGPVGYQGMKGEKGSRGEKGSRGPKGYKGEKGKRGMDGVDGMKGETGFPGLPGCKGSPGFDGIQGPPGPKGDPGAFGLKGQKGEPGADGEPGRPGSTGPPGDEGEPGEPGPPGEKGEAGDEGNAGPDGAPGERGGPGERGPRGTPGARGPRGDPGEAGPQGDQGREGPVGVPGDPGEAGPIGPKGYRGDEGPPGTEGPKGAPGPAGPPGDPGLMGERGEDGPPGNGTEGFPGFPGYPGSRGPPGINGTKGYPGLKGDEGEAGDPGEDNNDIAARGAKGAKGYRGPEGPQGPPGHVGPPGPDECEILDIIMKMCSCCECKCGPIDILFVLD'},
#     'COL6A1-Bub_bub': {'species': 'Bubalus bubalis','offset': -11, 'sequence': ' VMKKVSCLEIPGPPGPKGYRGQKGAKGNMGEPGEPGQKGRQGDPGIEGPIGFPGPKGVPGFKGEKGEFGADGRKGAPGLAGKNGTDGQKGKLGRIGPPGCKGDTGDRGPDGYVGEAGSPGERGDQGSKGDPGRPGRRGPPGENGAKGSKGYQGNNGAPGSPGLKGAKGGPGPRGPKGEPGRRGDPGTKGGPGSDGPKGEKGDPGPEGPRGLAGEVGNKGAKGDRGLPGPRGPQGTVGEPGKQGSRGDPGDAGPRGDSGQPGPKGDPGRPGFSYPGPRGAPGDKGEPGPPGPEGSRGDFGSKGEPGRKGQKGEPADPGPPGEPGPRGQRGAPGPEGEPGPPGDPGLTECDVMTYVRETCGCCDCEKRCGALDVVFVIDSSESIGYTNFTLEKNFVINVVNRLGAIAKDPKSETGTRVGVVQYS'},
#     'COL6A2-Bos_tau': {'species': 'Bos taurus','offset': -7, 'sequence': 'VSCLEIPGPPGPKGYRGQKGAKGNMGEPGEPGQKGRQGDPGIEGPIGFPGPKGVPGFKGEKGEFGADGRKGAPGLAGKNGTDGQKGKLGRIGPPGCKGDTGDRGPDGYVGEAGSPGERGDQGSKGDPGRPGRRGPPGENGAKGSKGYQGNNGAPGSPGLKGAKGGPGPRGPKGEPGRRGDPGTKGGPGSDGPKGEKGDPGPEGPRGLAGEVGNKGAKGDRGLPGPRGPQGTVGEPGKQGSRGDPGDAGPRGDSGQPGPKGDPGRPGFSYPGPRGAPGDKGEPGPPGPEGGRGDFGSKGEPGRKGQKGEPADPGPPGEPGPRGQRGAPGPEGEPGPPGDPGLTECDVMTYVRETCGCCDCEKRCGALDVVFVIDSSESIGYTNFTLEKNFVINVVNRLGAIAKDPKSETGTRVGVVQYSHEGTFEAIQLDDERIDSLSSFKEAVKNLEWIAGGTWTPSALKFAYNKLIKESRRQKTRVFAVVITDGRHDPRDDDLNLRALCNHEVTVTAIGIGDMFHEKHESENLYSIACDKPQQVRNMTLFSDLVAEKFIDDMEDVLCPDPQIVCPDLPCQTELYVAQCTQRPVDIVFLLDGSERLGEQNFHKVRRFVEEVSRRLTLARRDDDPLNARVALLQFGGPREQQVAFPLTSNLTVIQEALASARYLNSFSHVGTGIVQAINQVVQGARAGARRHAELSFVFLTDGVTGNDSLDEAVHSMRKQNVVPTVVAVGSDVDTDVLSKISLGDPAAVFREKDYDSLAQPGFFDRFIRWIC'},

#     'COL12A1-Bos_tau': {'species': 'Bos taurus','offset': -9, 'sequence': ' MRIRLPPALAALGAALLLSSIEAEVEPPSDLNFKIIDENTVHMSWAIPPDPIVGFKITVDPTTDGPTKEFTLSASTTETLLTELIPEIEYVVTITSYDEVEESVPVIGQLTIQTGGPTKPGEKKPGKTEIQKCSVSAWTDLVFLVDGSWSVGRNNFKYILDFIAALVSAFDIGEEKTRVGVVQYSSDTRTEFNLNQYYQQEELLAAIKKIPYKGGNTMTGEAIDYLIKNTFTESAGARVGFPKVAIIITDGKSQDEVEIPARELRNIGVEVFSLGIKAADAKELKQIASTPSLNHVFNVANFDAIVDIQNEIISQVCSGVDEQLGELVSGEEVVEPPSNLIATEVSSKYVKLSWSPSPSAVTGYKVLLIPMTTGSRQHALSVGPQTTTLSVRDLSADTEYQISVSAMKGLTSSEPVSIMEKTQPMKVQVECSRGVDIKADIVFLVDGSYSIGIANFVKVRAFLEVLVKSFEISPNRVQISLVQYSRDPHTEFTLKKFTKVEDIIEAINTFPYRGGSTNTGKAMTYVREKIFVPSKGSRSNVPKVMILITDGKSSDAFRDPAIKLRNSDVEIFAVGVKDAVRSELEAIASPPAETHVFTVEDFDAFQRISFELTQSICLRIEQELAAIKKKAYVPPKDINFSEVTSNSLKTSWSPAGENVFSYHITYKEATGDDQVTLVEPASSTSVVLSNLKPETLYLVNVTAEYEDGFSIPLAGEETTAEVKGAPRNLKVTDETTDSFKITWTQAPGRVLRYRITYRPVAGGESKEVTTPANQRRRTLENLTPDTKYEVSVIPEYVSGPGSPLTGNAATEEVRGNPRDLRVSDPTTSTMKLSWTRAPGKVKQYLITYTPVAGGDTQEVTVKGDTASTVLRGLNEGTQYALSVTALYASGAGDALFGEGTTLEERGSPGDLITKDITDTSIGAYWTSAPGMVRGYRVSWQSLYDDIETGEKSLPADVIHTVIENLQPETKYRVSIFATYSSGEGEPLSGEAMTELSEDSKTLKVDEETENTMRVTWKPAPGKVVNYRVMYRPRGGGRQMVAKVSPTVTSTVLKRLQPQTTYDITVLPVYKTGEGKLRQGSGTTASRFKSPRNLKTSDPTMSSFRVTWEPAPGEVKGYKVTFHPTGDDRRLGELVVGPYDNTVVLEELRAGTTYKVNVFGMFDGGESSPLVGQEMTTLSDTTVMPILSSGMECLTRAEADIVLLVDGSWSIGRANFRTVRSFISRIVEVFEIGPKRVQIALAQYSGDPRTEWQLNAHKDKKSLLQAVANLPYKGGNTLTGMALNFIRQQSFKTQAGMRPRARKIGVLITDGKSQDDVEAPSKKLKDEGVELFAIGIKNADEVELKMIATDPDDIHAYNVADFDSLSRIVDDLTNNLCNSVKGPGDLEAPSNLVLSERTHRSMRVSWTPPSDSVDRYKVEYYPVSGGKRQEFYVSRLERSTVLKDLKPETEYVVNVYSVVEDEYSEPLKGTEKTLPVPVVSLNIYDIGSTTMHVQWQPVGGATGYTVTYEPINATVPTKPKEMRLEPTVNDVQLSDLFPNTEYAVTVQAVLHDLTSEPVTAREVTLPLPSPRDLKLRDVTHSTMNVAWEPAPGKVRKYIVRYKTPEEEAKEVEVDRSRASTPLKDLFSQTLYTVSVSAVYEEGESPPVTAQETTHTVPAPTNLQITEVTPESFRGTWDHGASDVSLYRITWAPSGSADKMETILNGDENTLVFQNLNPNTLYEVSVTAIYPDESESDDLIGSERTLRLIPLTTQAPKSGPRNLQVYNATSNSLTVKWDPASGRVQKYRITYQPSTGEGNEQTTTIGGRQNSVLLQKLKPDTPYTITVSSLYPDGEGGRMTGRGKTKPLNTVRNLRVYDPSTSSLNVRWDHAEGSPRQYKLFYAPTAGGPEELVPIPGNTNYAILRNLQPDTQYTVTVVPVYSEGDGGRTSDTGRTLVRGLARNVQVYNPTPNSLDVRWDSAPGPVQQYRIVYSPQDGTRPSESIVVPGNTYTVHLEQLIPDTPYSVNIVALYSDAEGNPSPAQGRTLPRSGPRNLRVFGETTNSLSVAWDHADGPVQQYRIIYSPTVGDPIDEYTTVPGRRNNVILQPLQPDTPYKITVIPVYEDGDGTHLTGNGRTVGLLPPQNIHISDEWYTRFRVSWDPSPSPVLGYKIVYKPVGSNEPMEVFVGEVTSYTLHNLNPSTTYDVNVYAQYDSGLSVPLTDQGTTLYLNVTDLKTYQVGWDTFCVRWLAHRAATSYRLKLSPADGTRGQEITVRGSETSHCFTDLSPETDYGVTVFVQTPNLEGPGVSVKERTTLKPTEAPTEPPTPPPPPTIPPAREVCKGAKADIVFLTDASWSIGDDNFNKVVKFIFHTVGGFDEISPAGIQVSFVQYSDEVKSEFKLNTYNDKALALGALQNIRYRGGNTRTGKALTFIKEKVLTWESGMRKNVPKVLVVVTDGRSQDEVKKAALVIQQSGFSVFVVGVADVDYNELANIASKPSERHVFIVDDFESFEKIEDNLITFVCETATSSCPLIYLDGYTSPGFKMLEAYNLTEKNFASVQGVSLESGSFPSYSSYRLQKNAFVNQPTVELHPNGLPPSYTIILLFRLLPETPSDPFAIWQITDRDYKPQVGVIADPSSKTLSFFNKDTRGEVQTVTFDTDEVKALFYGSFHKVHIVVTSKSVKIYIDCYEIVEKDIKEAGNITTDGYEILGKLLKGERKSATFQIQNFDIVCSPVWTSRDRCCDIPSRRDEAKCPALPNACTCTQDSVGPPGPPGPAGGPGAKGPRGERGVSGAVGPPGPRGDTGPPGPQGPPGPQGPNGLSIPGEQGRQGMKGDAGEPGLPGRTGTPGLPGPPGPMGPPGDRGFTGKDGAMGPRGPPGPPGSPGSPGVTGPSGKPGKPGDHGRPGPSGLKGEKGDRGDIASQNMMRAVARQVCEQLISGQMNRFNQMLNQIPNDYHSSRNQPGPPGPPGPPGSAGARGEPGPGGRPGFPGTPGMQGPQGERGLPGEKGERGIGSPGPRGLPGPPGPQGESRTGPPGSTGSRGPPGPPGRPGNSGIRGPPGPPGYCDSSQCASIPYNGQGYPGSG'},
#     'COLunk-Bos_tau': {'species': 'Bos taurus','offset': -9, 'sequence': ' MRKHRHLPLVAILCLFFSGFSFTRGQQQPAQDSADIIFLIDGSNNTGSVNFAVILDFLVNLLERLSIGTQQIRVGVVQYSDEPRTMFSLNSYSTKAQVLDAVKALGFIGGELANVGLALDFVVENHFTRAGGSRAEEGVPQVLVLISAGPSSDEIRDGVVALKQASVFSFGLGAQAASKAELQHIATNDNLVFTVPEFRSLGDVQEQLLPYIVGVAQRHIVLQPPTIVTQVIEVNKRDIVFLVDGSSKLGLTNFNAIRDFIAKVIQRLEIRQDLIQVAVAQYADTVRPEFYFNTYPSKREVINAVRKMKALDGSALYTGSALDFVRNNLFTEAAGYRAAEGVPKLLVLVTGGKSLDAVSQPAQELKRSGILAFAVGNKVADQAELEEIAFDSSLVFTATEFRPAPLQGVLPGLLGPLRTLTGTTEVRVNKRDIIFLLDGSSNVGETNFPYVRDFVMNLVNSLDVGSDHIRVGLVQFSDTPVTEFSLNTYPTKSELLAHLRQMQLQGGSVLNTGAALSYVHANHFTEAGGSRIQDHVPQLLLLLTAGQSEDSYLQAANALARAGILTFCVGTSQADRAELEEIAFNPGLVYLMDDFSSLPALPQQLIQPLTTYVSGGVEEVPLAQPESKRDILFLFDGSANLMGQFTAARDFLYKVIDELDVKPEGTRVAVAQFSDDVKVESRFDQHQNKPEILNLVKRMKLKTGKALNLGYALDYAQRYIFVKSAGSRIEDGVLQFLVLLVAGKSSDRVDTPALNLKQSGVVPFILQAKNADPAELELIVPSPAFILVAESLPKIGDLQPQIVNLLKSVQNGAPAPVSVEKDVVFLIDGSEGVRSGFPLLKEFVQRVVESLDVGPDRVRVAVVQYSDRTRPEFYLNSYMDQQSVVGAIRGLTLLGGPAPNTGAALEFVLRNILVGSAGSRIAEGVPQLLIVLTADRSGDDVRGPSVVLRRGGAVPIGIGIGNADITEMQTLSFVPDFAVVIPTFRQLGTIQQVISERVTQLSREELSRLQASVTPLTTPVVSSKRDVVFLIDGSQSAGPEFQYIRTLIERLVDYLDVGFDTTRVAVIQFSDDPRVEFLLNVHSSKDEVQNAVRRLRPKGGRQVNVGGALEYVARNIFKRPLGSRIEEGVPQFLVLISSGKSDDEVEDSAIELKQFGVAPLTIARNVDQEELVKISLSPEYVFSVNTFRELPSLEQKLLTPLTTLTAGQIQQLLASTRYPPPAVESDAADIVFLIDSSDGVKPDGIAHIRDFVIRIVRRLNVGPNKVRIGVLQFSNDVFPEFQLKTYKSQASVLDAIRRLRFKGGSPLNTGKALEFVARNYFVKSAGSRIEDGVPQHLVLFLGGKSQDDISRYSQVIKSAGIASLGVGDRNIDRTELQTITSDPRLVFTVREFRDLPSIEERMVNSFGSSGVTPAPPGVDTPSPSRPEKKKADIVFLLDGSINFRRDSFQEVLRFVSEIVDTVYEGGDSIQVGLVQYNSDPTDEFFLKDFPTKQQIIDAINKVVYKGGRHANTKVGLEHLRRNHFVPEAGSRLDQRVPQIAFVITGGKSVEDAQEASMALTQRGVKVFAVGVRNIDSEEVGKIASNSATAFRVGNVQELSELSEQVLETLHDAMHETLCPGVTDVSKACNLDVILGFDGSGDQNVFVAQKGLEPKVDTILRRISQMQKISCSGSQLPTVRVSVVALTPSGPVEAFDFAEYQPELFEKFQNMRAQHPYVLTADTLKLYQNKFRQASMDNVKVVIHFTDGVDGDLADLQRASEELRQEGVRALILVGLERVANLEQLMQLEFGRGFTYNRPLRLNLLDLDYELAEQLDNIAEKACCGIPCKCSGQRGDRGPIGSIGPKGIPGEDGYRGYPGDEGGPGERGPPGVNGTQGFQGCPGQRGVKGSRGFPGEKGELGEIGLDGLDGEDGDKGLPGTSGEKGSPGRRGDKGPKGDKGERGDVGIRGDPGNSGQDSQQRGAKGETGDIGPVGLPGRDGVSGGPGEPGKSGGFGRRGPAGAKGNQGGPGQQGPVGEQGTRGAQGPPGPTGPPGLIGEQGIPGPRGGGGTAGAPGERGRTGPLGRKGEPGDPGPKGSIGNRGPRGETGDDGRDGVGSEGRIGKKGERGFPGYPGPKGNPGEPGTDGPPGPKGIRGRRGNSGPPGIAGQKGDPGYPGPSGYKGSRGDSMDQCALVQSIKDKCPCCYGPLECPVFPTELAFALDTSEGVTQDRFSQMREVVLKILDDLTIAESNCPRGARVAVVTYNNEVTTEIRFADSKKKSVLLDKIKNLQVSLTSKQQSLETAMSFVARNTFKRVRSGFLMRKVAVFFSNKPTRATPQLREAVLKLSDAGITPLFLTSQEDRQLINALQINNTAVGHALVLPAGGDLTDFLKNVLTCHVCLDICNIDPSCGFGSWRPSFRDRRAAGSDVDIDMAFILDSSESTTPFQFNEMRKYIEYLVRQLDVSPDPKASQHFARVAVVQHAPYEAVDNASVPPVKVEFSLTDYGSKEKLLAFLGSRMTQLQGTRALGRAIDYTIENIFESAPNPRDLKIVVLMLTGEVQKQQLEEAQRAILQAKCKGYFFVILGIGRKVNVKELYSSASEPNDVFFKLVDKSTELNEEPLMRFGRLLPSFVSSKDAFYLSPDIRKQCDWFQADQPAKNLVQFGYKQINVPNNVTSSPTSKLVTTTKPVITTTKPITVVNLPASKPAATRPVDERPAAGRSVATKMDSARPVATKTEATKPVATKTEATKTEAAKPVVTKAEAAKPTATKTEATKTEAAKLMATKTEATKPVAARPEAAKTATARPAAAAKPAAAKPAPARPPTAARSMATRSEAPRPQAVKLAASRPGAAKPVVKAPREIHVSEVTENSAKLHWERPEPPSPYLYNLTVTSAHDQAVVLKQNLTVTDRVIGGLLPGQTYHVTVTCSLRSQVRAIYQGSFSTKKIQPPPPQTERSASSATINLVVSADRLAGSKADICKLPKDGGTCREFVLKWYYDSVTENCARFWYGGCGGNENRFNSQDECEKVCPPVPIKQPGVIAAMGT'},
#     'FibMod-Bos_mut' : {'species': 'Bos mutus','offset': -1, 'sequence': 'MTPKESRVRRMHSPGRLEEEGCSCLPRCLVWKEKWTQRLTVPTSRQPPWACSSPEPSMEKCAAGTRANSTMNSICDYAQTKKGSKVSLPRGDTHPSGRCSLLKHKMQWASILLLAGLCSLSWAQYEEDSHWWFQFLRNQQSTYDDPYDPYPYEPYEPYPYGGEEGPAYAYGSPPQPEPRDCPQECDCPPNFPTAMYCDNRNLKYLPFVPSRMKYVYFQNNQISSIQEGVFDNATGLLWIALHGNQITSDKVGKKVFSKLRHLERLYLDHNNLTRIPSPLPRSLRELHLDHNQISRVPNNALEGLENLTALYLHHNEIQEVGSSMKGLRSLILLDLSYNHLRKVPDGLPSALEQLYLEHNNVFSVPDSYFRGSPKLLYVRLSHNSLTNNGLASNTFNSSSLLELDLSYNQLQKIPPVSTNLENLYLQGNRINEFSISSFCTVVDVMNFSKLQVLRLDGNEIKRSAMPADAPLCLRLASLIEI'},

#     'K2C1-Hom_sap': {'species': 'Homo sapiens','offset': 0, 'sequence': 'MSRQFSSRSGYRSGGGFSSGSAGIINYQRRTTSSSTRRSGGGGGRFSSCGGGGGSFGAGGGFGSRSLVNLGGSKSISISVARGGGRGSGFGGGYGGGGFGGGGFGGGGFGGGGIGGGGFGGFGSGGGGFGGGGFGGGGYGGGYGPVCPPGGIQEVTINQSLLQPLNVEIDPEIQKVKSREREQIKSLNNQFASFIDKVRFLEQQNQVLQTKWELLQQVDTSTRTHNLEPYFESFINNLRRRVDQLKSDQSRLDSELKNMQDMVEDYRNKYEDEINKRTNAENEFVTIKKDVDGAYMTKVDLQAKLDNLQQEIDFLTALYQAELSQMQTQISETNVILSMDNNRSLDLDSIIAEVKAQYEDIAQKSKAEAESLYQSKYEELQITAGRHGDSVRNSKIEISELNRVIQRLRSEIDNVKKQISNLQQSISDAEQRGENALKDAKNKLNDLEDALQQAKEDLARLLRDYQELMNTKLALDLEIATYRTLLEGEESRMSGECAPNVSVSVSTSHTTISGGGSRGGGGGGYGSGGSSYGSGGGSYGSGGGGGGGRGSYGSGGSSYGSGGGSYGSGGGGGGHGSYGSGSSSGGYRGGSGGGGGGSSGGRGSGGGSSGGSIGGRGSSSGGVKSSGGSSSVKFVSTTYSGVTR'},
#     'K22E-Hom_sap': {'species': 'Homo sapiens','offset': 0, 'sequence': 'MSCQISCKSRGRGGGGGGFRGFSSGSAVVSGGSRRSTSSFSCLSRHGGGGGGFGGGGFGSRSLVGLGGTKSISISVAGGGGGFGAAGGFGGRGGGFGGGSSFGGGSGFSGGGFGGGGFGGGRFGGFGGPGGVGGLGGPGGFGPGGYPGGIHEVSVNQSLLQPLNVKVDPEIQNVKAQEREQIKTLNNKFASFIDKVRFLEQQNQVLQTKWELLQQMNVGTRPINLEPIFQGYIDSLKRYLDGLTAERTSQNSELNNMQDLVEDYKKKYEDEINKRTAAENDFVTLKKDVDNAYMIKVELQSKVDLLNQEIEFLKVLYDAEISQIHQSVTDTNVILSMDNSRNLDLDSIIAEVKAQYEEIAQRSKEEAEALYHSKYEELQVTVGRHGDSLKEIKIEISELNRVIQRLQGEIAHVKKQCKNVQDAIADAEQRGEHALKDARNKLNDLEEALQQAKEDLARLLRDYQELMNVKLALDVEIATYRKLLEGEECRMSGDLSSNVTVSVTSSTISSNVASKAAFGGSGGRGSSSGGGYSSGSSSYGSGGRQSGSRGGSGGGGSISGGGYGSGGGSGGRYGSGGGSKGGSISGGGYGSGGGKHSSGGGSRGGSSSGGGYGSGGGGSSSVKGSSGEAFGSSVTFSFR'},




#     'TRYUP-Sus_sco': {'species': 'Bos taurus','offset': 0, 'sequence': 'IVGGYTCAANSIPYQVSLNSGSHFCGGSLINSQWVVSAAHCYKSRIQVRLGEHNIDVLEGNEQFINAAKIITHPNFNGNTLDNDIMLIKLSSPATLNSRVATVSLPRSCAAAGTECLISGWGNTKSSGSSYPSLLQCLKAPVLSDSSCKSSYPGQITGNMICVGFLEGGKDSCQGDSGGPVVCNGQLQGIVSWGYGCAQKNKPGVYTKVCNYVNWIQQTIAAN'},

#     'TPM1-Cam_dro' : {'species': 'Camelus dromarius','offset': 0, 'sequence':'MDAIKKKMQMLKLDKENAIDRAEQAEADKKQAEDRCKQLEEEQQALQKKLKGTEDEVEKYSESVKDAQEKLEQAEKKATDAEADVASLNRRIQLVEEELDRAQERLATALQKLEEAEKAADESERGMKVIENRAMKDEEKMELQEMQLKEAKHIAEDSDRKYEEVARKLVILEGELERSEERAEVAESRARQLEEELRTMDQALKSLMASEEEEGLGLPNSICCHSHNFALLFSPPPLPHCAISTVSPTVSSHPPPPHTPCSKCGDLEEELKIVTNNLKSLEAQADKYSTKEDKYEEEIKLLEEKLKEAETRAEFAERSVAKLEKTIDDLEDEVYAQKMKYKAISEELDNALNDITSL'},
#     'KS1-Hom_sap' : {'species': 'Homo sapiens','offset': 0, 'sequence':'SRQFSSRSGYRSGGGFSSGSAGIINYQRRTTSSSTRRSGGGGGRFSSCGGGGGSFGAGGGFGSRSLVNLGGSKSISISVARGGGRGSGFGGGYGGGGFGGGGFGGGGFGGGGIGGGGFGGFGSGGGGFGGGGFGGGGYGGGYGPVCPPGGIQEVTINQSLLQPLNVEIDPEIQKVKSREREQIKSLNNQFASFIDKVRFLEQQNQVLQTKWELLQQVDTSTRTHNLEPYFESFINNLRRRVDQLKSDQSRLDSELKNMQDMVEDYRNKYEDEINKRTNAENEFVTIKKDVDGAYMTKVDLQAKLDNLQQEIDFLTALYQAELSQMQTQISETNVILSMDNNRSLDLDSIIAEVKAQYEDIAQKSKAEAESLYQSKYEELQITAGRHGDSVRNSKIEISELNRVIQRLRSEIDNVKKQISNLQQSISDAEQRGENALKDAKNKLNDLEDALQQAKEDLARLLRDYQELMNTKLALDLEIATYRTLLEGEESRMSGECAPNVSVSVSTSHTTISGGGSRGGGGGGYGSGGSSYGSGGGSYGSGGGGGGGRGSYGSGGSSYGSGGGSYGSGGGGGGHGSYGSGSSSGGYRGGSGGGGGGSSGGRGSGGGSSGGSIGGRGSSSGGVKSSGGSSSVKFVSTTYSGVTR'},
#     'KS9-Hom_sap' : {'species': 'Homo sapiens','offset': 0, 'sequence':'MSCRQFSSSYLSRSGGGGGGGLGSGGSIRSSYSRFSSSGGGGGGGRFSSSSGYGGGSSRVCGRGGGGSFGYSYGGGSGGGFSASSLGGGFGGGSRGFGGASGGGYSSSGGFGGGFGGGSGGGFGGGYGSGFGGFGGFGGGAGGGDGGILTANEKSTMQELNSRLASYLDKVQALEEANNDLENKIQDWYDKKGPAAIQKNYSPYYNTIDDLKDQIVDLTVGNNKTLLDIDNTRMTLDDFRIKFEMEQNLRQGVDADINGLRQVLDNLTMEKSDLEMQYETLQEELMALKKNHKEEMSQLTGQNSGDVNVEINVAPGKDLTKTLNDMRQEYEQLIAKNRKDIENQYETQITQIEHEVSSSGQEVQSSAKEVTQLRHGVQELEIELQSQLSKKAALEKSLEDTKNRYCGQLQMIQEQISNLEAQITDVRQEIECQNQEYSLLLSIKMRLEKEIETYHNLLEGGQEDFESSGAGKIGLGGRGGSGGSYGRGSRGGSGGSYGGGGSGGGYGGGSGSRGGSGGSYGGGSGSGGGSGGGYGGGSGGGHSGGSGGGHSGGSGGNYGGGSGSGGGSGGGYGGGSGSRGGSGGSHGGGSGFGGESGGSYGGGEEASGSGGGYGGGSGKSSHS'},
#     'KS2-Hom_sap' : {'species': 'Homo sapiens','offset': 0, 'sequence':'SRQFSSRSGYRSGGGFSSGSAGIINYQRRTTSSSTRRSGGGGGRFSSCGGGGGSFGAGGGFGSRSLVNLGGSKSISISVARGGGRGSGFGGGYGGGGFGGGGFGGGGFGGGGIGGGGFGGFGSGGGGFGGGGFGGGGYGGGYGPVCPPGGIQEVTINQSLLQPLNVEIDPEIQKVKSREREQIKSLNNQFASFIDKVRFLEQQNQVLQTKWELLQQVDTSTRTHNLEPYFESFINNLRRRVDQLKSDQSRLDSELKNMQDMVEDYRNKYEDEINKRTNAENEFVTIKKDVDGAYMTKVDLQAKLDNLQQEIDFLTALYQAELSQMQTQISETNVILSMDNNRSLDLDSIIAEVKAQYEDIAQKSKAEAESLYQSKYEELQITAGRHGDSVRNSKIEISELNRVIQRLRSEIDNVKKQISNLQQSISDAEQRGENALKDAKNKLNDLEDALQQAKEDLARLLRDYQELMNTKLALDLEIATYRTLLEGEESRMSGECAPNVSVSVSTSHTTISGGGSRGGGGGGYGSGGSSYGSGGGSYGSGGGGGGGRGSYGSGGSSYGSGGGSYGSGGGGGGHGSYGSGSSSGGYRGGSGGGGGGSSGGRGSGGGSSGGSIGGRGSSSGGVKSSGGSSSVKFVSTTYSGVTRR'},
#     'KS10-Hom_sap' : {'species': 'Homo sapiens','offset': 0, 'sequence':'MSVRYSSSKHYSSSRSGGGGGGGGCGGGGGVSSLRISSSKGSLGGGFSSGGFSGGSFSRGSSGGGCFGGSSGGYGGLGGFGGGSFRGSYGSSSFGGSYGGIFGGGSFGGGSFGGGSFGGGGFGGGGFGGGFGGGFGGDGGLLSGNEKVTMQNLNDRLASYLDKVRALEESNYELEGKIKEWYEKHGNSHQGEPRDYSKYYKTIDDLKNQILNLTTDNANILLQIDNARLAADDFRLKYENEVALRQSVEADINGLRRVLDELTLTKADLEMQIESLTEELAYLKKNHEEEMKDLRNVSTGDVNVEMNAAPGVDLTQLLNNMRSQYEQLAEQNRKDAEAWFNEKSKELTTEIDNNIEQISSYKSEITELRRNVQALEIELQSQLALKQSLEASLAETEGRYCVQLSQIQAQISALEEQLQQIRAETECQNTEYQQLLDIKIRLENEIQTYRSLLEGEGSSGGGGRGGGSFGGGYGGGSSGGGSSGGGHGGGHGGSSGGGYGGGSSGGGSSGGGYGGGSSSGGHGGSSSGGYGGGSSGGGGGGYGGGSSGGGSSSGGGYGGGSSSGGHKSSSSGSVGESSSKGPRY'},
#     'KS1-Hom_sap' : {'species': 'Homo sapiens','offset': 0, 'sequence':'SRQFSSRSGYRSGGGFSSGSAGIINYQRRTTSSSTRRSGGGGGRFSSCGGGGGSFGAGGGFGSRSLVNLGGSKSISISVARGGGRGSGFGGGYGGGGFGGGGFGGGGFGGGGIGGGGFGGFGSGGGGFGGGGFGGGGYGGGYGPVCPPGGIQEVTINQSLLQPLNVEIDPEIQKVKSREREQIKSLNNQFASFIDKVRFLEQQNQVLQTKWELLQQVDTSTRTHNLEPYFESFINNLRRRVDQLKSDQSRLDSELKNMQDMVEDYRNKYEDEINKRTNAENEFVTIKKDVDGAYMTKVDLQAKLDNLQQEIDFLTALYQAELSQMQTQISETNVILSMDNNRSLDLDSIIAEVKAQYEDIAQKSKAEAESLYQSKYEELQITAGRHGDSVRNSKIEISELNRVIQRLRSEIDNVKKQISNLQQSISDAEQRGENALKDAKNKLNDLEDALQQAKEDLARLLRDYQELMNTKLALDLEIATYRTLLEGEESRMSGECAPNVSVSVSTSHTTISGGGSRGGGGGGYGSGGSSYGSGGGSYGSGGGGGGGRGSYGSGGSSYGSGGGSYGSGGGGGGHGSYGSGSSSGGYRGGSGGGGGGSSGGRGSGGGSSGGSIGGRGSSSGGVKSSGGSSSVKFVSTTYSGVTR'},
#     'KS1-Ovi_are' : {'species': 'Ovis aries','offset': 0, 'sequence':'SLHVNMSRHFSSRSGYRCGGGGGGFSSGSAGVVSYQRRSTSSSVRRSGGGGGRFSGGVCGGGAGGGFGVAGGGGRGGFGGGYGGGYGGGGFGGGGFGGGGFGGGGFGGGGIGGGFGGGGFGGGGFGGGGVWGGGGGIQESQEREQIKSLNNRFASFIDKVRFLEQQNQVLQTKWELLQQINTSTRTYSLEPFFEAYISNLRRNVDQLKSDRSRLDSELKNMQDLVEDYRSKYEDEINKRTNAENEFVGIKKDVDAAYITKVDLQAKFDNLLQEIDFFKTLFQAELSQMQTHISDTNVILSMDNNRNLDLNSIIDEVKNQYEEIAQRSKAEAEALYQTKYEELQITAGQHGDSLKNTKMEISELNRVIQRLRSEIDSVKKQISSLQQAISDAEQRGENAIKDAQNKLNELEDALQKAKEDMARLLRDYQELMNTKLALDVEIATYRTLLEGEESRMSGECVPNVSVSVSTSHTSISGGGGRGGGGFSSGGGGYSSGGGGGGGYGSGGGGGYGSGGGSSYGSRGGRADPPRWGRAGLYGRTSSSCLSRHGGGSGVFGGGGFGGGGFSSRSLINLGGSRSTSISVSGGGGSFGSYGGFGGRGSGFGGGLGVGGGSFGGGRFGGGGGFGGFGGPGGFGGPAGFGGYGSGGRGSSSGGGFSSGSSSYSSGGRRSGSRSGGGGSYGSGGGSRGGSSFGGGYSSGGKHSSGGGARGGSSSGGGTYGSERGASGSSEGGSSSEKSVVEGICTLESSFLKESPCH'},
#     'KS1c9-Hom_sap' : {'species': 'Homo sapiens','offset': 0, 'sequence':'MSCRQFSSSYLSRSGGGGGGGLGSGGSIRSSYSRFSSSGGGGGGGRFSSSSGYGGGSSRVCGRGGGGSFGYSYGGGSGGGFSASSLGGGFGGGSRGFGGASGGGYSSSGGFGGGFGGGSGGGFGGGYGSGFGGFGGFGGGAGGGDGGILTANEKSTMQELNSRLASYLDKVQALEEANNDLENKIQDWYDKKGPAAIQKNYSPYYNTIDDLKDQIVDLTVGNNKTLLDIDNTRMTLDDFRIKFEMEQNLRQGVDADINGLRQVLDNLTMEKSDLEMQYETLQEELMALKKNHKEEMSQLTGQNSGDVNVEINVAPGKDLTKTLNDMRQEYEQLIAKNRKDIENQYETQITQIEHEVSSSGQEVQSSAKEVTQLRHGVQELEIELQSQLSKKAALEKSLEDTKNRYCGQLQMIQEQISNLEAQITDVRQEIECQNQEYSLLLSIKMRLEKEIETYHNLLEGGQEDFESSGAGKIGLGGRGGSGGSYGRGSRGGSGGSYGGGGSGGGYGGGSGSRGGSGGSYGGGSGSGGGSGGGYGGGSGGGHSGGSGGGHSGGSGGNYGGGSGSGGGSGGGYGGGSGSRGGSGGSHGGGSGFGGESGGSYGGGEEASGSGGGYGGGSGKSSHS'},



#     'ALB-Bos_tau' : {'species': 'Bos taurus','offset': 0, 'sequence':'MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGLVLIAFSQYLQQCPFDEHVKLVNELTEFAKTCVADESHAGCEKSLHTLFGDELCKVASLRETYGDMADCCEKQEPERNECFLSHKDDSPDLPKLKPDPNTLCDEFKADEKKFWGKYLYEIARRHPYFYAPELLYYANKYNGVFQECCQAEDKGACLLPKIETMREKVLASSARQRLRCASIQKFGERALKAWSVARLSQKFPKAEFVEVTKLVTDLTKVHKECCHGDLLECADDRADLAKYICDNQDTISSKLKECCDKPLLEKSHCIAEVEKDAIPENLPPLTADFAEDKDVCKNYQEAKDAFLGSFLYEYSRRHPEYAVSVLLRLAKEYEATLEECCAKDDPHACYSTVFDKLKHLVDEPQNLIKQNCDQFEKLGEYGFQNALIVRYTRKVPQVSTPTLVEVSRSLGKVGTRCCTKPESERMPCTEDYLSLILNRLCVLHEKTPVSEKVTKCCTESLVNRRPCFSALTPDETYVPKAFDEKLFTFHADICTLPDTEKQIKKQTALVELLKHKPKATEEQLKTVMENFVAFVDKCCAADDKEACFAVEGPKLVVSTQTALA'},
#     'ALB-Equ_cab' : {'species': 'Equus caballus','offset': 0, 'sequence':'MKWVTFVSLLFLFSSAYSRGVLRRDTHKSEIAHRFNDLGEKHFKGLVLVAFSQYLQQCPFEDHVKLVNEVTEFAKKCAADESAENCDKSLHTLFGDKLCTVATLRATYGELADCCEKQEPERNECFLTHKDDHPNLPKLKPEPDAQCAAFQEDPDKFLGKYLYEVARRHPYFYGPELLFHAEEYKADFTECCPADDKLACLIPKLDALKERILLSSAKERLKCSSFQNFGERAVKAWSVARLSQKFPKADFAEVSKIVTDLTKVHKECCHGDLLECADDRADLAKYICEHQDSISGKLKACCDKPLLQKSHCIAEVKEDDLPSDLPALAADFAEDKEICKHYKDAKDVFLGTFLYEYSRRHPDYSVSLLLRIAKTYEATLEKCCAEADPPACYRTVFDQFTPLVEEPKSLVKKNCDLFEEVGEYDFQNALIVRYTKKAPQVSTPTLVEIGRTLGKVGSRCCKLPESERLPCSENHLALALNRLCVLHEKTPVSEKITKCCTDSLAERRPCFSALELDEGYVPKEFKAETFTFHADICTLPEDEKQIKKQSALAELVKHKPKATKEQLKTVLGNFSAFVAKCCGREDKEACFAEEGPKLVASSQLALA'},
#     'ALB-Equ_asn' : {'species': 'Equus asinus','offset': 0, 'sequence':'MKWVTFVSLLFLFSSAYFRGVLRRDTHKSEIAHRFNDLGEKHFKGLVLVAFSQYLQQCPFEDHVKLVNEVTEFAKKCAADESAENCDKSLHTLFGDKLCTVATLRATYGELADCCEKQEPERNECFLTHKDDHPNLPKLKPEPDAQCAAFQEDPDKFLGKYLYEVARRHPYFYGPELLFHAEEYKADFTECCPADDKAGCLIPKLDALKERILLSSAKERLKCSSFQKFGERAFKAWSVARLSQKFPKADFAEVSKIVTDLTKVHKECCHGDLLECADDRADLTKYICEHQDSISGKLKACCDKPLLQKSHCIAEVKEDDLPSDLPALAADFAEDKEICKHYKDAKDVFLGTFLYEYSRRHPDYSVSLLLRIAKTYEATLEKCCAEADPPACYATVFDQFTPLVEEPKSLVKKNCDLFEEVGEYDFQNALIVRYTKKAPQVSTPTLVEIGRTLGKVGSRCCKLPESERLPCSENHLALALNRLCVLHEKTPVSEKITKCCTDSLAERRPCFSALELDEGYIPKEFKAETFTFHADICTLPEDEKQIKKQSALAELVKHKPKATKEQLKTVLGNFSAFVAKCCGAEDKEACFAEEGPKLVASSQLALA'},
#     'ALB-Sus_scr' : {'species': 'Sus scrofa','offset': 0, 'sequence':'MKWVTFISLLFLFSSAYSRGVFRRDTYKSEIAHRFKDLGEQYFKGLVLIAFSQHLQQCPYEEHVKLVREVTEFAKTCVADESAENCDKSIHTLFGDKLCAIPSLREHYGDLADCCEKEEPERNECFLQHKNDNPDIPKLKPDPVALCADFQEDEQKFWGKYLYEIARRHPYFYAPELLYYAIIYKDVFSECCQAADKAACLLPKIEHLREKVLTSAAKQRLKCASIQKFGERAFKAWSLARLSQRFPKADFTEISKIVTDLAKVHKECCHGDLLECADDRADLAKYICENQDTISTKLKECCDKPLLEKSHCIAEAKRDELPADLNPLEHDFVEDKEVCKNYKEAKHVFLGTFLYEYSRRHPDYSVSLLLRIAKIYEATLEDCCAKEDPPACYATVFDKFQPLVDEPKNLIKQNCELFEKLGEYGFQNALIVRYTKKVPQVSTPTLVEVARKLGLVGSRCCKRPEEERLSCAEDYLSLVLNRLCVLHEKTPVSEKVTKCCTESLVNRRPCFSALTPDETYKPKEFVEGTFTFHADLCTLPEDEKQIKKQTALVELLKHKPHATEEQLRTVLGNFAAFVQKCCAAPDHEACFAVEGPKFVIEIRGILA'},

#     'COL1A1-Gal_gal': {'species': 'Gallus gallus','offset': -7, 'sequence': 'SAGVAVPGPMGPAGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPAGPPGKNGDDGEAGKPGRPGQRGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGQPGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGPSGPAGARGNDGAPGAAGPPGPTGPAGPPGFPGAAGAKGETGPQGARGSEGPQGARGEPGPPGPAGAAGPAGNPGADGQPGAKGATGAPGIAGAPGFPGARGPSGPQGPSGAPGPKGNSGEPGAPGNKGDTGAKGEPGPAGVQGPPGPAGEEGKRGARGEPGPAGLPGPAGERGAPGSRGFPGADGIAGPKGPPGERGSPGAVGPKGSPGEAGRPGEPGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKPGERGAPGPPGAVGAAGKDGEAGAQGPPGPTGPAGERGEQGPAGAPGFQGLPGPAGPPGEAGKPGEQGVPGDAGAPGPAGARGERGFPGERGVQGPPGPQGPRGANGAPGNDGAKGDAGAPGAPGNQGPPGLQGMPGERGAAGLPGAKGDRGDPGPKGADGAPGKDGLRGLTGPIGPPGPAGAPGDKGEAGPPGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGETGDAGAKGDAGPPGPAGPTGAPGPAGAVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNIGLPGPPGPSGKEGGKGPRGETGPAGRPGEPGPAGPPGPPGEKGSPGADGPIGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGSPGERGPPGPMGPPGLAGPPGEAGREGAPGAEGAPGRDGAAGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKNGDRGETGPAGPAGPPGPAGARGPAGPQGPRGDKGETGEQGDRGMKGHRGFSGLQGPPGPPGAPGEQGPSGASGPAGPRGPPGSAGAAGKDGLNGLPGPIGPPGPRGRTGEVGPVGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEK'},
#     'COL1A1-Clu_har': {'species': 'Clupea harengus','offset': 0, 'sequence': 'GPMGPMGSRGSPGPPGNSGPQGFTGPPGEPGEAGAPGSLGPRGAAGPPGKNGEDGEPGKPGRGGERGSNGPQGARGFPGTPGLPGIKGHRGFSGLDGAKGDSGPAGPKGEGGASGENGTPGAMGPRGLPGERGRSGPNGAAGARGNDGSAGAAGPPGPTGPAGPPGFPGGPGSKGELGAQGARGGEGPAGARGEPGNPGPAGAAGPAGNNGADGAPGNKGAPGAAGVAGAPGFPGPRGPPGAAGATGAPGSKGNTGEAGAPGSKGESGAKGEAGVAGVQGPPGSAGEEGKRGARGEPGGVGTRGAPGERGAPGSRGFPGADGSAGPKGATGERGGAGVVGPKGSSGEPGRNGEPGMPGSKGMTGSPGSPGPDGKLGPSGASGQDGRPGAPGPVGARGQPGVMGFPGPKGAAGEGGKPGERGVAGPLGPGGAAGKDGDVGAPGAPGPSGPSGERGEQGPAGPPGFQGLPGPQGSTGEPGKAGEQGLSGEAGAVGPSGSRGDRGFPGERGGPGAAGPAGARGSPGSAGNDGAKGDVGAPGASGGQGAPGLQGMPGERGAAGLPGLKGERGEQGAKGTDGVTGKDGVRGMTGPIGPPGPAGAPGDKGEGGALGAAGPTGARGPPGERGESGPPGPAGFAGPPGSDGQAGGKGESGDTGSKGDSGSPGPAGATGAPGPQGPVGATGSKGARGAAGPPGATGFPGAVGRVGPPGPSGNSGPAGPPGPGGKEGPKGNRGDTGPAGRTGELGAAGPPGAPGEKGAPGGEGPSGASGTPGPSGIAGARGIVGLPGQRGERGFPGLSGQLGEPGKQGPAGPSGERGPNGPMGPAGLAGPPGEAGREGAPGQEGAVGRDGAAGQKGERGESGVSGAPGAPGPSGAPGPVGPAGKSGDRGESGPSGPAGSAGPAGPRGPAGPAGARGDKGEAGEAGERGMKGHRGFTGMQGPPGSPGSSGEAGPAGSSGPAGPRGPGGSAGSAGKDGMSGLPGPIGPPGPRGRNGEIGPAGPPGPPGPPGPPGAPGGGFDLGFIAQPQEKAPDPFR'},
#     'COL1A1-Fal_tin': {'species': 'Falco tinnunculus','offset': 0, 'sequence': 'GPMGPAGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPAGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGEPGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGPSGPAGARGNDGAPGAAGPPGPTGPAGPPGFPGAAGAKGETGPQGARGSEGPQGARGEPGPPGPAGAAGPAGNPGADGQPGAKGATGAPGIAGAPGFPGARGPSGPQGPSGAPGPKGNSGEPGAPGNKGDTGAKGEPGPAGVQGPPGPAGEEGKRGARGEPGPAGLPGPAGERGAPGSRGFPGADGIAGPKGPPGERGSPGPAGPKGSPGEAGRPGEPGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKPGERGAPGPPGAVGAAGKDGEAGAQGPPGPTGPAGERGEQGPAGAPGFQGLPGPAGPPGEAGKPGEQGVPGDAGAPGPAGARGERGFPGERGVQGPPGPQGPRGANGAPGNDGAKGDAGAPGAPGNQGPPGLQGMPGERGAAGLPGAKGDRGDPGPKGADGAPGKDGLRGLTGPIGPPGPAGAPGDKGEAGPPGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGETGDAGAKGDAGPPGPAGPTGAPGPAGAVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNIGLPGPPGPSGKEGGKGPRGETGPAGRPGEPGPAGPPGPPGEKGSPGADGPIGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGSPGERGPPGPMGPPGLAGPPGEAGREGAPGAEGAPGRDGAAGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKNGDRGETGPQGPAGPPGPAGARGPAGPQGPRGDKGETGEQGDRGMKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAAGKDGLNGLPGPIGPPGPRGRTGDVGPVGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEKAHDGGR'},
#     'COL1A1-Apr_xxx': {'species': 'Apr xxx','offset': -0, 'sequence': 'GPMGPAGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPAGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGEPGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGPSGPAGARGNDGSPGAAGPPGPTGPAGPPGFPGAAGAKGETGPQGGRGSEGPQGARGEPGPPGPAGAAGPAGNPGADGQPGAKGATGAPGIAGAPGFPGARGPSGPQGPSGAPGPKGNSGEPGAPGNKGDTGAKGEPGPAGVQGPPGPAGEEGKRGARGEPGPAGLPGPAGERGAPGSRGFPGADGIAGPKGPPGERGSPGPAGPKGSPGESGRPGEPGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKPGERGAPGPPGAVGAAGKDGEAGAQGPPGPTGPAGERGEQGPAGAPGFQGLPGPAGAPGEAGKPGEQGVPGDAGAPGPAGARGERGFPGERGVQGPPGPQGPRGANGAPGNDGAKGDAGAPGAPGNQGPPGLQGMPGERGAAGLPGAKGDRGDPGPKGADGIPGKDGLRGLTGPIGPPGPAGAPGDKGEAGPPGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGETGDAGAKGDAGPPGPAGPTGAPGPAGAVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNIGLPGPPGPSGKEGGKGPRGETGPAGRPGEPGPAGPPGPPGEKGSPGSDGPIGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGSPGERGPPGPMGPPGLAGPPGEAGREGAPGAEGAPGRDGSAGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKNGDRGETGPAGPAGPPGPAGARGPSGPQGPRGDKGETGEQGDRGMKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAAGKDGLNGLPGPIGPPGPRGRTGDVGPVGPPGPPGPPGPPGPPSGGFDFSFLPQ'},
#     'COL1A1-Caa_xxx': {'species': 'Caa xxx','offset': -0, 'sequence': 'GPMGPAGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPAGPPGKNGEDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGEPGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGPSGPAGARGNDGAPGAAGPPGPTGPAGPPGFPGAGGAKGETGPPGARGSEGPQGARGEPGPPGPAGAAGPAGNPGADGQPGAKGATGAPGIAGAPGFPGARGPSGPQGPSGAPGPKGNAGEPGAPGNKGDTGAKGEPGPAGVQGPPGPAGEEGKRGARGEPGPAGLPGPAGERGAPGSRGFPGADGIAGPKGPPGERGSPGPVGPKGSPGEAGRPGEPGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQGGVMGFPGPKGAAGEPGKPGERGAPGPPGAVGAAGKDGEVGAQGPPGPTGPAGERGEQGPAGAPGFQGLPGPAGPPGEAGKPGEQGVPGDAGAPGPAGARGERGFPGERGVQGPPGPQGPRGANGAPGNDGAKGDAGAPGAPGNQGPPGLQGMPGERGAAGLPGAKGDRGDPGPKGADGAPGKDGLRGLTGPIGPPGPAGAPGDKGEAGPPGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGETGETGAKGDAGPPGPAGPTGAPGPAGAVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNIGLPGPPGPSGKEGGKGPRGETGPAGRPGEPGPAGPPGPPGEKGSPGADGPIGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGAPGERGPPGPMGPPGLAGPPGEAGREGSPGAEGAPGRDGAAGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPQGPAGPPGPAGARGPAGPQGPRGDKGETGEQGDRGMKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAAGKDGLNGLPGPIGPPGPRGRTGEAGPVGPPGPPGPPGPPGPPSGGFDFSFMPQ'},
#     'COL1A1-Nop_xxx': {'species': 'Nop xxx','offset': -0, 'sequence': 'GPMGPAGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPAGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGEPGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGPSGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAAGAKGETGPQGARGSEGPQGARGEPGPPGPAGAAGPAGNPGADGQPGAKGATGAPGIAGAPGFPGARGPSGPQGPSGAPGPKGNSGEPGAPGNKGDTGAKGEPGPAGVQGPPGPAGEEGKRGARGEPGPAGLPGPAGERGAPGSRGFPGADGIAGPKGPPGERGSPGAVGPKGSPGEAGRPGEPGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKPGERGAPGPPGAVGAAGKDGEAGAQGPPGPTGPAGERGEQGPAGAPGFQGLPGPAGPPGEAGKPGEQGVPGDAGAPGPAGARGERGFPGERGVQGPPGPQGPRGANGAPGNDGAKGDAGAPGAPGNQGPPGLQGMPGERGAAGLPGAKGDRGDPGPKGADGAPGKDGLRGLTGPIGPPGPAGAPGDKGEAGPPGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGETGDAGAKGDAGPPGPAGPTGAPGPAGAVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNIGLPGPPGPSGKEGGKGPRGETGPAGRPGEPGPAGPPGPPGEKGSPGADGPIGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGSPGERGPPGPMGPPGLAGPPGEAGREGAPGAEGAPGRDGAAGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKNGDRGETGPAGPAGPPGPAGARGPAGPQGPRGDKGETGEQGDRGMKGHRGFSGLQGPPGPPGQPGEQGPSGASGPAGPRGPPGSAGAAGKDGLNGLPGPIGPPGPRGRTGDVGPVGPPGPPGPPGPPGPPSGGFDFSFLPQ'},
#     'COL1A1-Tha_sir': {'species': 'Thamnophis sirtalis','offset': -0, 'sequence': 'GPMGPAGPRGLPGPPGAPGPQGFQGPPGEPGEPGSSGPMGPRGPAGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQVGPRGLPGERGRPGPAGSAGARGNDGAAGAAGPTGPTGPAGPPGFPGPAGPKGETGPQGARGGEGAQGARGEPGAAGPAGAAGPAGNPGSDGQPGAKGAPGAPGISGAPGFPGARGAAGPQGPTGAPGPKGNSGEPGAPGNKGDAGAKGEAGPAGVQGPSGPPGEEGKRGSRGEPGPSGLPGPAGERGAPGSRGFPGADGIAGPKGPPGERGAPGPAGPKGSTGEAGRTGEPGLPGAKGLTGSPGSPGADGKTGPAGPAGQDGRPGPAGPPGARGQAGVMGFPGPKGSAGEPGKPGERGAPGATGANGAPGKDGDAGAQGPPGPAGPAGERGEQGPSGAPGFQGLPGPAGAPGESGKPGEQGVPGDVGAPGPSGARGERGFPGERGAQGPAGPSGPRGANGAPGNDGAKGDAGAPGAPGGQGPPGLQGMPGERGAAGLPGAKGDRGDPGAKGTDGSPGKDGPRGLTGPIGPPGPAGSPGDKGESGPSGPAGPTGARGAPGDRGEPGPAGPAGFAGPPGTDGQPGAKGEPGDAGAKGDAGPPGPAGATGPAGPAGPVGAPGPKGARGNAGPPGATGFPGAAGRVGPPGPSGNIGLPGPPGPSGKEGSKGPRGETGPAGRPGEVGPAGPPGPSGEKGSPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPTGEPGKQGPSGASGERGPPGPSGPPGLAGPPGEAGREGSPGAEGAPGRDGAAGPKGDRGESGPAGAPGAPGAPGAPGPVGPAGKNGERGESGPAGPAGPAGPSGARGPAGAQGPRGDKGETGEQGDRGMKGHRGFSGLQGPPGPPGSPGEQGPSGSSGPAGPRGPPGSAGSSGKDGLNGLPGPIGPPGPRGRTGDVGPAGPAGPPGPPGPPGPPSGGFDFSFLPQPPQEKAHDGGR'},
#     'COL1A2-Chk_COL1A2?': {'species': 'Gallus gallus','offset': -4, 'sequence': 'AADFGPGPMGLMGPRGPPGASGPPGPPGFQGVPGEPGEPGQTGPQGPRGPPGPPGKAGEDGHPGKPGRPGERGVAGPQGARGFPGTPGLPGFKGIRGHNGLDGQKGQPGTPGTKGEPGAPGENGTPGQPGARGLPGERGRIGAPGPAGARGSDGSAGPTGPAGPIGAAGPPGFPGAPGAKGEIGPAGNVGPTGPAGPRGEIGLPGSSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPPGPAGPSGARGLVGEPGPAGAKGESGNKGEPGAAGPPGPPGPSGEEGKRGSNGEPGSAGPPGPAGLRGVPGSRGLPGADGRAGVMGPAGNRGASGPVGAKGPNGDAGRPGEPGLMGPRGLPGQPGSPGPAGKEGPVGFPGADGRVGPIGPAGNRGEPGNIGFPGPKGPTGEPGKPGEKGNVGLAGPRGAPGPEGNNGAQGPPGVTGNQGAKGETGPAGPPGFQGLPGPSGPAGEAGKPGERGLHGEFGVPGPAGPRGERGLPGESGAVGPAGPIGSRGPSGPPGPDGNKGEPGNVGPAGAPGPAGPGGIPGERGVAGVPGGKGEKGAPGLRGDTGATGRDGARGLPGAIGAPGPAGGAGDRGEGGPAGPAGPAGARGIPGERGEPGPVGPSGFAGPPGAAGQPGAKGERGPKGPKGETGPTGAIGPIGASGPPGPVGAAGPAGPRGDAGPPGMTGFPGAAGRVGPPGPAGITGPPGPPGPAGKDGPRGLRGDVGPVGRTGEQGIAGPPGFAGEKGPSGEAGAAGPPGTPGPQGILGAPGILGLPGSRGERGLPGIAGATGEPGPLGVSGPPGARGPSGPVGSPGPNGAPGEAGRDGNPGNDGPPGRDGAPGFKGERGAPGNPGPSGALGAPGPHGQVGPSGKPGNRGDPGPVGPVGPAGAFGPRGLAGPQGPRGEKGEPGDKGHRGLPGLKGHNGLQGLPGLAGQHGDQGPPGNNGPAGPRGPPGPSGPPGKDGRNGLPGPIGPAGVRGSHGSQGPAGPPGPPGPPGPPGPNGGGYEVGFDAEYYR'},
#     'COL1A2-Gal_gal?': {'species': 'Gallus gallus','offset': -2, 'sequence': 'AGSVGMGYPPQPISGFPGPPGPSGPPGPPGHAGPPGSNGYQGPPGEPGQPGPSGPPGPAGMIGPAGPPGKDGEPGRPGRNGDRGIPGLPGHKGHPGMPGMPGMKGARGFDGKDGAKGDSGAPGPKGEAGQPGANGSPGQPGPRGPTGERGRPGNPGGPGAHGKDGAPGAAGPPGPPGPPGTAGFPGSPGFKGEAGPPGPAGASGSPGERGEPGPQGQAGPPGPQGPPGRAGSPGNKGEMGPSGIPGAPGLPGGRGLPGPPGTSGNPGAKGTPGEPGKNGAKGDPGPKGERGENGTPGAPGPPGEEGKRGANGEPGQNGVPGTPGERGSPGFRGLPGSNGLPGEKGPAGERGSPGPPGPSGPAGDRGQDGGPGLPGMRGLPGIPGSPGSDGKPGPPGNQGEPGRSGPPGPAGPRGQPGVMGFPGPKGNEGAPGKNGERGPGGPPGTPGPAGKNGDVGLPGPPGPAGPAGDRGEPGPSGSPGLQGLPGGPGPAGENGKPGEPGPKGDIGGPGFPGPKGENGIPGERGAQGPPGPTGARGGPGPAGSEGAKGPPGPPGAPGGTGLPGLQGMPGERGASGSPGPKGDKGEPGGKGADGLPGARGERGNVGPIGPPGPAGPPGDKGETGPAGAPGPAGSRGGPGERGEQGLPGPAGFPGAPGQNGEPGGKGERGPPGLRGEAGPPGAAGPQGGPGAPGPPGPQGVKGERGSPGGPGAAGFPGARGLPGPPGNNGSPGPPGNAGPPGKDGPPGPPGNTGPPGGSGPPGLRGEPGAPGEKGPPGARGERGTPGDPGPQGIIGSRGSTGLPGPRGLPGPAGMAGGKGEDGKPGVNGVPGERGAPGPQGPMGQRGLPGEPGRDGNPGSDGSPGRDGSPGGKGDRGESGPPGVPGPPGHPGPAGNNGAPGKAGERGFQGPPGPPGSAGPAGARGPAGPQGPRGDKGETGERGSAGIKGHRGFPGTPGLPGPPGPLGPQGAIGSPGASGARGPPGPAGPPGKDGRGGYPGPIGPPGPRGNRGESGPAGPPGQPGLPGPSGPPGPCCGGGVASLGAGEK'},
#     'COL1A2-Ayf_COL1A2': {'species': 'Ayf xxx','offset': -0, 'sequence': ' GPMGLMGPRGPPGASGPPGPPGFQGVPGEPGEPGQTGPQGPRGPPGPPGKAGEDGHPGKPGRPGERGVAGPQGARGFPGTPGLPGFKGIRGHNGLDGQKGQPGTPGTKGEPGAPGENGTPGQPGARGLPGERGRVGAPGPAGARGSDGSAGPTGPAGPIGAAGPPGFPGAPGAKGEIGPAGNVGPTGPAGPRGEIGLPGSSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPPGPAGPSGARGLVGEPGPAGAKGESGNKGEPGAAGPPGPPGPSGEEGKRGSNGEPGSAGPPGPAGLRGVPGSRGLPGADGRAGVMGPAGNRGASGPVGAKGPNGDGGRPGEPGLMGPRGLPGQPGSPGPAGKEGPVGFPGADGRVGPIGPAGNRGEPGNIGFPGPKGPTGEPGKPGEKGNVGLAGPRGAPGPEGNNGAQGPPGVTGNQGAKGEQGPAGPPGFQGLPGPSGPAGEAGKPGERGLHGEFGVPGPAGPRGERGPPGESGAVGPAGPIGSRGPSGPPGPDGNKGEPGNVGAAGGPGPAGPGGIPGERGVAGVPGGKGEKGAPGLRGDTGATGRDGARGLPGAIGAPGPAGGAGDRGEGGPAGPAGPAGARGIPGERGEPGPVGPSGFAGPPGAAGQPGAKGERGPKGPKGETGPTGAIGPIGASGPPGPAGAAGPAGPRGDAGPPGMTGFPGAAGRVGPPGPAGITGPPGPPGPAGKDGARGLRGDVGPVGRTGEQGIAGPPGFAGEKGPSGEAGAPGPPGTPGPQGILGAPGILGLPGSRGERGLPGISGATGEPGPLGVSGPPGARGPSGPVGSPGPNGAPGEAGRDGNPGNDGPPGRDGAPGFKGERGAPGSPGPSGALGAPGPHGQVGPAGKPGNRGDPGPAGHVGPAGAFGPRGLAGPQGPRGEKGEPGDKGHRGLPGLKGHNGLQGLPGLAGQHGDQGPPGNNGPAGPRGPPGPSGPPGKDGRNGLPGPIGPAGARGSHGSQGPAGPPGPPGPPGPPGPNGGGYEVGYDAEYYRADQPSLRPK'},
#     'COL1A2-Com_COL1A2': {'species': 'Com xxx','offset': -0, 'sequence': ' GPMGLMGPRGPPGASGPPGPPGFQGLPGEPGEPGQTGPQGPRGPPGPPGKAGEDGHPGKPGRPGERGVAGPQGARGFPGTPGLPGFKGIRGHNGLDGQKGQPGTPGSKGEPGAPGENGTPGQPGARGLPGERGRIGAPGPAGARGSDGSTGPTGPAGPIGAAGPPGFPGAPGAKGEIGPAGNVGPTGPAGPRGEIGLPGSSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGILGPPGPAGPSGARGLVGEPGPAGAKGESGMKGEPGAAGPAGPPGPSGEEGKRGSPGEPGSAGPPGPAGLRGVPGSRGLPGADGRAGVMGPAGNRGASGPVGAKGPSGDAGRPGEPGLMGPRGLPGQPGSPGPAGKEGPVGFPGADGRVGPIGPAGNRGEPGNIGFPGPKGPTGEPGKPGEKGNVGLAGPRGAPGPEGNNGAQGPPGVTGNPGGKGETGPAGPPGFQGLPGPSGPAGEAGKPGERGLHGEFGVPGPAGPRGERGPPGESGAVGPAGAIGSRGPSGPPGPDGNKGEPGNVGAAGAPGPAGPGGIPGERGVAGVPGGKGEKGAPGLRGDTGATGRDGARGLPGAIGAPGPAGGAGDRGEGGPAGPAGPAGARGIPGERGEPGPVGPNGFAGPPGAAGQPGAKGERGPKGPKGESGPTGAIGPIGASGPPGPVGAAGPAGPRGDAGPPGMTGFPGAAGRVGPPGPAGITGPPGPPGPAGKDGPRGLRGDVGPVGRTGEQGIAGPPGFAGEKGPSGEAGAAGPPGTPGPQGILGAPGILGLPGSRGERGLPGISGATGEPGPLGVSGPPGARGPSGPVGSPGPNGAPGEAGRDGNPGNDGPPGRDGAPGFKGERGAPGSPGPSGALGAPGPHGQVGPSGKPGNRGEPGPAGAVGPAGAFGPRGLAGPQGPRGEKGLPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGPPGNTGPAGPRGPPGPSGPPGKDGRNGLPGPIGPAGVRGSHGSQGPAGPPGPPGPPGPPGPNGGGYEVGFDAEYYRADQPSLRPK'},
#     'COL1A2-Stc_COL1A2': {'species': 'Struthio camelus','offset': -0, 'sequence': ' GPMGLMGPRGPPGASGPPGPPGFQGLPGEPGEPGQTGPQGPRGPPGPPGKAGEDGHPGKPGRPGERGVAGPQGARGFPGTPGLPGFKGIRGHNGLDGQKGQPGTPGSKGEPGAPGENGTPGQPGARGLPGERGRIGAPGPAGARGSDGSTGPTGPAGPIGAAGPPGFPGAPGAKGEIGPAGNVGPTGPAGPRGEIGLPGSSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGILGPPGPAGPSGARGLVGEPGPAGAKGESGMKGEPGAAGPAGPPGPSGEEGKRGSPGEPGSAGPPGPAGLRGVPGSRGLPGADGRAGVMGPAGNRGASGPVGAKGPSGDAGRPGEPGLMGPRGLPGQPGSPGPAGKEGPVGFPGADGRVGPIGPAGNRGEPGNIGFPGPKGPTGEPGKPGEKGNVGLAGPRGAPGPEGNNGAQGPPGVTGNPGGKGETGPAGPPGFQGLPGPSGPAGEAGKPGERGLHGEFGVPGPAGPRGERGPPGESGAVGPAGAIGSRGPSGPPGPDGNKGEPGNVGAAGAPGPAGPGGIPGERGVAGVPGGKGEKGAPGLRGDTGATGRDGARGLPGAIGAPGPAGGAGDRGEGGPAGPAGPAGARGIPGERGEPGPVGPNGFAGPPGAAGQPGAKGERGPKGPKGESGPTGAIGPIGASGPPGPVGAAGPAGPRGDAGPPGMTGFPGAAGRVGPPGPAGITGPPGPPGPAGKDGPRGLRGDVGPVGRTGEQGIAGPPGFAGEKGPSGEAGAAGPPGTPGPQGILGAPGILGLPGSRGERGLPGISGATGEPGPLGVSGPPGARGPSGPVGSPGPNGAPGEAGRDGNPGNDGPPGRDGAPGFKGERGAPGSPGPSGALGAPGPHGQVGPSGKPGNRGEPGPAGAVGPAGAFGPRGLAGPQGPRGEKGLPGDKGPRGLPGLKGHNGLQGLPGLAGQHGDQGPPGNTGPAGPRGPPGPSGPPGKDGRNGLPGPIGPAGVRGSHGSQGPAGPPGPPGPPGPPGPNGGGYEVGFDAEYYRADQPSLRPK'},
#     'COL1A2-EquxPrez': {'species': 'EquusxPrezwal','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPAGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQAGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPAGAAGATGARGLVGEPGPAGSKGESGNKGEPGAAGPQGPPGPSGEEGKRGPNGEPGSTGPAGPPGLRGSPGSRGLPGADGRAGVMGPAGSRGASGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGEPGKPGDKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEVGKPGERGLPGEFGLPGPAGARGERGPPGESGAAGPAGPIGSRGPSGPPGPDGNKGEPGVLGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGEIGNPGRDGARGAPGAVGAPGPAGANGDRGEAGAAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGVTGFPGAAGRTGPPGPSGISGPPGPPGAAGKEGLRGPRGDQGPVGRAGETGASGPPGFAGEKGPSGEPGTAGPPGTPGPQGLLGAPGILGLPGSRGERGLPGVAGSLGEPGPLGIAGPPGARGPPGAVGAPGVNGAPGEAGRDGNPGSDGPPGRDGQPGHKGERGYPGNAGPVGAVGAPGPHGPVGPTGKHGHRGEPGPVGSVGPVGAVGPRGPSGPQGVRGDKGEPGDKGPRGLPGIKGHNGLQGLPGLAGQHGDQGAPGSVGPAGPRGPAGPTGPVGKDGRSGQPGTVGPAGVRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGYDGDFYRA'},
#     'COL1A2-Bub_Bub': {'species': 'Bubalus bubalis','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGPAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPSGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGVRGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAVGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPAGISGPPGPPGPAGKEGLRGPRGDQGPVGRTGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGLPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPVGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGSVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYDFGFDGDFYRA'},
#     'COL1A2-Bos_ind': {'species': 'Bos indicus','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAIGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGISGPPGPPGPAGKEGLRGPRGDQGPVGRSGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGAPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPXGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYEFGFDGDFYRA'},
#     'COL1A2-Bis_bis': {'species': 'Bison bison','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAIGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGISGPPGPPGPAGKEGLRGPRGDQGPVGRSGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGAPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPVGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYEFGFDGDFYRA'},
#     'COL1A2-Bosxbos': {'species': 'BosxBos','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAIGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGISGPPGPPGPAGKEGLRGPRGDQGPVGRSGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGAPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPVGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYEFGFDGDFYRA'},
#     'COL1A2-Phy_cas': {'species': 'Physter catodon','offset': -8, 'sequence': 'QYDGKGVGIGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKSGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGTPGVKGEPGAPGENGIPGQVGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPLGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGSRGEVGLPGVSGPVGPPGNPGVNGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGDSGNKGEPGAVGPTGPPGPSGEEGKRGTTGEIGSAGPPGPPGLRGNPGSRGLPGADGRAGVMGLHGSRGGTGPAGVRGPSGDSGRPGEPGLMGPRGFPGSPGNAGPAGKEGPTGLPGIDGRPGPIGPAGTRGEPGNIGFPGPKGPTGDPGKNGEKGHAGLAGPRGAPGPDGNNGAQGPPGLQGVSGGKGEQGPAGPPGFQGLPGPAGTAGEAGKAGERGIPGEFGLPGPAGPRGERGPPGESGAAGPAGPIGSRGPSGPAGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDVGSHGRDGARGAPGAVGAPGPAGANGDRGEAGPAGAAGPAGSRGSPGERGEVGPAGPNGFAGSAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPAGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGITGPPGPPGPAGKEGLRGPRGDQGPVGRTGETGASGPPGFVGEKGPSGEPGTAGSPGTPGPQGLLGSPGFLGLPGSRGERGLPGVAGSLGEPGPLGIAGPTGARGPPGAVGNPGVNGAPGEAGRDGNPGSDGPPGRDGQAGHKGERGYPGNAGPTGTAGAPGPQGPVGPTGKHGNRGESGPSGPVGLAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGTVGPAGPRGPAGPSGPSGKDGRTGHPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPAGPSGGGYDFGFEGDFYRA'},
#     'COL1A2-Bis_bis': {'species': 'Bison bison','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAIGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGISGPPGPPGPAGKEGLRGPRGDQGPVGRSGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGAPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPVGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYEFGFDGDFYRA'},
#     'COL1A2-Bos_mut': {'species': 'Bubalus bubalis','offset': -7, 'sequence': 'QFDAKGGGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGARGPPGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGELGPVGNPGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLPGAKGAAGLPGVAGAPGLPGPRGIPGPVGASGATGARGLVGEPGPAGSKGESGNKGEPGAVGQPGPPGPSGEEGKRGSTGEIGPAGPPGPPGLRGNPGSRGLPGADGRAGVMGPAGSRGATGPAGVRGPNGDSGRPGEPGLMGPRGFPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPSGDPGKAGEKGHAGLAGARGAPGPDGNNGAQGPPGLQGVQGGKGEQGPAGPPGFQGLPGPAGTAGEAGKPGERGIPGEFGLPGPAGARGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAPGTAGPSGPSGLPGERGAAGIPGGKGEKGETGLRGDIGSPGRDGARGAPGAIGAPGPAGANGDRGEAGPAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGTKGPKGENGPVGPTGPVGAAGPSGPNGPPGPAGSRGDGGPPGATGFPGAAGRTGPPGPSGISGPPGPPGPAGKEGLRGPRGDQGPVGRSGETGASGPPGFVGEKGPSGEPGTAGPPGTPGPQGLLGAPGFLGLPGSRGERGLPGVAGSVGEPGPLGIAGPPGARGPPGNVGNPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNAGPVGAAGAPGPQGPVGPVGKHGNRGEPGPAGAVGPAGAVGPRGPSGPQGIRGDKGEPGDKGPRGLPGLKGHNGLQGLPGLAGHHGDQGAPGAVGPAGPRGPAGPSGPAGKDGRIGQPGAVGPAGIRGSQGSQGPAGPPGPPGPPGPPGPSGGGYEFGFDGDFYRA'},
#     'COL1A1-Bos_mut': {'species': 'Bos mutus','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEGGPQGPRGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGSKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPAGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGAPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGAPGKDGVRGLTGPIGPPGPAGAPGDKGEAGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPAGPPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGSKGPRGETGPAGRPGEVGPPGPPGPAGEKGAPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPMGPPGLAGPPGESGREGAPGAEGSPGRDGSPGAKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPIGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGSPGKDGLNGLPGPIGPPGPRGRTGDAGPAGPPGPPGPPGPPGPPSGGYDLSFLPQPPQEKAHDGGR'},
#     'COL1A1-Cam_fer': {'species': 'Bos taurus','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGSSGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEAGPQGARGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGNKGDTGAKGEPGPTGVQGPPGPAGEEGKRGARGEPGPAGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGSPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGSPGKDGVRGLTGPIGPPGPAGAPGDKGETGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPTGPPGPIGSVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPVGKEGSKGPRGETGPAGRPGEVGPPGPPGPAGEKGAPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGPNGERGPPGPMGPPGLAGPPGESGREGAPGAEGSPGRDGSPGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKSGDRGETGPAGPAGPIGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGAPGKDGLNGLPGPIGPPGPRGRTGDAGPVGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEKAGGDGR'},
#     'COL1A1-Vul_vul': {'species': 'Bos taurus','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARGLPGTAGLPGMKGHRGFSGLDGAKGDAGPAGPKGEPGSPGENGAPGQMGPRGLPGERGRPGAPGPAGARGNDGATGAAGPPGPTGPAGPPGFPGAVGAKGEAGPQGARGSEGPQGVRGEPGPPGPAGAAGPAGNPGADGQPGAKGANGAPGIAGAPGFPGARGPSGPQGPSGPPGPKGNSGEPGAPGNKGDTGAKGEPGPTGIQGPPGPAGEEGKRGARGEPGPTGLPGPPGERGGPGSRGFPGADGVAGPKGPAGERGSPGPAGPKGSPGEAGRPGEAGLPGAKGLTGSPGSPGPDGKTGPPGPAGQDGRPGPPGPPGARGQAGVMGFPGPKGAAGEPGKAGERGVPGPPGAVGPAGKDGEAGAQGPPGPAGPAGERGEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGARGERGFPGERGVQGPPGPAGPRGANGAPGNDGAKGDAGAPGAPGSQGAPGLQGMPGERGAAGLPGPKGDRGDAGPKGADGSPGKDGVRGLTGPIGPPGPAGAPGDKGEAGPSGPAGPTGARGAPGDRGEPGPPGPAGFAGPPGADGQPGAKGEPGDAGAKGDAGPPGPAGPTGPPGPIGNVGAPGPKGARGSAGPPGATGFPGAAGRVGPPGPSGNAGPPGPPGPAGKEGGKGPRGETGPAGRPGEVGPPGPPGPAGEKGSPGADGPAGAPGTPGPQGIAGQRGVVGLPGQRGERGFPGLPGPSGEPGKQGPSGASGERGPPGPMGPPGLAGPPGESGREGSPGAEGSPGRDGSPGPKGDRGETGPAGPPGAPGAPGAPGPVGPAGKNGDRGETGPAGPAGPIGPVGARGPAGPQGPRGDKGETGEQGDRGIKGHRGFSGLQGPPGPPGSPGEQGPSGASGPAGPRGPPGSAGSPGKDGLNGLPGPIGPPGPRGRTGDAGPVGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEKAHDGGR'},
#     'COL3A1-BosxBos': {'species': 'BosxBos','offset': -17, 'sequence': 'QYEAYDVKSGIAGGGIAGYPGPAGPPGPPGPPGTSGHPGAPGAPGYQGPPGEPGQAGPAGPPGPPGAIGPSGPAGKDGESGRPGRPGERGFPGPPGMKGPAGMPGFPGMKGHRGFDGRNGEKGETGAPGLKGENGVPGENGAPGPMGPRGAPGERGRPGLPGAAGARGNDGARGSDGQPGPPGPPGTAGFPGSPGAKGEVGPAGSPGSSGAPGQRGEPGPQGHAGAPGPPGPPGSNGSPGGKGEMGPAGIPGAPGLIGARGPPGPPGTNGVPGQRGAAGEPGKNGAKGDPGPRGERGEAGSPGIAGPKGEDGKDGSPGEPGANGLPGAAGERGVPGFRGPAGANGLPGEKGPPGDRGGPGPAGPRGVAGEPGRDGLPGGPGLRGIPGSPGGPGSDGKPGPPGSQGETGRPGPPGSPGPRGQPGVMGFPGPKGNDGAPGKNGERGGPGGPGPQGPAGKNGETGPQGPPGPTGPSGDKGDTGPPGPQGLQGLPGTSGPPGENGKPGEPGPKGEAGAPGIPGGKGDSGAPGERGPPGAGGPPGPRGGAGPPGPEGGKGAAGPPGPPGSAGTPGLQGMPGERGGPGGPGPKGDKGEPGSSGVDGAPGKDGPRGPTGPIGPPGPAGQPGDKGESGAPGVPGIAGPRGGPGERGEQGPPGPAGFPGAPGQNGEPGAKGERGAPGEKGEGGPPGAAGPAGGSGPAGPPGPQGVKGERGSPGGPGAAGFPGGRGPPGPPGSNGNPGPPGSSGAPGKDGPPGPPGSNGAPGSPGISGPKGDSGPPGERGAPGPQGPPGAPGPLGIAGLTGARGLAGPPGMPGARGSPGPQGIKGENGKPGPSGQNGERGPPGPQGLPGLAGTAGEPGRDGNPGSDGLPGRDGAPGAKGDRGENGSPGAPGAPGHPGPPGPVGPAGKSGDRGETGPAGPSGAPGPAGSRGPPGPQGPRGDKGETGERGAMGIKGHRGFPGNPGAPGSPGPAGHQGAVGSPGPAGPRGPVGPSGPPGKDGASGHPGPIGPPGPRGNRGERGSEGSPGHPGQPGPPGPPGAPGPCCGAGGVAAIAGVGAEKAGGFAPYYG'},
#     'AHSG-Bos_tau' : {'species': 'Bos taurus','offset': 0, 'sequence':'IPLDPVAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLANCSVRQQTQHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNDSRVVHAVEVALATFNAESNGSYLQLVEISRAQFVPLPVSVSVEFAVAATDCIAKEVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVRVTCTLFQTQPVIPQPQPDGAEAEAPSAVPDAAGPTPSAAGPPVASVVVGPSVVAVPLPLHRAHYDLRHTFSGVASVESSSGEAFHVGKTPIVGQPSIPGGPVRLCPGRIRYFKI'},
#     'AHSG-Bub_bub' : {'species': 'Bubalus bubalis','offset': 0, 'sequence':'IPLDPVAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLANCSVRQQTEHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNDSRVVHAVEVALATFNAQSNGSYLQLVEISRAQFVPLPASVSVEFAVAATDCIAKDVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVAVTCTLFQTQPVILQPQPDGAEAGAPSAVPDAAGPAPSAAGPPVASVVVGPSVVAVPLPLHRAHYDLRHTFSGVASVESASGEAFHVGKTPIVGQPSVPGGPVRLCPGRIRYFKI'},
#     'AHSG-Equ_cab' : {'species': 'Equus caballus','offset': 1, 'sequence':' LPNGLSPAYRQLNCDDPETEQAALLAVDYINSHIHQGYKHVLNQIDKVQVWAQPTGESFKLEIDTLETTCHALDPTPLANCSVRQLTQHAVEGDCDVRLLKQNGQFSVSFVKCKSSPDSAEDVRKVCLDCPLLAPLNDTRVVHAVEAALAAFNAQNNGSYFQLVEISRAQLVPLPVSVHVEFAVAATDCVAKEVIDPAKCNLLAEKQYGFCKATLTEKVGGEDVAVTCTVFQTQPVVLLPQPDGPDVGVPGPVADAVTPAPSPADLPVASLVVGPVVVAASQLPPPVHRAHYDLRHAFAGVGSGESASGEAFHVEKPPKVAHPNTAAAAGPVVRPCPGRIRYFKII'},
#     'AHSG-Ovi_are' : {'species': 'Ovis aries','offset': 1, 'sequence':'IPLDPIAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLVNCSVRQQTEHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNNSQVVHAAEVALATFNAQNNGSYFQLVEISRAQFVPLPGSVSVEFAVAATDCIAKEVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVTVTCTLFQTQPVIPQPQPEGAEAGAPSAVPDAAVPDAAVPAPSAAGLPVGSVVAGPSVVAVPLPLHRAHYDLRHTFSGVASVESASGEAFHVGKTPIVGQPSVPGGPVHLCPGRIRYFKI'},
#     'AHSG-Cap_hir' : {'species': 'Capra hircus','offset': 1, 'sequence':'IPLDPIAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLANCSVRQQTEHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNNSQVVHAAEVALATFNAQNNGSYFQLVEISRAQFVPLPVSVSVEFAVAATDCIAKEVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVAVTCTLFQTQPVIPQPQPEGAEAGAPSAVPDAAVPAPSAAGLPVGSVVAGPSVVAVALPLHRAHYDLRHTFSGVASVESASGEAFHVGKTPIVGQPSVPGGPVHLCPGRIRYFKI'},
#     'AHSG-Equ_asn' : {'species': 'Equus asinus','offset': 1, 'sequence':' LPNGLSPAYRQLNCDDPETEQAALLAVDYINSHIHQGYKHVLNQIDKVQVWAQPTGESFKLEIDTLETTCHALDPTPLANCSVRQLTQHAVEGDCDVRLLKQNGQFSVSFVKCKSSPDSAEDVRKVCLDCPLLAPLNDTRVVHAVEAALAAFNAQNNGSYFQLVEISRAQLVPLPVSVHVEFAVAATDCVAKEVIDPAKCNLLAEKQYGFCKATLTEKVGGEDVAVTCTVFQTQPVVPLPQPDGPDVGVPGPVADAATPEPSPADLPVASLVVGPVVVAAPQLPPPVHRAHYDLRHAFAGVGSGESASGEAFHVEKPPKVAHPNTAAAAGPVVRPCPGRIRYFKI'},
#     'yNFM-Pse_aer' : {'species': 'Pseudomonas aeruginosa','offset': 1, 'sequence':'MALAAPFVGRLSGGRERSRVIAFAASLLCVPMLLAAHAGSFAAFLLWRFAEGMLIPLLFASSVAYIGERWSGGAVIELTSLYVAGTILGGFAGRFLTGLLTEEWDWRVAFRILAGLTLLIAVAIRALLPANSGRHAPAPGKRPLVSRELFRAPLLSSYAVGFCVLFSQVAMFTYIGIHLSRPPYSLDTAQLGSIYAVFLLALLVIPASGRLARTRPHRQLLLGASLLGVCGSLLTLAPGLPTILVGLALSATGVFLAQSTVNAFTASNAGTDKAGAVGLYLTCYYAGGSLGAVLPAPFWERWGWPGCLGLILLAQLLPLLLTRYGWSAPQAERAALRASQGRP'},
#     'PIV-Pse_aer' : {'species': 'Pseudomonas keratitis','offset': 1, 'sequence':'MHKRTYLNACLVLALAAGASQASAAPGASEMAGDVAVLQASPASTGHARFANPNAATSAAGIHFAAPPARRVARAAPLAPKPGTPLQVGVGLKTATPEIDLATLEWIDTPDGRHTARFPISAAGAASLRAAIRLETRSGSLPDDVLLHFAGAGKEIFEASGKDLSXNRPYWSPVIEGDTLTVELVLPAXLQPGDLRLSVPQVSYFADSLYKAGYRDGFGASGSCEVDAVCATQSGTRAYDNATAAVAKMVFTSSADGGSYICTGTLLNNGNSPKRQLFWSAAHCIEDQATAATLQTIWFYNTTQCYGDASTINQSVTVLTGGANILHRDAKRDTLLLELKRTPPAGVFYQGWSATPIANGSLGHDIHHPRGDAKKYSQGNVSAVGVTYDGHTALTRVDWPSAVVEGGSSGSGLLTVAGDGSYQLRGGLYGGPSYCGAPTSQRNDYFSDFSGVYSQISRYFAP'}


#     }



###Enzymatic Cleavage Sites

In [7]:
def find_cleavage_sites_with_offset(collagen_sequences: dict) -> dict:
    """
    Find the positions of the letters 'K' and 'R' in given protein sequences stored in a dictionary,
    adjusting for an offset provided for each sequence.

    Parameters:
        collagen_sequences (dict): A dictionary where each key is a unique identifier for a protein,
                                   and each value is another dictionary with keys 'offset' and 'sequence'
                                   representing the offset from the start of the protein and the amino
                                   acid sequence, respectively.

    Returns:
        dict: A dictionary where each key is the protein name (GNC) and each value is a list of adjusted
              positions (1-based) where either 'K' or 'R' occurs in the sequence, considering the offset.
    """
    cleavage_sites_results = {}

    for key, value in collagen_sequences.items():
        # Use the new extract_gene_identifier function to get the gene name
        protein_name = extract_gene_identifier(key)

        sequence = value['sequence']
        offset = value['offset']
        cleavage_sites = []

        for i, amino_acid in enumerate(sequence):
            if amino_acid in ['K', 'R']:
                position_with_offset = i + 1 + offset
                cleavage_sites.append(position_with_offset)

        cleavage_sites_results[protein_name] = cleavage_sites

    return cleavage_sites_results


def extract_gene_identifier(full_gene_name):
    """
    Extracts the gene identifier from the full gene name.

    Parameters:
    - full_gene_name (str): The full gene name, e.g., 'COL3A1-Bos_mut'.

    Returns:
    - str: The extracted gene identifier, e.g., 'COL3A1'.
    """
    # Split the full gene name at the hyphen and return the first part
    return full_gene_name.split('-')[0]


In [8]:
# Merge sequence dictionaries
all_sequences = {**sequences_dict,
                # **non_sequences_dict
                 }
# Find cleavage sites considering the offsets
cleavage_sites_with_offset = find_cleavage_sites_with_offset(all_sequences)

for protein, sites in cleavage_sites_with_offset.items():
    print(f"{protein}: {sites}")

COL1A1: [9, 42, 50, 59, 62, 66, 75, 87, 90, 99, 108, 126, 132, 134, 144, 174, 183, 192, 219, 237, 252, 264, 270, 290, 291, 294, 309, 315, 327, 333, 342, 350, 360, 374, 386, 396, 408, 416, 420, 434, 453, 479, 498, 501, 507, 519, 531, 555, 564, 567, 573, 581, 585, 603, 618, 624, 648, 657, 684, 687, 704, 725, 729, 732, 740, 756, 780, 789, 792, 806, 816, 836, 848, 855, 858, 884, 888, 906, 915, 918, 927, 930, 933, 963, 974, 990, 992, 1030, 1036]
COL1A2: [-3, 12, 45, 53, 62, 65, 69, 78, 90, 93, 102, 111, 129, 135, 137, 147, 177, 195, 222, 240, 255, 267, 273, 293, 294, 312, 318, 326, 336, 345, 353, 363, 377, 389, 399, 411, 419, 423, 432, 456, 482, 486, 501, 504, 522, 534, 558, 567, 570, 576, 584, 588, 606, 621, 627, 651, 654, 657, 660, 690, 707, 728, 732, 735, 743, 759, 792, 795, 819, 839, 851, 858, 861, 887, 891, 909, 918, 921, 927, 930, 936, 966, 977, 980, 993, 1031]
COL3A1: [-9, 59, 65, 68, 72, 81, 93, 96, 101, 105, 114, 132, 138, 140, 150, 156, 180, 198, 225, 243, 258, 266, 270, 276, 279,

##Sample Names

In [9]:
# Define a dictionary to map old names to new names
rename_dict = {
    'M3_20231109-1713': 'M3',
    'M1_20231109-1625': 'M1',
    'M4_20231109-1737': 'M4',
    'P3_20231109-1848': 'P3',
    'P4_20231109-1912': 'P4',
    'P2_20231109-1824': 'P2',
    'P1_20231109-1801': 'P1',
    'M2_20231109-1649': 'M2',
    '20230614-0225_QEHF3_1005194_ONJ_TR_MatthewEvo_13_LLM_C_WhiteP_02_OBi.raw' :'02_OBi_Pak_P',
    '20230614-0314_QEHF3_1005195_ONJ_TR_MatthewEvo_14_LLM_C_PinkP_12_ICib.raw' :' 12_ICib_Pak_P',
    '20230614-0402_QEHF3_1005196_ONJ_TR_MatthewEvo_15_LLM_C_Glue_14_OL.raw' :' 14_OL_Pak_FG',
    '20231109-1138_QEHF2_1007125_ONJ_TR_MJC_2_D7_Cz2.raw' :' Cz2_CFrag_FG',
    '20231109-1203_QEHF2_1007126_ONJ_TR_MJC_2_D8_03_OBO.raw' :'03_OBO_Pak_P',
    '20231109-1227_QEHF2_1007127_ONJ_TR_MJC_2_D9_16_OL.raw' :'16_OL_Pak_FG',
    '20231109-1251_QEHF2_1007128_ONJ_TR_MJC_2_D10_13_ICib.raw' :'13_ICib_Pak_W',
    '20231109-1315_QEHF2_1007129_ONJ_TR_MJC_2_D11_05_ICB.raw' :'05_ICB_Pak_P',
    '20231109-1339_QEHF2_1007130_ONJ_TR_MJC_2_D12_06_ICB.raw' :'06_ICB_Pak_FG',
    '20231109-1402_QEHF2_1007131_ONJ_TR_MJC_2_E1_09_ICB.raw' :'09_ICB_Pak_FG',
    '20231109-1426_QEHF2_1007132_ONJ_TR_MJC_2_E2_10_ICB.raw' :'10_ICB_Pak_Fiber',
    '20231109-1450_QEHF2_1007133_ONJ_TR_MJC_2_E3_Cz17.raw' :'Cz17_CFrag_W',
    '20231109-1514_QEHF2_1007134_ONJ_TR_MJC_2_E4_Cz11.raw' :'Cz11_CFrag_W',
    '20231109-1538_QEHF2_1007135_ONJ_TR_MJC_2_E5_Cz5b.raw' :'Cz5b_CFrag_W',
    '20231109-1538_QEHF2_1007135_ONJ_TR_MJC_2_E5_Cz5b.raw' :'Cz5b_CFrag_W',
    # '20240209-0622_QEHF2_1007825_ONJ_TR_MC_CZ21.raw' :'Cz21',
    # '20240209-0558 QEHF2_1007824_ONJ_TR_MC_CZ20.raw' :'Cz20',
    # '20240209-0534 QEHF2_1007823_ONJ_TR_MC_CZ19.raw' :'Cz19',
    # '20240209-0510_QEHF2_1007822_ONJ_TR_MC_CZ18.raw' :'Cz18',
    # '20240209-0422_QEHF2_1007820_ONJ_TR_MC_CZ17.raw' :'Cz17',
    # '20240209-0359 QEHF2_1007819_ONJ_TR_MC_CZ16.raw' :'Cz16',
    # '20240209-0335_QEHF2_1007818_ONJ_TR_MC_CZ15.raw' :'Cz15',
    # '20240209-0311_QEHF2_1007817_ONJ_TR_MC_CZ14.raw' :'Cz14',
    # '20240209-0247_QEHF2_1007816_ONJ_TR_MC_CZ13.raw' :'Cz13',
    # '20240209-0223_QEHF2_1007815_ONJ_TR_MC_CZ12.raw' :'Cz12',
    # '20240209-0159_QEHF2_1007814_ONJ_TR_MC_CZ11.raw' :'Cz11',
    # '20240209-0136_QEHF2_1007813_ONJ_TR_MC_CZ10.raw' :'Cz10',
    # '20240209-0112_QEHF2_1007812_ONJ_TR_MC_CZ9.raw' :'Cz9',
    # '20240209-0048_QEHF2_1007811_ONJ_TR_MC_CZ8.raw' :'Cz8',
    # '20240209-0024_QEHF2_1007810_ONJ_TR_MC_CZ7.raw' :'Cz7',
    # '20240209-0000_QEHF2_1007809_ONJ_TR_MC_CZ6.raw' :'Cz6',
    # '20240208-2337_QEHF2_1007808_ONJ_TR_MC_CZ5.raw' :'Cz5',
    # '20240208-2313_QEHF2_1007807_ONJ_TR_MC_CZ4.raw' :'Cz4',
    # '20240208-2249 QEHF2_1007806_ONJ_TR_MC_CZ3.raw' :'Cz3',
    # '20240208-2225 QEHF2_1007805_ONJ_TR_MC_CZ2.raw' :'Cz2',
    # '20240208-2201_QEHF2_1007804_ONJ_TR_MC_CZ1.raw' :'Cz1',
    '20240209-0645_QEHF2' :'Cz22',
    '20240209-0622_QEHF2' :'Cz21',
    '20240209-0558_QEHF2' :'Cz20',
    '20240209-0534_QEHF2' :'Cz19',
    '20240209-0510_QEHF2' :'Cz18',
    '20240209-0422_QEHF2' :'Cz17',
    '20240209-0359_QEHF2' :'Cz16',
    '20240209-0335_QEHF2' :'Cz15',
    '20240209-0311_QEHF2' :'Cz14',
    '20240209-0247_QEHF2' :'Cz13',
    '20240209-0223_QEHF2' :'Cz12',
    '20240209-0159_QEHF2' :'Cz11',
    '20240209-0136_QEHF2' :'Cz10',
    '20240209-0112_QEHF2' :'Cz9',
    '20240209-0048_QEHF2' :'Cz8',
    '20240209-0024_QEHF2' :'Cz7',
    '20240209-0000_QEHF2' :'Cz6',
    '20240208-2337_QEHF2' :'Cz5',
    '20240208-2313_QEHF2' :'Cz4',
    '20240208-2249_QEHF2' :'Cz3',
    '20240208-2225_QEHF2' :'Cz2',
    '20240208-2201_QEHF2' :'Cz1',
    '20240208-2201_QEHF2' :'Cz1',
    '20231109-1050_QEHF2' :'MACHINE_BSA',
    '20231109-1114_QEHF2' :'INJECTION_BLANK',
    '20231109-1602_QEHF2' :'INJECTION_BLANK',

}

###Amino Acid cand PTM colours

In [10]:
# Amino acids colors
amino_acids_colors = {
    "I": "#009688", "V": "#8bc34a", "B": "009688", "L": "#009688",
    "F": "#507351", "C": "#ffeb3b", "M": "#ffeb3b", "A": "#bdd54e",
    "G": "#9e9e9e", "T": "#ffc75e", "W": "#f49272", "S": "#ffc107",
    "Y": "#30802f", "P": "#607d8b", "H": "#673ab7", "Z": "average",
    "Q": "#f44336", "E": "#f44336", "N": "#e81e63", "D": "#f44336",
    "X": "#9d9e9e", "K": "#701637", "R": "#bd3e04"
}

# PTM colors - if a colour is not defined, the PTM is not plotted

ptm_colors = {
    "Hydroxylation": "#DFDEE3",
    "Deamidated": "#8F0030",
    "DeamidationN" :"#FF0008",
    "DeamidationQ" :"#C41E23",
    "Oxidation": "#A1CF6B",
    "Dioxidation": "#7AC74F",
    "Trioxidation ": "#61AB36",
    "Carboxy": "#f44336",
    "Dehydrated": "#c6d8f5",
    "Ammonia-loss": "#9A879D",
    "Phospho": "#F4845F",
    "Thiophospho": "#F25C54",
    "Carboxymethyl": "#7A3B69",
    "Formyl": "#9A879D",
    "Pentose": "#CE4257",
    "Arg->Orn": "#563F1B",
    "Sulfide": "#F7F052"
}

## Functions

### Directory functions

In [11]:
# def combine_spectra_files(base_paths):
#   """
#   Combines spectra files from multiple base paths into a single DataFrame.

#   Args:
#       base_paths: A list of base paths containing the "pFind-Filtered.spectra" files.

#   Returns:
#       A pandas DataFrame containing the combined spectra data (or None if no files found).
#   """
#   all_data = []
#   for base_path in base_paths:
#     spectra_file = f'{base_path}/result/pFind-Filtered.spectra'
#     try:
#       # Read the spectra file using pandas.read_csv
#       data = pd.read_csv(spectra_file)
#       all_data.append(data)
#     except FileNotFoundError:
#       print(f"File not found: {spectra_file}")

#   # Combine all dataframes into a single one (if any files were found)
#   if all_data:
#     combined_df = pd.concat(all_data, ignore_index=True)
#     return combined_df
#   else:
#     print("No spectra files found in provided paths.")
#     return None

# import pandas as pd

def combine_spectra_files(base_paths):
    """
    Combines spectra files from multiple base paths into a single DataFrame.
    Adjusted to handle tab-separated values (TSV) by specifying the sep parameter.

    Args:
        base_paths: A list of base paths containing the "pFind-Filtered.spectra" files.

    Returns:
        A pandas DataFrame containing the combined spectra data (or None if no files found).
    """
    all_data = []
    for base_path in base_paths:
        spectra_file = f'{base_path}result/pFind-Filtered.spectra'
        try:
            # Read the spectra file using pandas.read_csv with sep='\t' for tab-separated files
            data = pd.read_csv(spectra_file, sep='\t')
            all_data.append(data)
        except FileNotFoundError:
            print(f"File not found: {spectra_file}")

    # Combine all dataframes into a single one (if any files were found)
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df
    else:
        print("No spectra files found in provided paths.")
        return None


def find_spectra_files(base_path):
    """
    Walk through the directory structure starting from base_path to find all
    pFind-Filtered.spectra files.

    Parameters:
    - base_path (str): The base directory to start the search from.

    Returns:
    - list: A list of paths to pFind-Filtered.spectra files.
    """
    spectra_files = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == 'pFind-Filtered.spectra':
                spectra_files.append(os.path.join(root, file))
    return spectra_files

def combine_spectra_files_from_single_directory(spectra_files):
    """
    Combine spectra files into a single pandas DataFrame.

    Parameters:
    - spectra_files (list): A list of paths to pFind-Filtered.spectra files.

    Returns:
    - DataFrame: A combined DataFrame of all spectra files.
    """
    df_list = []
    for file in spectra_files:
        df = pd.read_csv(file, sep='\t')  # Assuming the file is tab-separated
        df_list.append(df)
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

### Dictionary Manipulations

In [12]:
# def levenshtein_distance(s1, s2):
#     """
#     Calculate the Levenshtein distance between two strings.
#     The Levenshtein distance measures the minimum number of single-character edits
#     (insertions, deletions, or substitutions) required to change one word into the other.

#     Parameters:
#     - s1 (str): The first string to compare.
#     - s2 (str): The second string to compare.

#     Returns:
#     - int: The Levenshtein distance between the two strings.
#     """
#     if len(s1) < len(s2):
#         return levenshtein_distance(s2, s1)

#     if len(s2) == 0:
#         return len(s1)

#     previous_row = range(len(s2) + 1)
#     for i, c1 in enumerate(s1):
#         current_row = [i + 1]
#         for j, c2 in enumerate(s2):
#             insertions = previous_row[j + 1] + 1
#             deletions = current_row[j] + 1
#             substitutions = previous_row[j] + (c1 != c2)
#             current_row.append(min(insertions, deletions, substitutions))
#         previous_row = current_row

#     return previous_row[-1]



def find_match_and_update(row, sequence_position_dict, NCP_peptide_positions_dict):
    """
    Update the row based on matches found in sequence_position_dict and NCP_peptide_positions_dict.

    Parameters:
    - row: A pandas Series representing a row in a DataFrame.
    - sequence_position_dict: A dictionary containing matched sequences with gene data.
    - NCP_peptide_positions_dict: A dictionary containing NCP peptide positions.

    Returns:
    - Updated row with match data from sequence_position_dict or NCP_peptide_positions_dict.
    """
    sequence = row['Sequence']
    # Check for a match in the sequence_position_dict
    if sequence in sequence_position_dict:
        # Use the extract_gene_identifier function to simplify gene name extraction
        gene = extract_gene_identifier(sequence_position_dict[sequence]['gene'])
        # Update 'GNC&spp' with the extracted gene, append if already exists
        row['GNC&spp'] = f"{gene}" if pd.isnull(row['GNC&spp']) else f"{row['GNC&spp']},{gene}"

    if sequence in NCP_peptide_positions_dict:
        gene = extract_gene_identifier(NCP_peptide_positions_dict[sequence]['gene'])
        # Append or set the gene identifier for NCP matches
        row['GNC&spp'] = f"{gene}" if pd.isnull(row['GNC&spp']) else f"{row['GNC&spp']},{gene}"

    return row



###Find positions and create Sequence_postion dictionary Functions

In [13]:
def match_peptides(peptides, sequences):
    """
    Match peptides exactly against sequences in the provided dictionary.

    Parameters:
    - peptides (list): List of peptide sequences to match.
    - sequences (dict): Dictionary of gene sequences with details.
    """
    for peptide in peptides:
        for gene, details in sequences.items():
            sequence = details['sequence']
            offset = details['offset']
            position = sequence.find(peptide)
            if position != -1:  # Peptide found in the sequence
                corrected_position = position + offset  # Adjust for sequence offset
                sequence_position_dict[peptide] = {
                    'gene': gene,
                    'position': corrected_position,
                    'length': len(peptide)  # Add length of peptide
                }
                matched_peptides.add(peptide)
                break

def is_one_amino_acid_difference(peptide1, peptide2):
    """
    Check if two peptides of the same length differ by exactly one amino acid.
    """
    if len(peptide1) != len(peptide2):
        return False

    diff_count = sum(1 for a, b in zip(peptide1, peptide2) if a != b)
    return diff_count == 1

def is_within_mismatch_limit(peptide1, peptide2, max_mismatches):
    """
    Check if two peptides of the same length differ by a number of amino acids up to the max_mismatches limit.
    """
    if len(peptide1) != len(peptide2):
        return False

    diff_count = sum(1 for a, b in zip(peptide1, peptide2) if a != b)
    return diff_count <= max_mismatches

def match_unmatched_peptides(remaining_unmatched_peptides, sequence_position_dict):
    """
    Attempt to match remaining unmatched peptides with those in sequence_position_dict,
    allowing for one amino acid substitution.
    """
    newly_matched_peptides = set()

    for unmatched_peptide in remaining_unmatched_peptides:
        for matched_peptide in sequence_position_dict.keys():
            if is_one_amino_acid_difference(unmatched_peptide, matched_peptide):
                newly_matched_peptides.add(unmatched_peptide)
                break

    remaining_unmatched_peptides -= newly_matched_peptides
    return remaining_unmatched_peptides

def iterative_match_unmatched_peptides(unmatched_peptides, sequence_position_dict, sequences, iterations=1): #change the number of interations here)
    """
    Iteratively match remaining unmatched peptides with those in sequence_position_dict,
    allowing for an increasing number of amino acid substitutions up to the specified iterations.
    Each newly matched peptide is added to sequence_position_dict, broadening the search in subsequent iterations.
    """
    for i in range(1, iterations + 1):  # Allow mismatches up to 'iterations'
        newly_matched_peptides = {}

        for unmatched_peptide in unmatched_peptides:
            for matched_peptide, details in sequence_position_dict.items():
                if is_within_mismatch_limit(unmatched_peptide, matched_peptide, i):
                    # Record newly matched peptide with the details of the matched_peptide it compares to
                    newly_matched_peptides[unmatched_peptide] = details
                    break

        # Add newly matched peptides to sequence_position_dict and remove from unmatched_peptides
        for peptide, details in newly_matched_peptides.items():
            sequence_position_dict[peptide] = details
            unmatched_peptides.remove(peptide)

        # Optional: Report progress after each iteration
        print(f"Iteration {i}: Found {len(newly_matched_peptides)} new matches.")

    return unmatched_peptides

def annotate_row(row, positions_dict):
    """
    Annotate a single row with gene and start position based on a given positions dictionary.

    Parameters:
    - row: A row of a pandas DataFrame.
    - positions_dict: A dictionary with sequences as keys and dictionaries with 'gene' and 'position' as values.

    Returns:
    - row: The modified row with 'GNC&spp' and 'start_position' annotated if the sequence is found in positions_dict.
    """
    sequence = row['Sequence']
    if sequence in positions_dict:
        row['GNC&spp'] = positions_dict[sequence].get('gene')
        row['start_position'] = positions_dict[sequence].get('position')
    return row


###Identify PTMS and Parse peptide

In [14]:
def parse_modifications(mod_str):
    """
    Parses a string containing modification data into a structured list.

    This function extracts the position, type of modification, and the amino acid
    involved from a given modification string. The expected format of the modification
    string is "position,Modification[AminoAcid];".

    Parameters:
        mod_str (str): A string representing modifications, or NaN/None for no modifications.

    Returns:
        list of tuples: A list where each tuple contains (position, modification type, amino acid).
    """
    # Check if mod_str is not a string (e.g., NaN represented as float in pandas)
    if not isinstance(mod_str, str):
        return []

    parsed_mods = []
    mods = mod_str.strip(';').split(';')
    for mod in mods:
        parts = mod.split(',')
        if len(parts) == 2:
            pos, mod_detail = parts
            try:
                mod_type, aa = mod_detail.split('[')
                aa = aa.rstrip(']')
                parsed_mods.append((int(pos), mod_type, aa))
            except ValueError:
                # Skip modification if parsing fails
                continue

    return parsed_mods

def parse_peptide(peptide):
    """
    Parses peptide strings to identify amino acids and extract them along with their positions.

    :param peptide: A string representation of a peptide.
    :return: A list of tuples, each representing an amino acid and its position in the peptide.
    """
    # A simple pattern that matches each amino acid in the sequence.
    aa_pattern = re.compile(r'[A-Z]')
    parsed_peptides = []
    pos = 1  # Position initialization

    for match in aa_pattern.finditer(peptide):
        aa = match.group()  # Extract the amino acid
        parsed_peptides.append((aa, pos))
        pos += 1  # Increment position for each amino acid

    return parsed_peptides

###Dataframe manipulation

In [15]:
def parse_and_update_df(sequence, sequence_position_dict): #combined_df
    """
    Check if the sequence matches an entry in the sequence_position_dict and return gene and position.

    Parameters:
    - sequence (str): The peptide sequence to check.
    - sequence_position_dict (dict): Dictionary with peptide sequences as keys and details as values.

    Returns:
    - tuple: (gene, position) if match is found; (None, None) otherwise.
    """
    if sequence in sequence_position_dict:
        # Extract gene and position from the matched entry
        gene = sequence_position_dict[sequence]['gene']
        position = sequence_position_dict[sequence]['start_position']
        return gene, position
    else:
        # Return None or placeholders if no match is found
        return None, None

def update_NCP_matches(row, NCP_peptide_positions_dict):
    """
    Update the row with NCP peptide positions if the sequence matches an NCP peptide.

    Parameters:
    - row: A pandas Series representing a row in a DataFrame.
    - NCP_peptide_positions_dict: A dictionary containing NCP peptide positions.

    Returns:
    - Updated row with NCP peptide match data.
    """
    sequence = row['Sequence']
    if sequence in NCP_peptide_positions_dict:
        match_data = NCP_peptide_positions_dict[sequence]
        # Check and process the gene name specifically if it exists
        if 'gene' in match_data:
            gene = match_data['gene'].split('-', 1)[0]  # Extract gene name before the underscore
            row['GNC&spp'] = gene  # Assuming you're updating to a gene-specific column
        else:
            for key, value in match_data.items():
                row[key] = value
    return row


def extract_protein_frequencies_and_associations(df, column_name):
    """
    Extracts the frequency of each protein ID and the five most common proteins
    listed with each ID from a specified column in a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing protein data.
    column_name (str): The name of the column with protein IDs separated by '/'.

    Returns:
    pd.DataFrame: A DataFrame with each protein's frequency and its five most common associations.
    """
    # Parse the specified column to create a list of proteins per row
    #df['Proteins_List'] = df[column_name].apply(lambda x: x.split('/'))
    df['Proteins_List'] = df[column_name].apply(lambda x: [p.split('&') for p in x.split('/')])

    # Initialize a dictionary to keep count of each protein's frequency
    protein_counts = Counter()
    # Initialize a dictionary to keep track of proteins listed together
    protein_associations = defaultdict(lambda: Counter())

    # Count frequencies and associations
    for proteins in df['Proteins_List']:
        for protein in proteins:
            protein_counts[protein] += 1
            # Add counts for other proteins in the list to the associations
            for associated_protein in proteins:
                if associated_protein != protein:
                    protein_associations[protein][associated_protein] += 1

    # Prepare the final report data
    report_data = [{
        'Protein': protein,
        'Frequency': count,
        'Most_Common_Associations': [assoc for assoc, _ in protein_associations[protein].most_common(5)]
    } for protein, count in protein_counts.items()]

    # Convert report_data to a DataFrame
    report_df = pd.DataFrame(report_data)

    return report_df

def expand_df_to(df):
    """
    Expands a DataFrame by processing each peptide sequence and constructing detailed rows
    for each amino acid within the peptide sequences, including modification information and
    simplified sample name.

    Parameters:
        df (pd.DataFrame): A pandas DataFrame containing peptide information.

    Returns:
        pd.DataFrame: An expanded DataFrame with detailed amino acid and modification information.
    """
    expanded_rows = []
    # for _, row in df.iterrows():
    #     try:
    #         start_position = float(row['start_position'])
    #         if not start_position.is_integer():
    #             continue  # Skip if start_position is not a whole number
    #         start_position = int(start_position)
    #     except (ValueError, TypeError):
    #         continue  # Skip if start_position is not convertible to float
    for _, row in df.iterrows():
        try:
            # Directly convert start_position to an integer
            start_position = int(row['start_position'])
        except ValueError:
            # Skip this row if start_position is not a valid integer
            continue

        modifications = parse_modifications(row.get('Modification', ''))
        mod_dict = {mod[0]: {'type': mod[1], 'aa': mod[2]} for mod in modifications}
        amino_acids = parse_peptide(row['Sequence'])

        gnc_spp_split = row.get('GNC&spp', '').split('-', maxsplit=1)
        if len(gnc_spp_split) != 2:
            continue  # Skip if the split does not result in two parts

        GNC, spp = gnc_spp_split
        simplified_file_name = "_".join(row.get('File_Name', '').split("_")[:2])

        for aa, pos in amino_acids:
            mod_info = mod_dict.get(pos, {'type': '', 'aa': ''})
            expanded_row = {
                'id': None,
                'sample_name': simplified_file_name,
                'GNC': GNC,  #Genome Nomenclature
                'spp': spp,  #Species
                'aa': aa,    #amino acid
                'position': pos - 1 + start_position, #-1 as python counts from 0
                'ptm': mod_info['type'], # infomation on any post translational modification
                'confidence': None,  #confidence score for the amino acid
                'spectraId': row['Scan_No'], #the ID of the MS/MS scan number
                'mz': row['Exp.MH+'],  #mass of the peptide detected (m/z, mass over charge)
                'z': row['Charge'],   #charge
                'pepMass': row['Calc.MH+'],  #mass
                'err': row['Mass_Shift(Exp.-Calc.)'],  #error between the observed and calculated mass
                'score': row['Final_Score'], # score for the peptide - note this is difference from the 'confidence' which is for the individual amion acid
                'scanNum': row['Scan_No'], #scan number
                'peptide_gene_name': row['Proteins'], #full information about the ID from mapping algorithm (e.g. MaxQuant, pFind, Orthrus)
                'RT': None, #Retention time
                'ppm': None, #error in ppm
            }
            expanded_rows.append(expanded_row)

    return pd.DataFrame(expanded_rows)

### Export functions

In [16]:
def generate_sequence_with_gaps(group):
    """
    Generate a sequence with gaps for missing positions from a sorted group of sequences.

    Parameters:
    - group: A pandas DataFrame containing 'position' and 'aa' (amino acid) columns.

    Returns:
    - A string representing the sequence with gaps for missing positions.
    """
    group_sorted = group.sort_values('position')
    full_range = np.arange(group_sorted['position'].min(), group_sorted['position'].max() + 1)
    sequence_parts = []

    for position in full_range:
        if position not in group_sorted['position'].values:
            sequence_parts.append("-")
        else:
            aa_list = group_sorted.loc[group_sorted['position'] == position, 'aa'].unique()
            sequence_parts.append(f"({''.join(aa_list)})" if len(aa_list) > 1 else ''.join(aa_list))

    return ''.join(sequence_parts)

def format_fasta_header(row):
    """
    Format a FASTA header string based on the sample name and GNC from a DataFrame row.

    Parameters:
    - row: A pandas Series representing a row in a DataFrame.

    Returns:
    - A string representing the formatted FASTA header.
    """
    return f">{row['sample_name']}_{row['GNC'].replace(' ', '_')}"



###Plotting Functions

### Protein aggregation functions

In [17]:
def process_group(group):
    """
    Process a pandas DataFrame group to extract specific analytics and return results as a pd.Series.

    This function computes various metrics including the unique GNC, position, and
    sample_name values, counts for each amino acid, and detailed statistics for the
    top 3 amino acids. It includes mean, max, and standard deviation of scores,
    top scores and spectraIds, and PTM analysis. The result is returned as a pd.Series
    to facilitate easy integration with pandas data structures.

    Parameters:
    group (pandas.DataFrame): The DataFrame group to process.

    Returns:
    pandas.Series: A Series containing the computed metrics and statistics.
    """

    gnc, position, sample_name = group['GNC'].iloc[0], group['position'].iloc[0], group['sample_name'].iloc[0]

    aa_counts = group['aa'].value_counts()
    top_3_aas = aa_counts.index[:3]

    result = {'GNC': gnc, 'position': position, 'sample_name': sample_name}

    for i, aa in enumerate(top_3_aas, start=1):
        aa_group = group[group['aa'] == aa]
        scores = aa_group['score']
        top_scores = aa_group.nlargest(3, 'score')[['score', 'spectraId']]

        ptm_counts = aa_group['ptm'].value_counts()
        top_ptms = ptm_counts.index[:2]

        aa_info = {
            f'aa{i}': aa,
            f'n_aa{i}': aa_counts[aa],
            f'aa{i}_score_mean': scores.mean(),
            f'aa{i}_score_max': scores.max(),
            f'aa{i}_score_std': scores.std(),
        }

        aa_info.update({
            **{f'score{j}_aa{i}': score for j, (score, _) in enumerate(top_scores['score'], start=1)},
            **{f'spectraId{j}_aa{i}': spectra_id for j, (_, spectra_id) in enumerate(top_scores['spectraId'], start=1)}
        })

        for j, ptm in enumerate(top_ptms, start=1):
            ptm_group = aa_group[aa_group['ptm'] == ptm]
            scores = ptm_group['score']
            top_ptm_score = ptm_group.nlargest(1, 'score')[['score', 'spectraId']].iloc[0]

            aa_info.update({
                f'ptm{j}_aa{i}': ptm,
                f'%ptm{j}_aa{i}': (ptm_group.shape[0] / aa_group.shape[0]) * 100,
                f'ptm{j}_aa{i}_score_mean': scores.mean(),
                f'ptm{j}_aa{i}_score_max': scores.max(),
                f'ptm{j}_aa{i}_score_std': scores.std(),
                f'score1_ptm{j}_aa{i}': top_ptm_score['score'],
                f'spectraId1_ptm{j}_aa{i}': top_ptm_score['spectraId']
            })

        result.update(aa_info)

    return pd.Series(result)

In [18]:
def get_top_n(df, column, n=3):
    """
    Returns the top N items from a dataframe based on the count of a specified column.
    """
    return df[column].value_counts().nlargest(n).reset_index().rename(columns={'index': column, column: f'n_{column}'})

def get_scores_info(df, aa, prefix):
    """
    Aggregate score information for a given amino acid.
    """
    filtered_df = df[df['aa'] == aa]
    top_scores = filtered_df.nlargest(3, 'score')[['score', 'spectraId']]
    score_info = {
        f'{prefix}_score_mean': filtered_df['score'].mean(),
        f'{prefix}_score_max': filtered_df['score'].max(),
        f'{prefix}_score_SD': filtered_df['score'].std(),
    }
    for i, (score, spectraId) in enumerate(zip(top_scores['score'], top_scores['spectraId']), start=1):
        score_info.update({
            f'{prefix}_score{i}': score,
            f'{prefix}_spectraId{i}': spectraId,
        })
    return score_info

def get_ptm_info(df, aa, prefix):
    """
    Aggregate PTM information for a given amino acid.
    """
    # Filtering for non-null PTMs
    filtered_df = df[(df['aa'] == aa) & (df['ptm'].notnull())]
    if filtered_df.empty:
        return {f'{prefix}_ptm': None}  # Return None or appropriate value if no PTMs are found

    # Assuming PTM data is categorical and can be directly counted
    top_ptms = filtered_df['ptm'].value_counts().nlargest(2).index
    ptm_info = {}
    for i, ptm in enumerate(top_ptms, start=1):
        ptm_group = filtered_df[filtered_df['ptm'] == ptm]
        ptm_info.update({
            f'{prefix}_ptm{i}': ptm,
            f'{prefix}_ptm{i}_count': ptm_group.shape[0],
            f'{prefix}_ptm{i}_percentage': (ptm_group.shape[0] / filtered_df.shape[0]) * 100,
            # Additional PTM-specific stats can be added here
        })

    return ptm_info

def aggregate_info(group):
    """
    Aggregates detailed information for each group of data.
    """
    top_aas = get_top_n(group, 'aa')
    aggregated_data = []

    for _, row in top_aas.iterrows():
        aa = row['aa']
        aa_info = get_scores_info(group, aa, 'aa1')
        ptm_info = get_ptm_info(group, aa, 'aa1')

        record = {
            'aa': aa,
            'n_aa': row['n_aa'],
            **aa_info,
            **ptm_info
        }

        aggregated_data.append(record)

    return pd.DataFrame(aggregated_data)

# Load files

##Data Path

### Muli-directories

In [31]:
# # Find pFind-Filtered.spectra files
# spectra_files = find_spectra_files(BASE_PATH)
# # Combine into a single DataFrame
# combined_df = combine_spectra_files(spectra_files)
# #Add the length of the peptide
# combined_df['peptide_length'] = combined_df['Sequence'].apply(len)
# #----------------------Reporting
# #combined_df.shape
# #combined_df.info()
# ##combined_df.describe()
# #combined_df['Sequence'].nunique()
# #combined_df['Miss.Clv.Sites'].nunique()
# combined_df.head()

In [32]:
# Set the base path as per your Google Drive structure
STUDY_NAME1 = 'CE&Tuuli'
STUDY_NAME2 = 'CATE'
STUDY_NAMES = [STUDY_NAME1, STUDY_NAME2]

PFIND_FOLDER1 = f'pFind_{STUDY_NAME1}'  # Folder path for study 1
PFIND_FOLDER2 = f'pFind_{STUDY_NAME2}'  # Folder path for study 2

BASE_PATH1 = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{STUDY_NAME1}/{PFIND_FOLDER1}/'
BASE_PATH2 = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{STUDY_NAME2}/{PFIND_FOLDER2}/'

def find_base_paths(study_names):
  base_paths = []
  for study_name in study_names:
    PFIND_FOLDER = f'pFind_{study_name}'
    base_path = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{study_name}/{PFIND_FOLDER}/'
    base_paths.append(base_path)
  return base_paths

# Find base paths for both studies
base_paths = find_base_paths(STUDY_NAMES)

# Load dataframes from each study (assuming 'combine_spectra_files' works as intended)
df_study1 = combine_spectra_files([base_paths[0]])  # Pass only the base path for study 1
df_study2 = combine_spectra_files([base_paths[1]])  # Pass only the base path for study 2
combined_df = pd.concat([df_study1, df_study2], ignore_index=True)


# Check the shape of each DataFrame
print(f"Shape of df_study1: {df_study1.shape}")
print(f"Shape of df_study2: {df_study2.shape}")

# Reporting (assuming combined_df is not None)
if combined_df is not None:
  print(combined_df.shape)
  print(combined_df.info())
  print(f"Number of unique sequences: {combined_df['Sequence'].nunique()}")
#----------------------Reporting
#combined_df.shape
#combined_df.info()
##combined_df.describe()
combined_df['Sequence'].nunique()
#combined_df['Miss.Clv.Sites'].nunique()
#combined_df.head()

Shape of df_study1: (48116, 19)
Shape of df_study2: (30429, 19)
(78545, 19)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78545 entries, 0 to 78544
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   File_Name               78545 non-null  object 
 1   Scan_No                 78545 non-null  int64  
 2   Exp.MH+                 78545 non-null  float64
 3   Charge                  78545 non-null  int64  
 4   Q-value                 78545 non-null  float64
 5   Sequence                78545 non-null  object 
 6   Calc.MH+                78545 non-null  float64
 7   Mass_Shift(Exp.-Calc.)  78545 non-null  float64
 8   Raw_Score               78545 non-null  float64
 9   Final_Score             78545 non-null  float64
 10  Modification            56710 non-null  object 
 11  Specificity             78545 non-null  int64  
 12  Proteins                78545 non-null  object 
 13  Positions      

7569

In [33]:
# # Find base paths for both studies
# base_paths = find_base_paths(STUDY_NAMES)

# # Load dataframes from each study (assuming 'combine_spectra_files' works as intended)
# df_study1 = combine_spectra_files([base_paths[0]])  # Pass only the base path for study 1
# df_study2 = combine_spectra_files([base_paths[1]])  # Pass only the base path for study 2
# combined_df = pd.concat([df_study1, df_study2], ignore_index=True)


# # Check the shape of each DataFrame
# print(f"Shape of df_study1: {df_study1.shape}")
# print(f"Shape of df_study2: {df_study2.shape}")

# # Reporting (assuming combined_df is not None)
# if combined_df is not None:
#   print(combined_df.shape)
#   print(combined_df.info())
#   print(f"Number of unique sequences: {combined_df['Sequence'].nunique()}")
# #----------------------Reporting
# #combined_df.shape
# #combined_df.info()
# ##combined_df.describe()
# combined_df['Sequence'].nunique()
# #combined_df['Miss.Clv.Sites'].nunique()
# #combined_df.head()

### Single directory

In [34]:
# STUDY_NAME = 'CE&Tuuli'
# PFIND_FOLDER = f'pFind_{STUDY_NAME}'
# #Find pFind-Filtered.spectra files
# #spectra_files = find_spectra_files(BASE_PATH)

# spectra_files = [f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{STUDY_NAME}/{PFIND_FOLDER}/result/pFind-Filtered.spectra']
# # Combine into a single DataFrame
# combined_df = combine_spectra_files_from_single_directory(spectra_files)
# #Add the length of the peptide

# combined_df['peptide_length'] = combined_df['Sequence'].apply(len)
# #----------------------Reporting
# #combined_df.shape
# #combined_df.info()
# ##combined_df.describe()
# combined_df['Sequence'].nunique()
# #combined_df['Miss.Clv.Sites'].nunique()
# #combined_df.head()

###Find Collagen motif
Iterates through each peptide in unique_peptides.
Uses a generator expression within all() to check if every third character, starting from the offset determined by len(peptide) % 3, is a 'G'.
This method directly reflects the requirement for 'G' to be every third character after an initial offset and accounts for sequences that may not start with 'G' or might be truncated.

# Dictionary Matching

In [54]:
# Initialize dictionary to store matched peptide details with sequence length
sequence_position_dict = {}

# Initialize set for matched peptides
matched_peptides = set()

# Extract unique peptides from DataFrame
unique_peptides = combined_df['Sequence'].unique().tolist()

# First round of matching
match_peptides(unique_peptides, sequences_dict)

# Calculate unmatched peptides after the first round
remaining_unmatched_unique_found_peptides = set(unique_peptides) - matched_peptides

# Iterative second round of matching, including sequences_dict for gene and position info
remaining_unmatched_after_iterations = iterative_match_unmatched_peptides(remaining_unmatched_unique_found_peptides, sequence_position_dict, sequences_dict, iterations=1)

# Reporting the outcome
total_matched_after_iterations = len(sequence_position_dict)  # Updated total matches include initial and iterative matches
total_unmatched_after_iterations = len(remaining_unmatched_after_iterations)


print(f"Total matched peptides after iterations: {total_matched_after_iterations}")
print(f"Total unmatched peptides after iterations: {total_unmatched_after_iterations}")


Iteration 1: Found 804 new matches.
Total matched peptides after iterations: 2911
Total unmatched peptides after iterations: 4658


###Add Gene IDs and start postion to peptides

In [55]:
# Initialize new columns to avoid KeyError
combined_df['GNC&spp'] = None
combined_df['start_position'] = None

# Annotate Rols
combined_df = combined_df.apply(annotate_row, positions_dict=sequence_position_dict, axis=1)

# Check for non-null entries if nulls are np.nan (standard missing data marker in pandas)
non_null_entries = combined_df['GNC&spp'].notnull().sum()

# If empty strings are used to represent missing data, you might want to adjust the method to something like:
# non_empty_entries = combined_df[combined_df['GNC&spp'] != ''].shape[0]

print(f"Number of non-null/empty entries in 'GNC&spp': {non_null_entries}")

# Most common entries in 'GNC&spp'
most_common_entries = combined_df['GNC&spp'].value_counts().head(15)  # Adjust the number inside head() as needed
print("Most common entries in 'GNC&spp':")
print(most_common_entries)

# Calculating the percentage of rows with an entry in 'GNC&spp'
total_rows = combined_df.shape[0]
percentage_with_entry = (non_null_entries / total_rows) * 100

print(f"Percentage of rows with an entry in 'GNC&spp': {percentage_with_entry:.2f}%")

Number of non-null/empty entries in 'GNC&spp': 62006
Most common entries in 'GNC&spp':
COL1A1-Bos_tau    16629
COL1A2-Bos_mut     7888
TRYUP-Sus_sco      4435
COL1A2-Lox_afr     3796
COL3A1-Bos_tau     3357
COL1A1-Hip_amp     2682
K2C1-Hom_sap       2220
COL1A2-Equ_cab     2154
KS10-Hom_sap       1875
PIV-Pse_aer        1865
COL3A1-Equ_cab     1763
COL1A2-Hip_amp     1438
KS9-Hom_sap        1418
COL1A2-Equ_asi     1374
COLunk-Bos_tau     1072
Name: GNC&spp, dtype: int64
Percentage of rows with an entry in 'GNC&spp': 78.94%


In [56]:
# Filter the DataFrame for rows where 'GNC&spp' or 'start_position' is missing
missing_annotations_df = combined_df[combined_df['GNC&spp'].isna() | combined_df['start_position'].isna()]

# Print the top 100 entries for the 'Proteins' column from these rows
print(missing_annotations_df['Proteins'].head(400))

# Extract all protein IDs, split by '/', from the 'Proteins' column
protein_list = missing_annotations_df['Proteins'].str.split('/').explode()

# Find the 100 most common protein strings
top_400_proteins = protein_list.value_counts().head(400)


top_400_proteins.to_csv('top_400_proteins.txt', header=False, index=True)


# Get protein names and their counts
protein_counts = protein_list.value_counts()

# Ensure at least one protein per count (using nlargest)
one_protein_per_count = protein_counts.nlargest(len(protein_counts.unique()))

# Combine protein name and count into a DataFrame
result_df = pd.DataFrame({'protein_id': one_protein_per_count.index, 'count': one_protein_per_count.values})

# Print the results (protein ID and count)
print(result_df)


429     C0HLI7.1/XP_027728615.1/XP_020853290.1/XP_0103...
463                                       XP_041562691.1/
498     KFW82933.1/XP_016402454.1/XP_016334116.1/TEU55...
504     KFW82933.1/XP_016402454.1/XP_016334116.1/TEU55...
519     KAF1529257.1/WP_168397204.1/3B63_L/5ADX_H/KAG8...
599                                       XP_026294502.1/
637     KAF6085594.1/KAF6085593.1/XP_006178597.1/XP_01...
667     KFW82933.1/XP_016402454.1/XP_016334116.1/TEU55...
735     C0HLI7.1/XP_027728615.1/XP_020853290.1/XP_0103...
766                          KAE8693106.1/XP_039012742.1/
779     MCP6497603.1/MCP6564868.1/ABA42866.1/3V03_A/5Y...
798     KTF78707.1/KAG1967375.1/XP_051571265.1/XP_0425...
826     KTF78707.1/KAG1967375.1/XP_051571265.1/XP_0425...
878     XP_038174782.1/XP_038174780.1/XP_038174781.1/X...
880     XP_038174782.1/XP_038174780.1/XP_038174781.1/X...
899     KFW82933.1/XP_016402454.1/XP_016334116.1/TEU55...
905     MCP6497603.1/MCP6564868.1/ABA42866.1/3V03_A/5Y...
906           

In [58]:
# Get one protein ID for each unique count
unique_counts = top_400_proteins.sort_values(ascending=False)
# Get the first occurrence for each unique count using idxmin
one_protein_per_count = top_400_proteins.groupby(top_400_proteins.values).agg(lambda x: x.iloc[0])

# Print the results (protein ID and count)
top_400_proteins

# Print the results (protein ID and count)
#print(one_protein_per_count(50))

                  16539
NWV22044.1          422
XP_005570993.2      416
XP_024113020.1      415
XP_014748466.1      415
XP_030678472.1      415
XP_032029551.1      415
XP_011857542.1      414
XP_028685340.1      414
AAG41947.1          414
XP_011727181.1      414
NP_006112.3         414
AFA52007.1          414
AAB47721.1          414
AFA52006.1          414
XP_003845964.2      414
XP_003906478.2      414
AFA52002.1          414
AFA52005.1          414
XP_025258843.1      414
XP_011857532.1      414
XP_001098182.2      414
XP_025258844.1      414
XP_050604771.1      414
XP_011857566.1      414
XP_011857549.1      414
XP_011903561.1      414
XP_025258845.1      414
XP_011857559.1      414
AKI71766.1          414
XP_004053220.3      414
PNI33041.1          414
NP_001104288.1      414
AFA52004.1          414
AFA52003.1          414
XP_037853004.1      413
XP_004429138.2      406
XP_011854305.1      404
XP_051624887.1      402
XP_050193369.1      398
NWZ27654.1          397
NXP47142.1      

In [ ]:
# Print the results (protein ID and count)
print(one_protein_per_count)

In [49]:
top_400_proteins

                  27100
XP_045876752.1     1172
XP_002918275.1     1124
XP_022358799.1     1115
XP_049497431.1     1115
XP_007076757.1     1115
XP_045350756.1     1115
XP_032730864.1     1114
XP_045350757.1     1112
XP_032161267.1     1111
XP_004767919.1     1111
XP_044101981.1     1111
XP_047551750.1     1111
XP_039092171.1     1053
KAF0880087.1       1046
XP_022441863.1      912
XP_029089490.1      908
XP_028700691.1      880
XP_005550260.1      880
NP_001253266.1      880
XP_004632680.1      814
XP_048195999.1      807
AKI72178.1          797
AKI72177.1          797
NP_000414.2         797
AKI72621.1          795
AKI72180.1          793
KAI4066109.1        793
AAB81946.1          793
AFA52006.1          780
PNI33042.1          779
XP_030856700.1      779
XP_034790894.1      776
XP_034790893.1      776
XP_008954692.2      776
XP_003475044.2      774
XP_005006094.1      774
XP_005570993.2      767
XP_032029551.1      766
XP_030678472.1      766
XP_011903561.1      765
XP_011857549.1  

###   Direct Matching for Collagen-Motif Peptides

In [ ]:
peptides_to_examine = [
    'IITHPNFNGNTLDNDIMLIK',
    'LGEHNIDVLEGNEQFINAAK',
    'YSQGNVSAVGVTYDGHTALTR',
    'TPPAGVFYQGWSATPIANGSLGHDIHHPR',
    'GPPGPMGPPGIAGPPGESGR',
    'SDLEMQYETLQEELMALKK'
]


print("Peptides from sequence_position_dict:")
for peptide in peptides_to_examine:
    if peptide in sequence_position_dict:
        details = sequence_position_dict[peptide]
        # Check if 'gene' and 'position' keys exist before accessing
        gene = details.get('gene', 'Gene information not available')
        position = details.get('position', 'Position information not available')
        print(f"Peptide: {peptide}, Gene: {gene}, Position: {position}")
    else:
        print(f"Peptide: {peptide} not found in sequence_position_dict.")


In [59]:
# Define the directory for saving the output file
output_directory = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/sample1/'
os.makedirs(output_directory, exist_ok=True)

# Get the current timestamp
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with study names and timestamp
filename = f"{output_directory}{'_'.join(STUDY_NAMES)}_{now}_sequence_positions.json"

# Assuming 'combined_df' is to be saved, convert it to a dictionary
sequence_position_dict = combined_df.to_dict()
with open(filename, 'w') as json_file:
    json.dump(sequence_position_dict, json_file, indent=4)

print(f'Dictionary saved to JSON file: {filename}')

Dictionary saved to JSON file: /content/drive/MyDrive/Colab_Notebooks/NovorCloud/sample1/CE&Tuuli_CATE_2024-03-17_13-24-29_sequence_positions.json


In [ ]:
# #Peek at the output of the Dictionaries =============================== report
# random_entries = random.sample(list(sequence_position_dict.items()), 100)
# for key, value in random_entries:
#     print(f"Key: {key}, Value: {value}")

# print(sequence_position_dict)
# #  matched_peptides in sequence_position_dict  dictionary
# for peptide, details in islice(sequence_position_dict.items(), 10):
#     print(f"Peptide: {peptide}, Gene: {details['gene']}, Position: {details['position']}")


### Step 4: Identify Unmatched Peptides

After attempting direct matching, identify peptides that remain unmatched:

In [ ]:
combined_df

In [ ]:
combined_df

In [ ]:
# #Reporting
# combined_df.head()
# #Check the df again!
# combined_df.head()
# combined_df.shape
# combined_df.info()
# #combined_df.describe()
# combined_df['Sequence'].nunique()

In [ ]:
# #To view the matches
# print("Collagen Matches:")
# for peptide, details in sequence_position_dict.items():
#     print(f"Peptide: {peptide}, Gene: {details['gene']}, Position: {details['position']}")
# print("\nNCP Matches:")
# for peptide, details in NCP_peptide_positions_dict.items():
#     print(f"Peptide: {peptide}, Gene: {details['gene']}, Position: {details['position']}")

In [ ]:
# # Report on the most frequent proteins - this is very SLOW!!!
# report_df = extract_protein_frequencies_and_associations(combined_df, 'Proteins')

# # Display or export `report_df` as needed
# print(report_df)

In [ ]:
# #Writing this report.csv
# # Construct the file path first
# csv_file_path = f'{BASE_PATH}/{STUDY_NAME}_prot_freq.csv'

# # Export the DataFrame to CSV using the correct method name
# report_df.to_csv(csv_file_path)

In [ ]:
# ##Check the df again!
# #combined_df.head()
# #combined_df.shape
# #combined_df.info()
# ##combined_df.describe()
# #combined_df['Sequence'].nunique()
# #combined_df['Miss.Clv.Sites'].nunique()
# combined_df.head()
# list(combined_df.columns)
# #print(combined_df[['GNC&spp', 'Sequence','Proteins', 'Positions', 'start_position']].head(40))
# spp_GNC_counts = combined_df['GNC&spp'].value_counts()

# # Print the counts of different entries
# print(spp_GNC_counts)

In [ ]:
#Generate smaller df - iltered_combined_df- for the Expand step
# Replace 'None' and empty strings with np.nan for uniformity
combined_df['GNC&spp'] = combined_df['GNC&spp'].replace({None: np.nan, '': np.nan})

# Filter rows where 'GNC&spp' is not NaN
filtered_combined_df = combined_df.dropna(subset=['GNC&spp'])

# Now, filtered_combined_df contains only the rows where 'GNC&spp' is not None, NaN, or blank
# You can perform your operations on filtered_combined_df

# Print the head of the filtered DataFrame to verify the filtering
print(filtered_combined_df.head())


In [60]:
#COUNT GNC
unique_names_GNC = filtered_combined_df['GNC&spp'].unique()
name_counts_GNC = filtered_combined_df['GNC&spp'].value_counts()
print("Unique GNC:", unique_names_GNC)
print("Counts of each GNC:", name_counts_GNC)

NameError: name 'filtered_combined_df' is not defined

## Expand df

In [ ]:
# # Assuming 'combined_df' is original dataframe with columns [''GNC&spp'', 'start_position', 'peptide_length', 'sequence']
# df_sorted = combined_df.sort_values(by=['GNC&spp','start_position', 'peptide_length','Sequence'], ascending=[True, False, False,True])

In [61]:
# Expand combined_df to detail each peptide's amino acid positions and associated PTMs
#exp_df = expand_df_to(filtered_combined_df) only contains collagen
exp_df = expand_df_to(combined_df)

In [70]:
# Display the first few rows of the expanded DataFrame to verify the result
print(exp_df.head())
list(exp_df.columns)
exp_df.shape

     id sample_name    GNC      spp aa  position ptm confidence  spectraId  \
0  None        Cz16  TRYUP  Sus_sco  I        69           None       6381   
1  None        Cz16  TRYUP  Sus_sco  I        70           None       6381   
2  None        Cz16  TRYUP  Sus_sco  T        71           None       6381   
3  None        Cz16  TRYUP  Sus_sco  H        72           None       6381   
4  None        Cz16  TRYUP  Sus_sco  P        73           None       6381   

            mz  z      pepMass     err         score  scanNum  \
0  2300.162702  2  2300.159002  0.0037  1.528900e-10     6381   
1  2300.162702  2  2300.159002  0.0037  1.528900e-10     6381   
2  2300.162702  2  2300.159002  0.0037  1.528900e-10     6381   
3  2300.162702  2  2300.159002  0.0037  1.528900e-10     6381   
4  2300.162702  2  2300.159002  0.0037  1.528900e-10     6381   

                                   peptide_gene_name    RT   ppm  
0  5XW1_A/P00761.1/1AKS_A/1AVW_A/1FMG_A/1AN1_E/3M...  None  None  
1  5XW

(1075011, 18)

In [ ]:
unique_names_GNC = exp_df['GNC'].unique()
name_counts_GNC = exp_df['GNC'].value_counts()
print("Unique sample names:", unique_names_GNC)
print("Counts of each sample name:", name_counts_GNC)

In [63]:
# Rename entries in 'sample_name' column using the rename dictionary
exp_df['sample_name'] = exp_df['sample_name'].replace(rename_dict)

In [62]:
#Rename PTMs based on conditions
exp_df['ptm'] = exp_df.apply(
    lambda x: 'Hydroxylation' if x['aa'] == 'P' and x['ptm'] == 'Oxidation' else
             'DeamidationN' if x['aa'] == 'N' and x['ptm'] == 'Deamidated' else
             'DeamidationQ' if x['aa'] == 'Q' and x['ptm'] == 'Deamidated' else
             x['ptm'],  # Leave other PTMs unchanged
    axis=1
)

In [64]:
unique_names = exp_df['sample_name'].unique()
name_counts = exp_df['sample_name'].value_counts()
print("Unique sample names:", unique_names)
print("Counts of each sample name:", name_counts)

Unique sample names: ['Cz16' 'Cz12' 'M2' 'Cz13' 'Cz3' 'Cz4' 'Cz17' 'Cz9' 'Cz14' 'Cz19' 'Cz22'
 'P3' 'Cz1' 'Cz18' 'M1' 'Cz8' 'Cz21' 'Cz6' 'Cz15' 'Cz5' 'P4' 'M4' 'Cz7'
 'M3' 'Cz10' 'Cz2' 'P2' 'Cz11' 'P1' 'Cz20' '09_ICB' 'MACHINE_BSA' '16_OL'
 '12_ICib' '10_ICB' 'Cz17_CFrag' '06_ICB' '03_OBO' 'Cz11_CFrag' '13_ICib'
 '14_OL' 'Cz5b_CFrag' '02_OBi' '05_ICB' 'INJECTION_BLANK']
Counts of each sample name: 10_ICB             64262
Cz6                53366
Cz22               51508
Cz9                50398
06_ICB             49847
Cz17               49504
09_ICB             49199
16_OL              48987
Cz1                37640
Cz14               37350
Cz11_CFrag         37123
14_OL              36535
12_ICib            35896
Cz5b_CFrag         35818
Cz19               35307
13_ICib            33950
Cz11               29732
Cz18               28926
P3                 27535
Cz17_CFrag         26356
Cz5                21706
M3                 21558
P4                 20418
Cz3                16434

In [65]:
# Get the value counts of ptm entries
ptm_counts = exp_df['ptm'].value_counts()

# Print the ptm entries and their counts
print(ptm_counts.head(20))

# Calculate ptm_counts without aggregating sample_name
ptm_counts = exp_df.groupby(['sample_name','GNC', 'position', 'aa', 'ptm']).size().reset_index(name='ptm_count')

# Calculate total_counts without aggregating sample_name
total_counts = exp_df.groupby(['sample_name','GNC', 'position', 'aa']).size().reset_index(name='total_count')

# Merge ptm_counts with the original exp_df to preserve sample_name and score
df_merged = pd.merge(exp_df, ptm_counts, on=['sample_name','GNC', 'position', 'aa', 'ptm'], how='left')
df_merged = pd.merge(df_merged, total_counts, on=['sample_name','GNC', 'position', 'aa'], how='left')

# Calculate the percentage of aa in each position having a specific ptm
df_merged['ptm_percentage'] = (df_merged['ptm_count'] / df_merged['total_count']) * 100

# Filter to include only rows where ptm is not None or empty (i.e., ptm is present)
result_with_ptms = df_merged[df_merged['ptm'].notna() & (df_merged['ptm'] != '')]

# Display the filtered result with ptm details included
print(result_with_ptms[['GNC', 'sample_name', 'position', 'aa', 'ptm', 'ptm_percentage', 'sample_name', 'score']].head())


                     984914
Hydroxylation         70272
DeamidationN           5958
Oxidation              5947
DeamidationQ           5196
Carbamidomethyl         349
Dioxidation             229
Trioxidation            220
Methyl                  121
Methylphosphonate        90
Pro->pyro-Glu            89
Carboxymethyl            81
Ammonia-loss             79
Formyl                   79
Pro->Asn                 65
Dehydrated               59
Ethanedithiol            58
Carboxy                  54
Carbonyl                 51
Asn->Met                 39
Name: ptm, dtype: int64
      GNC sample_name  position aa           ptm  ptm_percentage sample_name  \
7   TRYUP        Cz16        76  N  DeamidationN       18.627451        Cz16   
16  TRYUP        Cz16        85  M     Oxidation       86.915888        Cz16   
36  TRYUP        Cz16        85  M     Oxidation       86.915888        Cz16   
47  TRYUP        Cz16        76  N  DeamidationN       18.627451        Cz16   
56  TRYUP       

In [ ]:
#COUNT GNC
unique_names_GNC = exp_df['GNC'].unique()
name_counts_GNC = exp_df['GNC'].value_counts()
print("Unique GNC:", unique_names_GNC)
print("Counts of each GNC:", name_counts_GNC)

###Explore PTMs

###Reshaping exp_df

In [71]:
def create_grouped_df(exp_df):
    """
    Create a grouped dataframe from the input dataframe `exp_df`.

    Args:
        exp_df (pandas.DataFrame): The input dataframe containing protein identification data.

    Returns:
        pandas.DataFrame: The grouped dataframe containing the requested analysis.
    """
    # Group the data by GNC, sample_name, and position
    grouped = exp_df.groupby(['GNC', 'sample_name', 'position'])

    # Create a list to store the grouped analysis
    grouped_analysis = []

    # Iterate over each group
    for group_keys, group_data in grouped:
        # Unpack the group keys
        gnc, sample_name, position = group_keys

        # Get the top 3 amino acids and their counts
        aa_counts = group_data['aa'].value_counts().head(3)
        top_aas = aa_counts.index.tolist()
        aa_counts = aa_counts.to_dict()

        # Initialize a dictionary to store the analysis for this group
        group_analysis = {
            'GNC': gnc,
            'sample_name': sample_name,
            'position': position
        }

        # Analyze the top 3 amino acids
        for i, aa in enumerate(top_aas, start=1):
            aa_key = f'aa{i}'
            group_analysis[aa_key] = aa  # Add the amino acid name
            n_aa_key = f'n_{aa_key}'
            score_mean_key = f'{aa_key}_score_mean'
            score_max_key = f'{aa_key}_score_max'
            score_sd_key = f'{aa_key}_score_SD'
            score_keys = [f'score{j}_{aa_key}' for j in range(1, 4)]
            spectra_keys = [f'spectraId{j}_{aa_key}' for j in range(1, 4)]

            # Count of the amino acid
            group_analysis[n_aa_key] = aa_counts[aa]

            # Mean, max, and standard deviation of the confidence scores
            aa_scores = group_data.loc[group_data['aa'] == aa, 'confidence']
            group_analysis[score_mean_key] = aa_scores.mean()
            group_analysis[score_max_key] = aa_scores.max()
            group_analysis[score_sd_key] = aa_scores.std()

            # Top 3 scores and spectra IDs
            aa_scores = group_data.loc[group_data['aa'] == aa, ['score', 'spectraId']].nlargest(3, 'score')
            for j, (score_key, spectra_key) in enumerate(zip(score_keys, spectra_keys)):
                if j < len(aa_scores):
                    group_analysis[score_key] = aa_scores.iloc[j]['score']
                    group_analysis[spectra_key] = aa_scores.iloc[j]['spectraId']
                else:
                    group_analysis[score_key] = None
                    group_analysis[spectra_key] = None

            # Analyze the PTMs for this amino acid
            aa_ptms = group_data.loc[group_data['aa'] == aa, 'ptm'].value_counts()
            top_ptms = aa_ptms.index.tolist()[:2]
            aa_ptms = aa_ptms.to_dict()

            for j, ptm in enumerate(top_ptms, start=1):
                ptm_key = f'ptm{j}_{aa_key}'
                ptm_perc_key = f'%{ptm_key}'
                ptm_score_mean_key = f'{ptm_key}_score_mean'
                ptm_score_max_key = f'{ptm_key}_score_max'
                ptm_score_sd_key = f'{ptm_key}_score_SD'
                ptm_score_keys = [f'score{k}_{ptm_key}' for k in range(1, 4)]
                ptm_spectra_keys = [f'spectraId{k}_{ptm_key}' for k in range(1, 4)]


                # PTM name and percentage
                group_analysis[ptm_key] = ptm
                group_analysis[ptm_perc_key] = aa_ptms.get(ptm, 0) / (group_data[(group_data['aa'] == aa) & (group_data['position'] == position)]['aa'].count()) * 100


                # Mean, max, and standard deviation of the confidence scores for the PTM
                ptm_scores = group_data.loc[(group_data['aa'] == aa) & (group_data['ptm'] == ptm), 'confidence']
                group_analysis[ptm_score_mean_key] = ptm_scores.mean()
                group_analysis[ptm_score_max_key] = ptm_scores.max()
                group_analysis[ptm_score_sd_key] = ptm_scores.std()

                # Top 3 scores and spectra IDs for the PTM
                ptm_scores = group_data.loc[(group_data['aa'] == aa) & (group_data['ptm'] == ptm), ['score', 'spectraId']].nlargest(3, 'score')
                for k, (score_key, spectra_key) in enumerate(zip(ptm_score_keys, ptm_spectra_keys)):
                    if k < len(ptm_scores):
                        group_analysis[score_key] = ptm_scores.iloc[k]['score']
                        group_analysis[spectra_key] = ptm_scores.iloc[k]['spectraId']
                    else:
                        group_analysis[score_key] = None
                        group_analysis[spectra_key] = None

        # Append the analysis for this group to the list
        grouped_analysis.append(group_analysis)

    # Create the grouped dataframe from the list of dictionaries
    grouped_df = pd.DataFrame(grouped_analysis)

    return grouped_df

In [72]:
#-----------------------the SLOW one-----------------
#
group_df = create_grouped_df(exp_df)

In [73]:
# with list constructor
col_list = list(group_df.columns)
# with tolist method
col_list = group_df.columns.tolist()
print(col_list)

['GNC', 'sample_name', 'position', 'aa1', 'n_aa1', 'aa1_score_mean', 'aa1_score_max', 'aa1_score_SD', 'score1_aa1', 'spectraId1_aa1', 'score2_aa1', 'spectraId2_aa1', 'score3_aa1', 'spectraId3_aa1', 'ptm1_aa1', '%ptm1_aa1', 'ptm1_aa1_score_mean', 'ptm1_aa1_score_max', 'ptm1_aa1_score_SD', 'score1_ptm1_aa1', 'spectraId1_ptm1_aa1', 'score2_ptm1_aa1', 'spectraId2_ptm1_aa1', 'score3_ptm1_aa1', 'spectraId3_ptm1_aa1', 'ptm2_aa1', '%ptm2_aa1', 'ptm2_aa1_score_mean', 'ptm2_aa1_score_max', 'ptm2_aa1_score_SD', 'score1_ptm2_aa1', 'spectraId1_ptm2_aa1', 'score2_ptm2_aa1', 'spectraId2_ptm2_aa1', 'score3_ptm2_aa1', 'spectraId3_ptm2_aa1', 'aa2', 'n_aa2', 'aa2_score_mean', 'aa2_score_max', 'aa2_score_SD', 'score1_aa2', 'spectraId1_aa2', 'score2_aa2', 'spectraId2_aa2', 'score3_aa2', 'spectraId3_aa2', 'ptm1_aa2', '%ptm1_aa2', 'ptm1_aa2_score_mean', 'ptm1_aa2_score_max', 'ptm1_aa2_score_SD', 'score1_ptm1_aa2', 'spectraId1_ptm1_aa2', 'score2_ptm1_aa2', 'spectraId2_ptm1_aa2', 'score3_ptm1_aa2', 'spectraId3

In [ ]:
group_df

In [74]:
print(group_df)

         GNC sample_name  position aa1  n_aa1  aa1_score_mean  aa1_score_max  \
0        ALB      06_ICB        65   L      7             NaN            NaN   
1        ALB      06_ICB        66   V      7             NaN            NaN   
2        ALB      06_ICB        67   N      7             NaN            NaN   
3        ALB      06_ICB        68   E      7             NaN            NaN   
4        ALB      06_ICB        69   L      7             NaN            NaN   
...      ...         ...       ...  ..    ...             ...            ...   
80535  TRYUP          P4       102   V     22             NaN            NaN   
80536  TRYUP          P4       103   S     22             NaN            NaN   
80537  TRYUP          P4       104   I     21             NaN            NaN   
80538  TRYUP          P4       105   P     22             NaN            NaN   
80539  TRYUP          P4       106   R     22             NaN            NaN   

       aa1_score_SD  score1_aa1  spectr

In [68]:
PFIND_FOLDER1 = f'pFind_{STUDY_NAME1}'  # Folder path for study 1
directory_path = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{STUDY_NAME1}/{PFIND_FOLDER1}/result/'

# Define the directory for saving the output file
#directory_path = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/sample1/'
os.makedirs(directory_path , exist_ok=True)
group_df.to_csv(f'{directory_path}pFind_grouped2.csv', index=False)

In [ ]:
#Writing exp.csv
# Construct the file path first
csv_file_path = f'{BASE_PATH}/{STUDY_NAME}exp.csv'

# Export the DataFrame to CSV using the correct method name
exp_df.to_csv(csv_file_path)

In [ ]:
#grouped_CGPT_df = analyze_protein_data(test_df)

In [ ]:
#print(grouped_CGPT_df)

In [69]:
#----------------------------------Summary Table of Data

# Calculate counts for 'GNC&spp' across the entire DataFrame
counts = group_df['GNC'].value_counts()

# Filter 'GNC&spp' entries that meet the threshold criterion (e.g., count >= 100)
filtered_entries = counts[counts >= 100].index.tolist()

# Filter the DataFrame to only include those 'GNC&spp' entries
filtered_df = group_df[group_df['GNC'].isin(filtered_entries)]

# Prepare data for PrettyTable
# Initialize PrettyTable with 'File_Name' plus each 'GNC&spp' that meets the threshold as columns
columns = ['sample_name'] + filtered_entries
pt = PrettyTable()
pt.field_names = columns

# Populate PrettyTable rows with counts of 'GNC&spp' for each 'File_Name'
for file_name in filtered_df['sample_name'].unique():
    row_data = [file_name]
    for entry in filtered_entries:
        count = filtered_df[(filtered_df['sample_name'] == file_name) & (filtered_df['GNC'] == entry)].shape[0]
        row_data.append(count)
    pt.add_row(row_data)

# Set table title
pt.title = 'Summary Output'

# Print the PrettyTable
print(pt)


+-----------------------------------------------------------------------------------------------------------------------------------------+
|                                                              Summary Output                                                             |
+-----------------+--------+--------+--------+------+-----+------+--------+------+-------+-----+---------+-----+--------+--------+--------+
|   sample_name   | COL1A1 | COL1A2 | COL3A1 | K2C1 | KS9 | KS10 | COLunk | K22E | TRYUP | PIV | COL12A1 | ALB | COL6A2 | FibMod | COL5A1 |
+-----------------+--------+--------+--------+------+-----+------+--------+------+-------+-----+---------+-----+--------+--------+--------+
|      06_ICB     |  638   |  620   |  152   |  66  |  87 |  18  |  946   |  12  |   40  |  0  |   441   | 100 |  106   |  111   |   0    |
|      09_ICB     |  671   |  645   |   98   |  52  | 152 |  27  |  698   |  35  |   40  |  0  |   560   | 101 |   94   |  131   |   12   |
|      10_ICB     | 

In [76]:
# GNC to examine (replace with the GNC you want to analyze)
gnc_of_interest = 'COL3A1'

# Filter the DataFrame to only include the GNC of interest
gnc_df = group_df[group_df['GNC'] == gnc_of_interest]

# Print the GNC DataFrame
print(gnc_df)


          GNC sample_name  position aa1  n_aa1  aa1_score_mean  aa1_score_max  \
44447  COL3A1      06_ICB       224   G     46             NaN            NaN   
44448  COL3A1      06_ICB       225   E     46             NaN            NaN   
44449  COL3A1      06_ICB       226   M     46             NaN            NaN   
44450  COL3A1      06_ICB       227   G     46             NaN            NaN   
44451  COL3A1      06_ICB       228   P     46             NaN            NaN   
...       ...         ...       ...  ..    ...             ...            ...   
53181  COL3A1          P4       793   M      9             NaN            NaN   
53182  COL3A1          P4       794   P      9             NaN            NaN   
53183  COL3A1          P4       795   G      9             NaN            NaN   
53184  COL3A1          P4       796   S      6             NaN            NaN   
53185  COL3A1          P4       797   R      9             NaN            NaN   

       aa1_score_SD  score1

In [97]:
column_headers = gnc_df.columns.tolist()
print(column_headers)

['GNC', 'sample_name', 'position', 'aa1', 'n_aa1', 'aa1_score_mean', 'aa1_score_max', 'aa1_score_SD', 'score1_aa1', 'spectraId1_aa1', 'score2_aa1', 'spectraId2_aa1', 'score3_aa1', 'spectraId3_aa1', 'ptm1_aa1', '%ptm1_aa1', 'ptm1_aa1_score_mean', 'ptm1_aa1_score_max', 'ptm1_aa1_score_SD', 'score1_ptm1_aa1', 'spectraId1_ptm1_aa1', 'score2_ptm1_aa1', 'spectraId2_ptm1_aa1', 'score3_ptm1_aa1', 'spectraId3_ptm1_aa1', 'ptm2_aa1', '%ptm2_aa1', 'ptm2_aa1_score_mean', 'ptm2_aa1_score_max', 'ptm2_aa1_score_SD', 'score1_ptm2_aa1', 'spectraId1_ptm2_aa1', 'score2_ptm2_aa1', 'spectraId2_ptm2_aa1', 'score3_ptm2_aa1', 'spectraId3_ptm2_aa1', 'aa2', 'n_aa2', 'aa2_score_mean', 'aa2_score_max', 'aa2_score_SD', 'score1_aa2', 'spectraId1_aa2', 'score2_aa2', 'spectraId2_aa2', 'score3_aa2', 'spectraId3_aa2', 'ptm1_aa2', '%ptm1_aa2', 'ptm1_aa2_score_mean', 'ptm1_aa2_score_max', 'ptm1_aa2_score_SD', 'score1_ptm1_aa2', 'spectraId1_ptm1_aa2', 'score2_ptm1_aa2', 'spectraId2_ptm1_aa2', 'score3_ptm1_aa2', 'spectraId3

In [114]:
def analyze_protein_variations(df, gnc):
    # Filter for the specific GNC
    df = df[df['GNC'] == gnc]

    # Initialize result containers
    missing_aa_positions = []
    single_aa_positions = []
    multiple_aa_positions = []

    # Group by 'position' to analyze each position
    grouped = df.groupby('position')

    for position, group in grouped:
        # Identify missing amino acids
        if group[['aa1', 'aa2', 'aa3']].isna().all().all():
            missing_aa_positions.append(position)
            continue

        # Identify conserved amino acids (single AA across all samples)
        unique_aas = pd.concat([group['aa1'], group['aa2'], group['aa3']]).dropna().unique()
        if len(unique_aas) == 1:
            aa_count = group['n_aa1'].sum()  # Assuming 'n_aa1' counts occurrences of 'aa1'
            single_aa_positions.append((position, unique_aas[0], aa_count))
            continue

        # Analyze positions with multiple amino acids
        for aa_col in ['aa1', 'aa2', 'aa3']:
            for aa in group[aa_col].dropna().unique():
                aa_count = group[group[aa_col] == aa]['n_aa1'].sum()  # Assuming 'n_aa1' corresponds to 'aa1'
                top_score = group[group[aa_col] == aa][f'{aa_col}_score_max'].min()
                multiple_aa_positions.append((position, aa, aa_count, top_score))

    # Convert to DataFrames for easier handling
    single_aa_df = pd.DataFrame(single_aa_positions, columns=['Position', 'Amino Acid', 'Count'])
    multiple_aa_df = pd.DataFrame(multiple_aa_positions, columns=['Position', 'Amino Acid', 'Count', 'Top Score'])

    return missing_aa_positions, single_aa_df, multiple_aa_df

# Assuming 'df' is your DataFrame and 'COL1A2' is the GNC of interest
missing_aa_positions, single_aa_df, multiple_aa_df = analyze_protein_variations(gnc_df, 'COL3A1')

# Printing tables
# Missing Amino Acid Positions
print("Missing Amino Acid Positions:")
if missing_aa_positions:
    table = PrettyTable(['Position'])
    for position in missing_aa_positions:
        table.add_row([position])
    print(table)
else:
    print("No missing amino acid positions found.")

# Single Amino Acid Positions
print("\nSingle Amino Acid Positions:")
if not single_aa_df.empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count'])
    for row in single_aa_df.itertuples(index=False):
        table.add_row([row.position, row.aa1, row.countt])
    print(table)
else:
    print("No positions with single amino acids found.")


# # Print single amino acid positions
# print("\nSingle Amino Acid Positions:")
# if not protein_variations['Single AA Positions'].empty:
#     table = PrettyTable(['Position', 'Amino Acid', 'Count'])
#     for row in protein_variations['Sin# Single Amino Acid Positions
print("\nSingle Amino Acid Positions:")
if not single_aa_df.empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count'])
    for row in single_aa_df.itertuples(index=False):
        table.add_row([row.position, row.aa1, row.countt])
    print(table)
else:
    print("No positions with single amino acids found.")
gle AA Positions'].itertuples():
#         table.add_row([row.position, row.aa1, row.count])
#     print(table)
# else:
#     print("No positions with single amino acids found.")

# Multiple Amino Acid Positions
print("\nMultiple Amino Acid Positions:")
if not multiple_aa_df.empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count', 'Top Score'])
    for row in multiple_aa_df.itertuples(index=False):
        table.add_row([row.Position, row.Amino_Acid, row.Count, row.Top_Score])
    print(table)
else:
    print("No positions with multiple amino acids found.")


Missing Amino Acid Positions:
No missing amino acid positions found.

Single Amino Acid Positions:


AttributeError: 'Pandas' object has no attribute 'position'

In [104]:
import pandas as pd
from prettytable import PrettyTable

def analyze_protein_variations(gnc_df):
    # Ensure 'position' is not set as an index to avoid ambiguity
    gnc_df.reset_index(drop=True, inplace=True)

    # Group data by position
    grouped_by_position = gnc_df.groupby('position')

    # Analyze positions with missing amino acids
    # Filtering directly, so no ambiguity should arise here
    missing_aa_positions = gnc_df[gnc_df['aa1'].isna()].groupby('position').size().index.tolist()

    # Analyze positions with a single amino acid
    # This approach avoids ambiguity by working with grouped objects directly
    single_aa_summary = gnc_df.dropna(subset=['aa1']).groupby('position').filter(lambda x: x['aa1'].nunique() == 1)
    single_aa_summary = single_aa_summary.groupby(['position', 'aa1']).size().reset_index(name='count')

    # Analyze positions with multiple amino acids
    multiple_aa_positions = gnc_df.dropna(subset=['aa1']).groupby('position').filter(lambda x: x['aa1'].nunique() > 1)
    multiple_aa_analysis = multiple_aa_positions.groupby(['position', 'aa1']).size().reset_index(name='count')

    # Combine results
    summary = {
        'Missing AA Positions': missing_aa_positions,
        'Single AA Positions': single_aa_summary,
        'Multiple AA Positions': multiple_aa_analysis
    }

    return summary


# Assuming 'gnc_df' is your DataFrame filtered for a specific GNC of interest
protein_variations = analyze_protein_variations(gnc_df)

# Print missing amino acid positions
print("Missing Amino Acid Positions:")
if protein_variations['Missing AA Positions']:
    table = PrettyTable(['Position'])
    table.add_rows([[position] for position in protein_variations['Missing AA Positions']])
    print(table)
else:
    print("No missing amino acid positions found.")

# Print single amino acid positions
print("\nSingle Amino Acid Positions:")
if not protein_variations['Single AA Positions'].empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count'])
    for row in protein_variations['Single AA Positions'].itertuples():
        table.add_row([row.position, row.aa1, row.count])
    print(table)
else:
    print("No positions with single amino acids found.")

# Print multiple amino acid positions
print("\nMultiple Amino Acid Positions:")
if not protein_variations['Multiple AA Positions'].empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count', 'Top Score'])
    for row in protein_variations['Multiple AA Positions'].itertuples():
        table.add_row([row.position, row.aa1, row.count, row.top_score])
    print(table)
else:
    print("No positions with multiple amino acids found.")


Missing Amino Acid Positions:
No missing amino acid positions found.

Single Amino Acid Positions:
+----------+------------+-------+
| Position | Amino Acid | Count |
+----------+------------+-------+
|    42    |     G      |   1   |
|    43    |     P      |   1   |
|    44    |     P      |   1   |
|    45    |     G      |   1   |
|    46    |     P      |   1   |
|    47    |     P      |   1   |
|    48    |     G      |   1   |
|    49    |     A      |   1   |
|    50    |     I      |   1   |
|    51    |     G      |   1   |
|    52    |     P      |   1   |
|    53    |     S      |   1   |
|    54    |     G      |   1   |
|    55    |     P      |   1   |
|    56    |     A      |   1   |
|    57    |     G      |   1   |
|    58    |     K      |   1   |
|    72    |     G      |   1   |
|    73    |     G      |   12  |
|    74    |     F      |   12  |
|    75    |     P      |   12  |
|    76    |     G      |   12  |
|    77    |     P      |   12  |
|    78    |     

AttributeError: 'Pandas' object has no attribute 'top_score'

In [101]:
import pandas as pd

def analyze_protein_variations(gnc_df):
    # Group data by position
    grouped_by_position = gnc_df.groupby('position')

    # Initialize containers for the analysis results
    missing_aa_positions = []
    single_aa_summary = []
    multiple_aa_analysis = []

    # Analyze each position
    for position, group in grouped_by_position:
        # Identify missing amino acids by checking if all values in 'aa1' are NaN
        if group['aa1'].isna().all():
            missing_aa_positions.append(position)
            continue  # Move to the next position if the current one has no data

        # Analyze positions with a single amino acid
        if group['aa1'].nunique() == 1:
            aa = group['aa1'].dropna().unique()[0]
            count = group['n_aa1'].sum()  # Summing up occurrences across all samples
            single_aa_summary.append({'position': position, 'amino_acid': aa, 'count': count})
        else:
            # Analyze positions with multiple amino acids
            aa_counts = group['aa1'].value_counts().reset_index()
            aa_counts.columns = ['amino_acid', 'count']
            for _, row in aa_counts.iterrows():
                multiple_aa_analysis.append({'position': position, 'amino_acid': row['amino_acid'], 'count': row['count']})

    # Convert analysis results into DataFrame for easy handling
    missing_aa_df = pd.DataFrame(missing_aa_positions, columns=['Position'])
    single_aa_df = pd.DataFrame(single_aa_summary)
    multiple_aa_df = pd.DataFrame(multiple_aa_analysis)

    # Compile the summary into a dictionary
    summary = {
        'Missing AA Positions': missing_aa_df,
        'Single AA Positions': single_aa_df,
        'Multiple AA Positions': multiple_aa_df
    }

    return summary

# Assuming 'gnc_df' is your DataFrame filtered for a specific GNC
# Example usage:
# protein_variations = analyze_protein_variations(gnc_df)
# The 'protein_variations' dictionary now contains three keys with DataFrames as their values.


In [102]:
# Print missing amino acid positions
print("Missing Amino Acid Positions:")
if protein_variations['Missing AA Positions']:
  table = PrettyTable(['Position'])
  table.add_rows([[position] for position in protein_variations['Missing AA Positions']])
  print(table)
else:
  print("No missing amino acid positions found.")

# Print single amino acid positions
print("\nSingle Amino Acid Positions:")
if protein_variations['Single AA Positions'].any():
  table = PrettyTable(['Position', 'Amino Acid', 'Count'])
  table.add_rows(protein_variations['Single AA Positions'].to_records(index=False))
  print(table)
else:
  print("No positions with single amino acids found.")

# Print multiple amino acid positions
print("\nMultiple Amino Acid Positions:")
if protein_variations['Multiple AA Positions'].any():
  table = PrettyTable(['Position', 'Amino Acid', 'Count', 'Top Score'])
  table.add_rows(protein_variations['Multiple AA Positions'].to_records(index=False))
  print(table)
else:
  print("No positions with multiple amino acids found.")

Missing Amino Acid Positions:


NameError: name 'protein_variations' is not defined

In [100]:
def analyze_protein_data(df, gnc):
    # Filter the DataFrame for the specified GNC
    df_gnc = group_df[df['GNC'] == gnc]

    # Prepare the output DataFrame structure
    columns = ['position', 'sample_name', 'amino_acid', 'count', 'score1', 'PTM_type', 'PTM_score']
    analysis_results = pd.DataFrame(columns=columns)

    # Iterate through each unique position
    for position in df_gnc['position'].unique():
        df_position = df_gnc[df_gnc['position'] == position]

        # Iterate through each sample_name
        for sample in df_position['sample_name'].unique():
            df_sample = df_position[df_position['sample_name'] == sample]

            # Analyze amino acids and PTMs for this sample and position
            for aa_col in ['aa1', 'aa2', 'aa3']:
                aa_data = df_sample[[aa_col, f'n_{aa_col}', f'score1_{aa_col}']].dropna()

                for _, row in aa_data.iterrows():
                    amino_acid = row[aa_col]
                    count = row[f'n_{aa_col}']
                    score1 = row[f'score1_{aa_col}']

                    # Add PTM analysis here if applicable
                    ptm_types = []  # Placeholder for actual PTM analysis
                    ptm_scores = []  # Placeholder for actual PTM analysis

                    # Append to results DataFrame
                    result_row = pd.DataFrame([[position, sample, amino_acid, count, score1, ptm_types, ptm_scores]], columns=columns)
                    analysis_results = pd.concat([analysis_results, result_row], ignore_index=True)

    return analysis_results

# Assuming 'df' is your DataFrame and 'COL3A1' is the GNC of interest
results_df = analyze_protein_data(group_df, 'COL3A1')


In [95]:
def analyze_protein_variations(gnc_df):
    # Group data by position
    grouped_by_position = gnc_df.groupby('position')

    # Analyze positions with missing amino acids
    missing_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].isna().any())['position'].tolist()

    # Analyze positions with a single amino acid
    single_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() == 1)
    single_aa_summary = single_aa_positions[['position', 'aa1']].value_counts().reset_index(name='count')

    # Analyze positions with multiple amino acids
    multiple_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() > 1)
    multiple_aa_analysis = []
    if not multiple_aa_positions.empty:  # Check if there are positions with multiple amino acids
        for (position, idx), group_data in multiple_aa_positions.groupby(level=0):
            # Check for any non-missing amino acid column (handle potential MultiIndex)
            amino_acid_cols = ['aa1', 'aa2', 'aa3']  # Assuming amino acid columns
            for aa_col in amino_acid_cols:
                if not group_data[aa_col].isna().all():
                    aa = group_data[aa_col].dropna().iloc[0]  # Get the first non-missing amino acid
                    count = group_data[aa_col].value_counts().get(aa, 0)  # Count occurrences

                    multiple_aa_analysis.append({'position': position, 'aa1': aa, 'count': count})
                    break  # Exit the inner loop after finding a non-missing amino acid column

    # Combine results
    summary = {
        'Missing AA Positions': missing_aa_positions,
        'Single AA Positions': single_aa_summary,
        'Multiple AA Positions': multiple_aa_analysis
    }

    summary = {
        'Missing AA Positions': missing_aa_positions,
        'Single AA Positions': single_aa_summary,
        'Multiple AA Positions': multiple_aa_analysis
    }
    return summary

In [96]:
protein_variations = analyze_protein_variations(gnc_df)

TypeError: cannot unpack non-iterable int object

In [91]:
def analyze_protein_variations(gnc_df):
    # Group data by position
    grouped_by_position = gnc_df.groupby('position')

    # Analyze positions with missing amino acids
    missing_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].isna().all())['position'].tolist()

    # Analyze positions with a single amino acid
    single_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() == 1)
    single_aa_summary = single_aa_positions[['position', 'aa1']].value_counts().reset_index(name='count')

    # Analyze positions with multiple amino acids
    multiple_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() > 1)
    multiple_aa_analysis = []
    if not multiple_aa_positions.empty:  # Check if there are positions with multiple amino acids
        for (position, idx), group_data in multiple_aa_positions:
            # Check for any non-missing amino acid column (handle potential MultiIndex)
            amino_acid_cols = ['aa1', 'aa2', 'aa3']  # Assuming amino acid columns
            for aa_col in amino_acid_cols:
                if not group_data[aa_col].isna().all():
                    aa = group_data[aa_col].dropna().iloc[0]  # Get the first non-missing amino acid
                    count = group_data[aa_col].value_counts().get(aa, 0)  # Count occurrences

                    # Access PTM scores dynamically based on the amino acid column
                    ptm_scores = [group_data[f'ptm{i}_{aa_col}'].dropna().iloc[idx] for i in range(1, len(group_data.columns) // 2 + 1) if not pd.isna(group_data[f'ptm{i}_{aa_col}'].iloc[idx])]
                    top_score = max(ptm_scores) if ptm_scores else None

                    multiple_aa_analysis.append({'position': position, 'aa1': aa, 'count': count, 'top_score': top_score})
                    break  # Exit the inner loop after finding a non-missing amino acid column

    # Combine results
    summary = {
        'Missing AA Positions': missing_aa_positions,
        'Single AA Positions': single_aa_summary,
        'Multiple AA Positions': pd.DataFrame(multiple_aa_analysis)
    }
    return summary

# Analyze variations in the protein
protein_variations = analyze_protein_variations(gnc_df.copy())

ValueError: too many values to unpack (expected 2)

In [83]:

# # Function to analyze variations in a protein
# def analyze_protein_variations(gnc_df):
#   # Group data by position
#   grouped_by_position = gnc_df.groupby('position')

#   # Analyze positions with missing amino acids
#   missing_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].isna().any())['position'].tolist()

#   # Analyze positions with a single amino acid
#   single_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() == 1)
#   single_aa_summary = single_aa_positions[['position', 'aa1']].value_counts().reset_index(name='count')

#   # Analyze positions with multiple amino acids
#   multiple_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() > 1)
#   multiple_aa_analysis = []
#   for position, group_data in multiple_aa_positions:
#     for aa_col in ['aa1', 'aa2', 'aa3']:  # Assuming you have 3 amino acid columns (aa1, aa2, aa3)
#       if not pd.isna(group_data[aa_col].any()):  # Check if there's data in this column
#         aa = group_data[aa_col].iloc[0]  # Assuming the first non-null value is the amino acid
#         count = group_data[aa_col].value_counts().get(aa, 0)  # Count occurrences (default to 0 if not present)
#         ptm_scores = [group_data[f'ptm{i}_score1'].iloc[idx] for i in range(1, len(group_data.columns) // 2 + 1) if not pd.isna(group_data[f'ptm{i}_{aa_col}'].iloc[idx])]
#         top_score = max(ptm_scores) if ptm_scores else None
#         multiple_aa_analysis.append({'position': position, 'aa1': aa, 'count': count, 'top_score': top_score})


#   # Combine results
#   summary = {
#       'Missing AA Positions': missing_aa_positions,
#       'Single AA Positions': single_aa_summary,
#       'Multiple AA Positions': multiple_aa_df
#   }
#   return summary

# # Analyze variations in the protein
# protein_variations = analyze_protein_variations(gnc_df.copy())

# Print the summary tables (assuming PrettyTable is available)
from prettytable import PrettyTable

# Print missing amino acid positions
print("Missing Amino Acid Positions:")
if protein_variations['Missing AA Positions']:
  table = PrettyTable(['Position'])
  table.add_rows([[position] for position in protein_variations['Missing AA Positions']])
  print(table)
else:
  print("No missing amino acid positions found.")

# Print single amino acid positions
print("\nSingle Amino Acid Positions:")
if protein_variations['Single AA Positions'].any():
  table = PrettyTable(['Position', 'Amino Acid', 'Count'])
  table.add_rows(protein_variations['Single AA Positions'].to_records(index=False))
  print(table)
else:
  print("No positions with single amino acids found.")

# Print multiple amino acid positions
print("\nMultiple Amino Acid Positions:")
if protein_variations['Multiple AA Positions'].any():
  table = PrettyTable(['Position', 'Amino Acid', 'Count', 'Top Score'])
  table.add_rows(protein_variations['Multiple AA Positions'].to_records(index=False))
  print(table)
else:
  print("No positions with multiple amino acids found.")


ValueError: too many values to unpack (expected 2)

In [ ]:
# # Filter the DataFrame to find rows with ptm "Deamidated"
# filtered_ptm_df = df_filtered[df_filtered['ptm'] == "Deamidated"]

# # Check if any rows were found
# if not filtered_ptm_df.empty:
#   # Print a limited number of rows (e.g., the first 3)
#   print(filtered_ptm_df.head(3))
# else:
#   # If no rows are found, print a message
#   print("No rows found with ptm='Deamidated'")

In [ ]:
# unique_names = result_with_ptms['sample_name'].unique()
# name_counts = result_with_ptms['sample_name'].value_counts()
# print("Unique sample names:", unique_names)
# print("Counts of each sample name:", name_counts)